In [ ]:
#!jupyter nbconvert --to script traffic_data_preprocess.ipynb

[NbConvertApp] Converting notebook traffic_data_preprocess.ipynb to script
[NbConvertApp] Writing 94355 bytes to traffic_data_preprocess.py


## Import Libraries and traffic_data_preprocessor.py

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from traffic_data_preprocessor import *

import keras_tuner as kt
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.layers import LSTM, GRU, Dense, Dropout, BatchNormalization, Input, Dense, Dropout, MultiHeadAttention, LayerNormalization, Flatten
from tensorflow.keras.optimizers import Adam, AdamW
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.callbacks import EarlyStopping

## Functions to Create Prediction Models

### LSTM

In [6]:
def create_basic_lstm_model(df_scaled, no_columns, n_steps=24, epochs=50, batch_size=32):

    # Create input output pairs 
    # Consider df_scaled has 2184 rows n_steps = 24
    # X is  X[0], X[1], X[2], ... , X[2159]
    # y is y[0], y[1], y[2], ... , y[2159] 
    def create_sequences(data, n_steps):
        X, y = [], []
        for i in range(len(data) - n_steps):
            X.append(data[i:i + n_steps])  # Use n_steps past data
            y.append(data[i + n_steps])    # Predict next step
        return np.array(X), np.array(y)
    
    X, y = create_sequences(df_scaled, n_steps)

    model = Sequential()
    model.add(LSTM(100, activation='relu', input_shape=(n_steps, no_columns), return_sequences=True))
    model.add(LSTM(50, activation='relu'))
    model.add(Dense(no_columns))
    model.compile(optimizer="adam", loss="mse")


    # X[0], X[1], ... , X[1727] for X_train
    # X[1728], X[1729], ... , X[2159] for X_test
    # y[0], y[1], ... , X[1727] for X_train
    # y[1728], y[1729], ... , X[2159] for X_test   
    split = int(0.8 * len(X))
    X_train, X_test = X[:split], X[split:]
    y_train, y_test = y[:split], y[split:]

   # Train the model
   # model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test, y_test))

    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test, y_test), callbacks=[early_stopping])

    return model 

In [7]:
def create_lstm_model(df_scaled, no_columns, lstm_model, n_steps=24, epochs=50, batch_size=32):
    # Function to create input-output pairs
    def create_sequences(data, n_steps):
        X, y = [], []
        for i in range(len(data) - n_steps):
            X.append(data[i:i + n_steps])  # Use n_steps past data
            y.append(data[i + n_steps])    # Predict next step
        return np.array(X), np.array(y)

    # Function to build model
    def build_model(hp, no_columns, n_steps):
        model = Sequential()
        # First LSTM layer
        model.add(LSTM(
            units=hp.Int('units_1', min_value=50, max_value=200, step=50),
            activation='relu',
            input_shape=(n_steps, no_columns),
            return_sequences=True
        ))
        # Second LSTM layer
        model.add(LSTM(
            units=hp.Int('units_2', min_value=50, max_value=200, step=50),
            activation='relu'
        ))
        # Dropout layer
        model.add(Dropout(hp.Float('dropout_rate', min_value=0.1, max_value=0.5, step=0.1)))
        # Dense output layer
        model.add(Dense(no_columns))
        # Compile model
        model.compile(
            optimizer=Adam(learning_rate=hp.Choice('learning_rate', [0.001, 0.0001, 0.01])),
            loss='mse'
        )
        return model

    # Create X, y sequence
    X, y = create_sequences(df_scaled, n_steps)

    # Splitting data into training and testing sets (80-20 split)
    split = int(0.8 * len(X))
    X_train, X_test = X[:split], X[split:]
    y_train, y_test = y[:split], y[split:]

    # Define Keras Tuner
    tuner = kt.RandomSearch(
        lambda hp: build_model(hp, no_columns, n_steps),
        objective='val_loss',
        max_trials=10,
        executions_per_trial=2,
        directory=f'tuner_data/{lstm_model}', 
        project_name='lstm_tuning'
    )

    # Perform hyperparameter tuning
    tuner.search(X_train, y_train, epochs=50, validation_data=(X_test, y_test), batch_size=batch_size)

    # Get the best hyperparameters
    best_hp = tuner.get_best_hyperparameters(num_trials=1)[0]

    # Save best hyperparameters to a file
    with open(f"best_hp/best_hp_{lstm_model}.txt", "w") as f:
        f.write("Best Hyperparameters Found:\n")
        f.write(f" - Units Layer 1: {best_hp.get('units_1')}\n")
        f.write(f" - Units Layer 2: {best_hp.get('units_2')}\n")
        f.write(f" - Dropout Rate: {best_hp.get('dropout_rate')}\n")
        f.write(f" - Learning Rate: {best_hp.get('learning_rate')}\n")

    # Print the best hyperparameters
    print("\nBest Hyperparameters Found:")
    print(f" - Units Layer 1: {best_hp.get('units_1')}")
    print(f" - Units Layer 2: {best_hp.get('units_2')}")
    print(f" - Dropout Rate: {best_hp.get('dropout_rate')}")
    print(f" - Learning Rate: {best_hp.get('learning_rate')}")

    # Build the best model
    model = tuner.hypermodel.build(best_hp)

    # Early stopping callback
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

    # Train the model
    model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test, y_test), callbacks=[early_stopping])

    return model  # Return the trained model

In [8]:
def create_lstm_model(df_scaled, no_columns, n_steps=24, epochs=50, batch_size=32, 
                      units_1=50, units_2=50, dropout_rate=0.2, learning_rate=0.01):

    # Create input output pairs 
    # Consider df_scaled has 2184 rows n_steps = 24
    # X is  X[0], X[1], X[2], ... , X[2159]
    # y is y[0], y[1], y[2], ... , y[2159] 
    def create_sequences(data, n_steps):
        X, y = [], []
        for i in range(len(data) - n_steps):
            X.append(data[i:i + n_steps])  # Use n_steps past data
            y.append(data[i + n_steps])    # Predict next step
        return np.array(X), np.array(y)
    
    X, y = create_sequences(df_scaled, n_steps)

    model = Sequential()
    # LSTM layer 1
    model.add(LSTM(units_1, activation='relu', input_shape=(n_steps, no_columns), return_sequences=True))
    model.add(Dropout(dropout_rate))
    model.add(BatchNormalization())

    #LSTM layer 2
    model.add(LSTM(units_2, activation='relu'))
    model.add(Dropout(dropout_rate))
    # Output layer
    model.add(Dense(no_columns))

    # Compile the model with a custom learning rate
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss="mse", metrics=['mae'])

    # X[0], X[1], ... , X[1727] for X_train
    # X[1728], X[1729], ... , X[2159] for X_test
    # y[0], y[1], ... , X[1727] for X_train
    # y[1728], y[1729], ... , X[2159] for X_test   
    split = int(0.8 * len(X))
    X_train, X_test = X[:split], X[split:]
    y_train, y_test = y[:split], y[split:]

    # With Early stopping to prevent overfitting
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test, y_test), callbacks=[early_stopping])
    
   # Without Early stopping
   # model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test, y_test))


    return model 

### GRU

In [9]:
def create_gru_model(df_scaled, no_columns, n_steps=24, epochs=50, batch_size=32, 
                     units_1=50, units_2=50, dropout_rate=0.2, learning_rate=0.01):

    # Create input-output pairs
    def create_sequences(data, n_steps):
        X, y = [], []
        for i in range(len(data) - n_steps):
            X.append(data[i:i + n_steps])
            y.append(data[i + n_steps])
        return np.array(X), np.array(y)
    
    X, y = create_sequences(df_scaled, n_steps)

    model = Sequential()
    # GRU layer 1
    model.add(GRU(units_1, activation='relu', input_shape=(n_steps, no_columns), return_sequences=True))
    model.add(Dropout(dropout_rate))
    model.add(BatchNormalization())

    # GRU layer 2
    model.add(GRU(units_2, activation='relu'))
    model.add(Dropout(dropout_rate))

    # Output layer
    model.add(Dense(no_columns))

    # Compile model
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss="mse", metrics=['mae'])

    # Train-test split
    split = int(0.8 * len(X))
    X_train, X_test = X[:split], X[split:]
    y_train, y_test = y[:split], y[split:]

    # Early stopping
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size,
              validation_data=(X_test, y_test), callbacks=[early_stopping])

    return model


### Transformer

In [10]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, LayerNormalization, MultiHeadAttention, Dropout, Add
from tensorflow.keras.callbacks import EarlyStopping

def create_transformer_model(df_scaled, no_columns, n_steps=24, epochs=50, batch_size=32,
                             head_size=64, num_heads=2, ff_dim=128, num_blocks=2,
                             dropout=0.1, learning_rate=0.001):

    def create_sequences(data, n_steps):
        X, y = [], []
        for i in range(len(data) - n_steps):
            X.append(data[i:i + n_steps])
            y.append(data[i + n_steps])
        return np.array(X), np.array(y)

    X, y = create_sequences(df_scaled, n_steps)

    inputs = Input(shape=(n_steps, no_columns))
    x = inputs

    for _ in range(num_blocks):
        attn_output = MultiHeadAttention(num_heads=num_heads, key_dim=head_size)(x, x)
        x = Add()([x, attn_output])
        x = LayerNormalization(epsilon=1e-6)(x)

        ffn = Dense(ff_dim, activation='relu')(x)
        ffn = Dense(no_columns)(ffn)
        x = Add()([x, ffn])
        x = LayerNormalization(epsilon=1e-6)(x)

    x = Dense(no_columns)(x[:, -1])  # Only use final timestep

    model = Model(inputs, x)
    model.compile(optimizer=Adam(learning_rate), loss='mse', metrics=['mae'])

    split = int(0.8 * len(X))
    X_train, X_test = X[:split], X[split:]
    y_train, y_test = y[:split], y[split:]

    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size,
              validation_data=(X_test, y_test), callbacks=[early_stopping])

    return model

## Load Highest Correlated Models

In [4]:
selected_stations_1r95_1  = ["TS40"]
selected_stations_1r95_2  = ["TS40", "TS36"]
selected_stations_1r95_3  = ["TS40", "TS36", "TS37"]
selected_stations_1r95_4  = ["TS40", "TS36", "TS37", "TS21"]
selected_stations_1r95_5  = ["TS40", "TS36", "TS37", "TS21", "TS6"]
selected_stations_1r95_6  = ["TS40", "TS36", "TS37", "TS21", "TS6", "TS15"]
selected_stations_1r95_7  = ["TS40", "TS36", "TS37", "TS21", "TS6", "TS15", "TS16"]
selected_stations_1r95_8  = ["TS40", "TS36", "TS37", "TS21", "TS6", "TS15", "TS16", "TS4"]
selected_stations_1r95_9  = ["TS40", "TS36", "TS37", "TS21", "TS6", "TS15", "TS16", "TS4", "TS13"]
selected_stations_1r95_10  = ["TS40", "TS36", "TS37", "TS21", "TS6", "TS15", "TS16", "TS4", "TS13", "TS22"]
selected_stations_1r95_20  = ["TS40", "TS36", "TS37", "TS21", "TS6" , "TS15", "TS16", "TS4" , "TS13", "TS22", 
                              "TS33", "TS1" , "TS11", "TS19", "TS28", "TS32", "TS45", "TS57", "TS2" , "TS1" ]
selected_stations_1r95_30  = ["TS40", "TS36", "TS37", "TS21", "TS6" , "TS15", "TS16", "TS4" , "TS13", "TS22", 
                              "TS33", "TS1" , "TS11", "TS19", "TS28", "TS32", "TS45", "TS57", "TS2" , "TS3" ,
                              "TS5" , "TS7" , "TS8" , "TS9" , "TS12", "TS14", "TS17", "TS18", "TS20", "TS23"]
selected_stations_1r95_40  = ["TS40", "TS36", "TS37", "TS21", "TS6" , "TS15", "TS16", "TS4" , "TS13", "TS22", 
                              "TS33", "TS1" , "TS11", "TS19", "TS28", "TS32", "TS45", "TS57", "TS2" , "TS3" ,
                              "TS5" , "TS7" , "TS8" , "TS9" , "TS12", "TS14", "TS17", "TS18", "TS20", "TS23",
                              "TS24", "TS25", "TS26", "TS27", "TS29", "TS30", "TS31", "TS34", "TS38", "TS39"]
selected_stations_1r95_50  = ["TS40", "TS36", "TS37", "TS21", "TS6" , "TS15", "TS16", "TS4" , "TS13", "TS22", 
                              "TS33", "TS1" , "TS11", "TS19", "TS28", "TS32", "TS45", "TS57", "TS2" , "TS3" ,
                              "TS5" , "TS7" , "TS8" , "TS9" , "TS12", "TS14", "TS17", "TS18", "TS20", "TS23",
                              "TS24", "TS25", "TS26", "TS27", "TS29", "TS30", "TS31", "TS34", "TS38", "TS39",
                              "TS41", "TS42", "TS43", "TS44", "TS46", "TS47", "TS48", "TS49", "TS50", "TS51"]
selected_stations_1r95_60  = ["TS40", "TS36", "TS37", "TS21", "TS6" , "TS15", "TS16", "TS4" , "TS13", "TS22", 
                              "TS33", "TS1" , "TS11", "TS19", "TS28", "TS32", "TS45", "TS57", "TS2" , "TS3" ,
                              "TS5" , "TS7" , "TS8" , "TS9" , "TS12", "TS14", "TS17", "TS18", "TS20", "TS23",
                              "TS24", "TS25", "TS26", "TS27", "TS29", "TS30", "TS31", "TS34", "TS38", "TS39",
                              "TS41", "TS42", "TS43", "TS44", "TS46", "TS47", "TS48", "TS49", "TS50", "TS51",
                              "TS52", "TS53", "TS54", "TS55", "TS56", "TS58", "TS59", "TS60"]


df_selected_1r95_1 = df_all_train_value_scaled[selected_stations_1r95_1].values #convert to array
df_selected_1r95_2 = df_all_train_value_scaled[selected_stations_1r95_2].values #convert to array
df_selected_1r95_3 = df_all_train_value_scaled[selected_stations_1r95_3].values #convert to array
df_selected_1r95_4 = df_all_train_value_scaled[selected_stations_1r95_4].values #convert to array
df_selected_1r95_5 = df_all_train_value_scaled[selected_stations_1r95_5].values #convert to array
df_selected_1r95_6 = df_all_train_value_scaled[selected_stations_1r95_6].values #convert to array
df_selected_1r95_7 = df_all_train_value_scaled[selected_stations_1r95_7].values #convert to array
df_selected_1r95_8 = df_all_train_value_scaled[selected_stations_1r95_8].values #convert to array
df_selected_1r95_9 = df_all_train_value_scaled[selected_stations_1r95_9].values #convert to array
df_selected_1r95_10 = df_all_train_value_scaled[selected_stations_1r95_10].values #convert to array
df_selected_1r95_20 = df_all_train_value_scaled[selected_stations_1r95_20].values #convert to array
df_selected_1r95_30 = df_all_train_value_scaled[selected_stations_1r95_30].values #convert to array
df_selected_1r95_40 = df_all_train_value_scaled[selected_stations_1r95_40].values #convert to array
df_selected_1r95_50 = df_all_train_value_scaled[selected_stations_1r95_50].values #convert to array
df_selected_1r95_60 = df_all_train_value_scaled[selected_stations_1r95_60].values #convert to array

In [44]:
selected_stations_1r90_1 = ["TS19"]
selected_stations_1r90_2 = ["TS19", "TS40"]
selected_stations_1r90_3 = ["TS19", "TS40", "TS36"]
selected_stations_1r90_4 = ["TS19", "TS40", "TS36", "TS6"]
selected_stations_1r90_5 = ["TS19", "TS40", "TS36", "TS6", "TS21"]
selected_stations_1r90_6 = ["TS19", "TS40", "TS36", "TS6", "TS21", "TS37"]
selected_stations_1r90_7 = ["TS19", "TS40", "TS36", "TS6", "TS21", "TS37", "TS47"]
selected_stations_1r90_8 = ["TS19", "TS40", "TS36", "TS6", "TS21", "TS37", "TS47", "TS45"]
selected_stations_1r90_9 = ["TS19", "TS40", "TS36", "TS6", "TS21", "TS37", "TS47", "TS45", "TS16"]
selected_stations_1r90_10 = ["TS19", "TS40", "TS36", "TS6", "TS21", "TS37", "TS47", "TS45", "TS16", "TS55"]
selected_stations_1r90_30  = ["TS19", "TS40", "TS36", "TS6", "TS21", "TS37", "TS47", "TS45", "TS16", "TS55",
                              "TS57", "TS15", "TS28", "TS42", "TS13", "TS14", "TS41", "TS46", "TS54", "TS1", 
                              "TS17", "TS33", "TS52", "TS32", "TS22", "TS30", "TS50", "TS4",  "TS44", "TS58"]

selected_stations_1r90_60  = ["TS40", "TS36", "TS37", "TS21", "TS6" , "TS15", "TS16", "TS4" , "TS13", "TS22", 
                              "TS33", "TS1" , "TS11", "TS19", "TS28", "TS32", "TS45", "TS57", "TS2" , "TS3" ,
                              "TS5" , "TS7" , "TS8" , "TS9" , "TS12", "TS14", "TS17", "TS18", "TS20", "TS23",
                              "TS24", "TS25", "TS26", "TS27", "TS29", "TS30", "TS31", "TS34", "TS38", "TS39",
                              "TS41", "TS42", "TS43", "TS44", "TS46", "TS47", "TS48", "TS49", "TS50", "TS51",
                              "TS52", "TS53", "TS54", "TS55", "TS56", "TS58", "TS59", "TS60"]

df_selected_1r90_1 = df_all_train_value_scaled[selected_stations_1r90_1].values #convert to array
df_selected_1r90_2 = df_all_train_value_scaled[selected_stations_1r90_2].values #convert to array
df_selected_1r90_3 = df_all_train_value_scaled[selected_stations_1r90_3].values #convert to array
df_selected_1r90_4 = df_all_train_value_scaled[selected_stations_1r90_4].values #convert to array
df_selected_1r90_5 = df_all_train_value_scaled[selected_stations_1r90_5].values #convert to array
df_selected_1r90_6 = df_all_train_value_scaled[selected_stations_1r90_6].values #convert to array
df_selected_1r90_7 = df_all_train_value_scaled[selected_stations_1r90_7].values #convert to array
df_selected_1r90_8 = df_all_train_value_scaled[selected_stations_1r90_8].values #convert to array
df_selected_1r90_9 = df_all_train_value_scaled[selected_stations_1r90_9].values #convert to array
df_selected_1r90_10 = df_all_train_value_scaled[selected_stations_1r90_10].values #convert to array
df_selected_1r90_30 = df_all_train_value_scaled[selected_stations_1r90_30].values #convert to array
df_selected_1r90_60 = df_all_train_value_scaled[selected_stations_1r90_60].values #convert to array

In [45]:
selected_stations_1r85_1  = ["TS16"]
selected_stations_1r85_2  = ["TS16", "TS19"]
selected_stations_1r85_3  = ["TS16", "TS19", "TS36"]
selected_stations_1r85_4  = ["TS16", "TS19", "TS36", "TS37"]
selected_stations_1r85_5  = ["TS16", "TS19", "TS36", "TS37", "TS40"]
selected_stations_1r85_6  = ["TS16", "TS19", "TS36", "TS37", "TS40", "TS47"]
selected_stations_1r85_7  = ["TS16", "TS19", "TS36", "TS37", "TS40", "TS47", "TS33"]
selected_stations_1r85_8  = ["TS16", "TS19", "TS36", "TS37", "TS40", "TS47", "TS33", "TS44"]
selected_stations_1r85_9  = ["TS16", "TS19", "TS36", "TS37", "TS40", "TS47", "TS33", "TS44", "TS4"]
selected_stations_1r85_10  = ["TS16", "TS19", "TS36", "TS37", "TS40", "TS47", "TS33", "TS44", "TS4", "TS13"]
selected_stations_1r85_30 =  ["TS16", "TS19", "TS36", "TS37", "TS40", "TS47", "TS33", "TS44", "TS4", "TS13", 
                              "TS45", "TS54", "TS17", "TS14", "TS22", "TS55", "TS59", "TS3" , "TS21", "TS57",
                              "TS1" , "TS28", "TS52", "TS6" , "TS30", "TS41", "TS42", "TS32", "TS51", "TS15"]

selected_stations_1r85_60  = ["TS40", "TS36", "TS37", "TS21", "TS6" , "TS15", "TS16", "TS4" , "TS13", "TS22", 
                              "TS33", "TS1" , "TS11", "TS19", "TS28", "TS32", "TS45", "TS57", "TS2" , "TS3" ,
                              "TS5" , "TS7" , "TS8" , "TS9" , "TS12", "TS14", "TS17", "TS18", "TS20", "TS23",
                              "TS24", "TS25", "TS26", "TS27", "TS29", "TS30", "TS31", "TS34", "TS38", "TS39",
                              "TS41", "TS42", "TS43", "TS44", "TS46", "TS47", "TS48", "TS49", "TS50", "TS51",
                              "TS52", "TS53", "TS54", "TS55", "TS56", "TS58", "TS59", "TS60"]

df_selected_1r85_1 = df_all_train_value_scaled[selected_stations_1r85_1].values #convert to array
df_selected_1r85_2 = df_all_train_value_scaled[selected_stations_1r85_2].values #convert to array
df_selected_1r85_3 = df_all_train_value_scaled[selected_stations_1r85_3].values #convert to array
df_selected_1r85_4 = df_all_train_value_scaled[selected_stations_1r85_4].values #convert to array
df_selected_1r85_5 = df_all_train_value_scaled[selected_stations_1r85_5].values #convert to array
df_selected_1r85_6 = df_all_train_value_scaled[selected_stations_1r85_6].values #convert to array
df_selected_1r85_7 = df_all_train_value_scaled[selected_stations_1r85_7].values #convert to array
df_selected_1r85_8 = df_all_train_value_scaled[selected_stations_1r85_8].values #convert to array
df_selected_1r85_9 = df_all_train_value_scaled[selected_stations_1r85_9].values #convert to array
df_selected_1r85_10 = df_all_train_value_scaled[selected_stations_1r85_10].values #convert to array
df_selected_1r85_30 = df_all_train_value_scaled[selected_stations_1r85_30].values #convert to array
df_selected_1r85_60 = df_all_train_value_scaled[selected_stations_1r85_60].values #convert to array

In [46]:
selected_stations_1r80_1 = ["TS22"]
selected_stations_1r80_2 = ["TS22", "TS19"]
selected_stations_1r80_3 = ["TS22", "TS19", "TS33"]
selected_stations_1r80_4 = ["TS22", "TS19", "TS33", "TS4"]
selected_stations_1r80_5 = ["TS22", "TS19", "TS33", "TS4", "TS16"]
selected_stations_1r80_6 = ["TS22", "TS19", "TS33", "TS4", "TS16", "TS32"]
selected_stations_1r80_7 = ["TS22", "TS19", "TS33", "TS4", "TS16", "TS32", "TS36"]
selected_stations_1r80_8 = ["TS22", "TS19", "TS33", "TS4", "TS16", "TS32", "TS36", "TS47"]
selected_stations_1r80_9 = ["TS22", "TS19", "TS33", "TS4", "TS16", "TS32", "TS36", "TS47", "TS54"]
selected_stations_1r80_10 = ["TS22", "TS19", "TS33", "TS4", "TS16", "TS32", "TS36", "TS47", "TS54", "TS3"]
selected_stations_1r80_30 = ["TS22", "TS19", "TS33", "TS4", "TS16", "TS32", "TS36", "TS47", "TS54", "TS3",
                             "TS13", "TS37", "TS21", "TS52", "TS55", "TS59", "TS14", "TS29", "TS40", "TS42",
                             "TS45", "TS44", "TS51", "TS57", "TS6", "TS28", "TS38", "TS1", "TS17", "TS46"]

selected_stations_1r80_60  = ["TS40", "TS36", "TS37", "TS21", "TS6" , "TS15", "TS16", "TS4" , "TS13", "TS22", 
                              "TS33", "TS1" , "TS11", "TS19", "TS28", "TS32", "TS45", "TS57", "TS2" , "TS3" ,
                              "TS5" , "TS7" , "TS8" , "TS9" , "TS12", "TS14", "TS17", "TS18", "TS20", "TS23",
                              "TS24", "TS25", "TS26", "TS27", "TS29", "TS30", "TS31", "TS34", "TS38", "TS39",
                              "TS41", "TS42", "TS43", "TS44", "TS46", "TS47", "TS48", "TS49", "TS50", "TS51",
                              "TS52", "TS53", "TS54", "TS55", "TS56", "TS58", "TS59", "TS60"]

df_selected_1r80_1 = df_all_train_value_scaled[selected_stations_1r80_1].values #convert to array
df_selected_1r80_2 = df_all_train_value_scaled[selected_stations_1r80_2].values #convert to array
df_selected_1r80_3 = df_all_train_value_scaled[selected_stations_1r80_3].values #convert to array
df_selected_1r80_4 = df_all_train_value_scaled[selected_stations_1r80_4].values #convert to array
df_selected_1r80_5 = df_all_train_value_scaled[selected_stations_1r80_5].values #convert to array
df_selected_1r80_6 = df_all_train_value_scaled[selected_stations_1r80_6].values #convert to array
df_selected_1r80_7 = df_all_train_value_scaled[selected_stations_1r80_7].values #convert to array
df_selected_1r80_8 = df_all_train_value_scaled[selected_stations_1r80_8].values #convert to array
df_selected_1r80_9 = df_all_train_value_scaled[selected_stations_1r80_9].values #convert to array
df_selected_1r80_10 = df_all_train_value_scaled[selected_stations_1r80_10].values #convert to array
df_selected_1r80_30 = df_all_train_value_scaled[selected_stations_1r80_30].values #convert to array
df_selected_1r80_60 = df_all_train_value_scaled[selected_stations_1r80_60].values #convert to array

In [47]:
#Stations for Training
selected_stations_1r75_1 = ["TS16"]
selected_stations_1r75_2 = ["TS16", "TS13"]
selected_stations_1r75_3 = ["TS16", "TS13", "TS19"]
selected_stations_1r75_4 = ["TS16", "TS13", "TS19", "TS22"]
selected_stations_1r75_5 = ["TS16", "TS13", "TS19", "TS22", "TS32"]
selected_stations_1r75_6 = ["TS16", "TS13", "TS19", "TS22", "TS32", "TS36"]
selected_stations_1r75_7 = ["TS16", "TS13", "TS19", "TS22", "TS32", "TS36", "TS47"]
selected_stations_1r75_8 = ["TS16", "TS13", "TS19", "TS22", "TS32", "TS36", "TS47", "TS4"]
selected_stations_1r75_9 = ["TS16", "TS13", "TS19", "TS22", "TS32", "TS36", "TS47", "TS4", "TS29"]
selected_stations_1r75_10 = ["TS16", "TS13", "TS19", "TS22", "TS32", "TS36", "TS47", "TS4", "TS29", "TS44"]
selected_stations_1r75_30 = ["TS16", "TS13", "TS19", "TS22", "TS32", "TS36", "TS47", "TS4", "TS29", "TS44",
                             "TS33", "TS54", "TS40", "TS55", "TS52", "TS3" , "TS30", "TS42", "TS45", "TS51",
                             "TS11", "TS17", "TS21", "TS28", "TS37", "TS38", "TS59", "TS1", "TS6", "TS14"]

selected_stations_1r75_60  = ["TS40", "TS36", "TS37", "TS21", "TS6" , "TS15", "TS16", "TS4" , "TS13", "TS22", 
                              "TS33", "TS1" , "TS11", "TS19", "TS28", "TS32", "TS45", "TS57", "TS2" , "TS3" ,
                              "TS5" , "TS7" , "TS8" , "TS9" , "TS12", "TS14", "TS17", "TS18", "TS20", "TS23",
                              "TS24", "TS25", "TS26", "TS27", "TS29", "TS30", "TS31", "TS34", "TS38", "TS39",
                              "TS41", "TS42", "TS43", "TS44", "TS46", "TS47", "TS48", "TS49", "TS50", "TS51",
                              "TS52", "TS53", "TS54", "TS55", "TS56", "TS58", "TS59", "TS60"]

df_selected_1r75_1 = df_all_train_value_scaled[selected_stations_1r75_1].values #convert to array
df_selected_1r75_2 = df_all_train_value_scaled[selected_stations_1r75_2].values #convert to array
df_selected_1r75_3 = df_all_train_value_scaled[selected_stations_1r75_3].values #convert to array
df_selected_1r75_4 = df_all_train_value_scaled[selected_stations_1r75_4].values #convert to array
df_selected_1r75_5 = df_all_train_value_scaled[selected_stations_1r75_5].values #convert to array
df_selected_1r75_6 = df_all_train_value_scaled[selected_stations_1r75_6].values #convert to array
df_selected_1r75_7 = df_all_train_value_scaled[selected_stations_1r75_7].values #convert to array
df_selected_1r75_8 = df_all_train_value_scaled[selected_stations_1r75_8].values #convert to array
df_selected_1r75_9 = df_all_train_value_scaled[selected_stations_1r75_9].values #convert to array
df_selected_1r75_10 = df_all_train_value_scaled[selected_stations_1r75_10].values #convert to array
df_selected_1r75_30 = df_all_train_value_scaled[selected_stations_1r75_30].values #convert to array
df_selected_1r75_60 = df_all_train_value_scaled[selected_stations_1r75_60].values #convert to array

## Train Prediction Models

### LSTM

#### Single Station

In [264]:
# Train LSTM model
column = df_all_train_value_scaled.columns

for i in column:
    lstm_model = create_lstm_model(
        df_scaled=df_all_train_value_scaled[i], 
        no_columns=1,
        n_steps=24,
        epochs=50,
        batch_size=32,
        units_1= 50,
        units_2= 50,
        dropout_rate=0.2,
        learning_rate=0.01
        )
    lstm_model.save(f"lstm_models_v4/lstm_model_for_{i}.keras") # Save the trained model

Epoch 1/50


/Users/manojherath/PythonProjects/MadridTraffic/.venv/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


54/54 ━━━━━━━━━━━━━━━━━━━━ 4s 16ms/step - loss: 0.0849 - mae: 0.2120 - val_loss: 0.0896 - val_mae: 0.2425
Epoch 2/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0247 - mae: 0.1242 - val_loss: 0.0808 - val_mae: 0.2333
Epoch 3/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0183 - mae: 0.1056 - val_loss: 0.0714 - val_mae: 0.2237
Epoch 4/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0158 - mae: 0.0987 - val_loss: 0.0626 - val_mae: 0.2099
Epoch 5/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0146 - mae: 0.0914 - val_loss: 0.0543 - val_mae: 0.1965
Epoch 6/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0124 - mae: 0.0876 - val_loss: 0.0509 - val_mae: 0.1920
Epoch 7/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0117 - mae: 0.0850 - val_loss: 0.0427 - val_mae: 0.1762
Epoch 8/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0115 - mae: 0.0835 - val_loss: 0.0392 - val_mae: 0.1679
Epoch 9/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0096 - mae: 0.0755 - v

#### 1 > r > 0.95

In [50]:
# Train LSTM model
lstm_model_1r95_1 = create_lstm_model(
    df_scaled=df_selected_1r95_1, 
    no_columns=len(selected_stations_1r95_1),
    n_steps=24,
    epochs=50,
    batch_size=32,
    units_1= 50,
    units_2= 50,
    dropout_rate=0.2,
    learning_rate=0.01
    )

lstm_model_1r95_1.save("lstm_models_v4/lstm_model_1r95_1.keras") # Save the trained model

Epoch 1/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - loss: 0.0457 - mae: 0.1638 - val_loss: 0.0828 - val_mae: 0.2342
Epoch 2/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0162 - mae: 0.0972 - val_loss: 0.0654 - val_mae: 0.2152
Epoch 3/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0119 - mae: 0.0846 - val_loss: 0.0696 - val_mae: 0.2151
Epoch 4/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0097 - mae: 0.0761 - val_loss: 0.0668 - val_mae: 0.2108
Epoch 5/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0077 - mae: 0.0665 - val_loss: 0.0586 - val_mae: 0.1992
Epoch 6/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0076 - mae: 0.0669 - val_loss: 0.0491 - val_mae: 0.1819
Epoch 7/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0078 - mae: 0.0666 - val_loss: 0.0411 - val_mae: 0.1684
Epoch 8/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0069 - mae: 0.0636 - val_loss: 0.0401 - val_mae: 0.1622
Epoch 9/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0068 - mae:

In [ ]:
# Train LSTM model
lstm_model_1r95_2 = create_lstm_model(
    df_scaled=df_selected_1r95_2, 
    no_columns=len(selected_stations_1r95_2),
    n_steps=24,
    epochs=50,
    batch_size=32,
    units_1= 50,
    units_2= 50,
    dropout_rate=0.2,
    learning_rate=0.01
    )

lstm_model_1r95_2.save("lstm_models_v4/lstm_model_1r95_2.keras") # Save the trained model

In [ ]:
# Train LSTM model
lstm_model_1r95_3 = create_lstm_model(
    df_scaled=df_selected_1r95_3, 
    no_columns=len(selected_stations_1r95_3),
    n_steps=24,
    epochs=50,
    batch_size=32,
    units_1= 50,
    units_2= 50,
    dropout_rate=0.2,
    learning_rate=0.01
    )

lstm_model_1r95_3.save("lstm_models_v4/lstm_model_1r95_3.keras") # Save the trained model

In [71]:
# Train LSTM model
lstm_model_1r95_4 = create_lstm_model(
    df_scaled=df_selected_1r95_4, 
    no_columns=len(selected_stations_1r95_4),
    n_steps=24,
    epochs=50,
    batch_size=32,
    units_1= 50,
    units_2= 50,
    dropout_rate=0.2,
    learning_rate=0.01
    )

lstm_model_1r95_4.save("lstm_models_v4/lstm_model_1r95_4.keras") # Save the trained model

Epoch 1/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - loss: 0.0906 - mae: 0.2247 - val_loss: 0.0913 - val_mae: 0.2412
Epoch 2/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0214 - mae: 0.1126 - val_loss: 0.0736 - val_mae: 0.2193
Epoch 3/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0148 - mae: 0.0929 - val_loss: 0.0552 - val_mae: 0.1959
Epoch 4/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0124 - mae: 0.0838 - val_loss: 0.0428 - val_mae: 0.1721
Epoch 5/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0110 - mae: 0.0786 - val_loss: 0.0381 - val_mae: 0.1652
Epoch 6/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0100 - mae: 0.0759 - val_loss: 0.0307 - val_mae: 0.1472
Epoch 7/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0083 - mae: 0.0697 - val_loss: 0.0336 - val_mae: 0.1526
Epoch 8/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0085 - mae: 0.0704 - val_loss: 0.0210 - val_mae: 0.1228
Epoch 9/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0075 - mae

In [ ]:
# Train LSTM model
lstm_model_1r95_5 = create_lstm_model(
    df_scaled=df_selected_1r95_5, 
    no_columns=len(selected_stations_1r95_5),
    n_steps=24,
    epochs=50,
    batch_size=32,
    units_1= 50,
    units_2= 50,
    dropout_rate=0.2,
    learning_rate=0.01
    )

lstm_model_1r95_5.save("lstm_models_v4/lstm_model_1r95_5.keras") # Save the trained model

In [67]:
# Train LSTM model
lstm_model_1r95_6 = create_lstm_model(
    df_scaled=df_selected_1r95_6, 
    no_columns=len(selected_stations_1r95_6),
    n_steps=24,
    epochs=50,
    batch_size=32,
    units_1= 50,
    units_2= 50,
    dropout_rate=0.2,
    learning_rate=0.01
    )

lstm_model_1r95_6.save("lstm_models_v4/lstm_model_1r95_6.keras") # Save the trained model

Epoch 1/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - loss: 0.1385 - mae: 0.2596 - val_loss: 0.0743 - val_mae: 0.2191
Epoch 2/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0255 - mae: 0.1239 - val_loss: 0.0659 - val_mae: 0.2049
Epoch 3/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0166 - mae: 0.0971 - val_loss: 0.0551 - val_mae: 0.1876
Epoch 4/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0138 - mae: 0.0875 - val_loss: 0.0445 - val_mae: 0.1687
Epoch 5/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0115 - mae: 0.0804 - val_loss: 0.0332 - val_mae: 0.1471
Epoch 6/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0095 - mae: 0.0724 - val_loss: 0.0264 - val_mae: 0.1315
Epoch 7/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0094 - mae: 0.0727 - val_loss: 0.0205 - val_mae: 0.1160
Epoch 8/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0088 - mae: 0.0698 - val_loss: 0.0187 - val_mae: 0.1092
Epoch 9/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0090 - mae:

In [ ]:
# Train LSTM model
lstm_model_1r95_7 = create_lstm_model(
    df_scaled=df_selected_1r95_7, 
    no_columns=len(selected_stations_1r95_7),
    n_steps=24,
    epochs=50,
    batch_size=32,
    units_1= 50,
    units_2= 50,
    dropout_rate=0.2,
    learning_rate=0.01
    )

lstm_model_1r95_7.save("lstm_models_v4/lstm_model_1r95_7.keras") # Save the trained model

In [ ]:
# Train LSTM model
lstm_model_1r95_8 = create_lstm_model(
    df_scaled=df_selected_1r95_8, 
    no_columns=len(selected_stations_1r95_8),
    n_steps=24,
    epochs=50,
    batch_size=32,
    units_1= 50,
    units_2= 50,
    dropout_rate=0.2,
    learning_rate=0.01
    )

lstm_model_1r95_8.save("lstm_models_v4/lstm_model_1r95_8.keras") # Save the trained model

In [66]:
# Train LSTM model
lstm_model_1r95_9 = create_lstm_model(
    df_scaled=df_selected_1r95_9, 
    no_columns=len(selected_stations_1r95_9),
    n_steps=24,
    epochs=50,
    batch_size=32,
    units_1= 50,
    units_2= 50,
    dropout_rate=0.2,
    learning_rate=0.01
    )

lstm_model_1r95_9.save("lstm_models_v4/lstm_model_1r95_9.keras") # Save the trained model

Epoch 1/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 4s 13ms/step - loss: 0.0915 - mae: 0.2253 - val_loss: 0.0729 - val_mae: 0.2234
Epoch 2/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0186 - mae: 0.1055 - val_loss: 0.0537 - val_mae: 0.1926
Epoch 3/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0144 - mae: 0.0908 - val_loss: 0.0459 - val_mae: 0.1796
Epoch 4/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0118 - mae: 0.0824 - val_loss: 0.0398 - val_mae: 0.1629
Epoch 5/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0104 - mae: 0.0770 - val_loss: 0.0396 - val_mae: 0.1640
Epoch 6/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0170 - mae: 0.0906 - val_loss: 0.0580 - val_mae: 0.1636
Epoch 7/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0150 - mae: 0.0928 - val_loss: 0.0186 - val_mae: 0.1060
Epoch 8/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0102 - mae: 0.0760 - val_loss: 0.0133 - val_mae: 0.0882
Epoch 9/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0085 - mae

In [ ]:
# Train LSTM model
lstm_model_1r95_10 = create_lstm_model(
    df_scaled=df_selected_1r95_10, 
    no_columns=len(selected_stations_1r95_10),
    n_steps=24,
    epochs=50,
    batch_size=32,
    units_1= 50,
    units_2= 50,
    dropout_rate=0.2,
    learning_rate=0.01
    )

lstm_model_1r95_10.save("lstm_models_v4/lstm_model_1r95_10.keras") # Save the trained model

In [32]:
# Train LSTM model
lstm_model_1r95_20 = create_lstm_model(
    df_scaled=df_selected_1r95_20, 
    no_columns=len(selected_stations_1r95_20),
    n_steps=24,
    epochs=50,
    batch_size=32,
    units_1= 100,
    units_2= 50,
    dropout_rate=0.2,
    learning_rate=0.01
    )

lstm_model_1r95_20.save("lstm_models_v4/lstm_model_1r95_20.keras") # Save the trained model

Epoch 1/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - loss: 0.1020 - mae: 0.2270 - val_loss: 0.0608 - val_mae: 0.1977
Epoch 2/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 0.0161 - mae: 0.0975 - val_loss: 0.0501 - val_mae: 0.1801
Epoch 3/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 0.0134 - mae: 0.0882 - val_loss: 0.0468 - val_mae: 0.1733
Epoch 4/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 0.0110 - mae: 0.0788 - val_loss: 0.0373 - val_mae: 0.1550
Epoch 5/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 0.0101 - mae: 0.0753 - val_loss: 0.0245 - val_mae: 0.1251
Epoch 6/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - loss: 0.0095 - mae: 0.0725 - val_loss: 0.0280 - val_mae: 0.1347
Epoch 7/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: 0.0087 - mae: 0.0685 - val_loss: 0.0160 - val_mae: 0.1012
Epoch 8/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - loss: 0.0078 - mae: 0.0657 - val_loss: 0.0144 - val_mae: 0.0934
Epoch 9/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 0.007

In [30]:
# Train LSTM model
lstm_model_1r95_30 = create_lstm_model(
    df_scaled=df_selected_1r95_30, 
    no_columns=len(selected_stations_1r95_30),
    n_steps=24,
    epochs=50,
    batch_size=32,
    units_1= 100,
    units_2= 50,
    dropout_rate=0.2,
    learning_rate=0.01
    )

lstm_model_1r95_30.save("lstm_models_v4/lstm_model_1r95_30.keras") # Save the trained model

Epoch 1/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - loss: 0.1249 - mae: 0.2482 - val_loss: 0.0438 - val_mae: 0.1692
Epoch 2/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 0.0188 - mae: 0.1051 - val_loss: 0.0459 - val_mae: 0.1720
Epoch 3/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - loss: 0.0138 - mae: 0.0875 - val_loss: 0.0328 - val_mae: 0.1449
Epoch 4/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - loss: 0.0124 - mae: 0.0827 - val_loss: 0.0272 - val_mae: 0.1306
Epoch 5/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - loss: 0.0112 - mae: 0.0787 - val_loss: 0.0198 - val_mae: 0.1118
Epoch 6/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - loss: 0.0097 - mae: 0.0722 - val_loss: 0.0193 - val_mae: 0.1076
Epoch 7/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 0.0091 - mae: 0.0692 - val_loss: 0.0189 - val_mae: 0.1056
Epoch 8/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - loss: 0.0096 - mae: 0.0713 - val_loss: 0.0108 - val_mae: 0.0762
Epoch 9/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - loss: 0.008

In [37]:
# Train LSTM model
lstm_model_1r95_40 = create_lstm_model(
    df_scaled=df_selected_1r95_40, 
    no_columns=len(selected_stations_1r95_40),
    n_steps=24,
    epochs=100,
    batch_size=32,
    units_1= 100,
    units_2= 50,
    dropout_rate=0.2,
    learning_rate=0.01
    )

lstm_model_1r95_40.save("lstm_models_v4/lstm_model_1r95_40.keras") # Save the trained model

Epoch 1/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - loss: 0.1246 - mae: 0.2509 - val_loss: 0.0450 - val_mae: 0.1658
Epoch 2/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - loss: 0.0169 - mae: 0.0996 - val_loss: 0.0437 - val_mae: 0.1603
Epoch 3/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 0.0128 - mae: 0.0855 - val_loss: 0.0338 - val_mae: 0.1421
Epoch 4/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 0.0112 - mae: 0.0794 - val_loss: 0.0365 - val_mae: 0.1496
Epoch 5/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: 0.0106 - mae: 0.0766 - val_loss: 0.0269 - val_mae: 0.1272
Epoch 6/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 0.0093 - mae: 0.0716 - val_loss: 0.0336 - val_mae: 0.1446
Epoch 7/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 0.0096 - mae: 0.0721 - val_loss: 0.0166 - val_mae: 0.0972
Epoch 8/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 0.0089 - mae: 0.0692 - val_loss: 0.0169 - val_mae: 0.0998
Epoch 9/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - lo

In [43]:
# Train LSTM model
lstm_model_1r95_50 = create_lstm_model(
    df_scaled=df_selected_1r95_50, 
    no_columns=len(selected_stations_1r95_50),
    n_steps=24,
    epochs=50,
    batch_size=32,
    units_1= 100,
    units_2= 50,
    dropout_rate=0.2,
    learning_rate=0.01
    )

lstm_model_1r95_50.save("lstm_models_v4/lstm_model_1r95_50.keras") # Save the trained model

Epoch 1/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - loss: 0.0936 - mae: 0.2158 - val_loss: 0.0401 - val_mae: 0.1575
Epoch 2/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 0.0142 - mae: 0.0902 - val_loss: 0.0443 - val_mae: 0.1615
Epoch 3/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 0.0114 - mae: 0.0796 - val_loss: 0.0501 - val_mae: 0.1727
Epoch 4/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - loss: 0.0098 - mae: 0.0729 - val_loss: 0.0514 - val_mae: 0.1770
Epoch 5/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 0.0089 - mae: 0.0690 - val_loss: 0.0390 - val_mae: 0.1520
Epoch 6/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 0.0088 - mae: 0.0684 - val_loss: 0.0329 - val_mae: 0.1393
Epoch 7/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 0.0086 - mae: 0.0676 - val_loss: 0.0219 - val_mae: 0.1116
Epoch 8/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 0.0079 - mae: 0.0647 - val_loss: 0.0214 - val_mae: 0.1095
Epoch 9/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: 0.008

In [11]:
# Train LSTM model
lstm_model_1r95_60 = create_lstm_model(
    df_scaled=df_selected_1r95_60, 
    no_columns=len(selected_stations_1r95_60),
    n_steps=24,
    epochs=50,
    batch_size=32,
    units_1= 50,
    units_2= 50,
    dropout_rate=0.2,
    learning_rate=0.01
    )

lstm_model_1r95_60.save("lstm_models_v4/lstm_model_1r95_60.keras") # Save the trained model

Epoch 1/50


/Users/manojherath/PythonProjects/MadridTraffic/.venv/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


54/54 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - loss: 0.0713 - mae: 0.1915 - val_loss: 0.0403 - val_mae: 0.1555
Epoch 2/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0131 - mae: 0.0856 - val_loss: 0.0327 - val_mae: 0.1386
Epoch 3/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0107 - mae: 0.0766 - val_loss: 0.0279 - val_mae: 0.1283
Epoch 4/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0099 - mae: 0.0728 - val_loss: 0.0242 - val_mae: 0.1188
Epoch 5/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0092 - mae: 0.0697 - val_loss: 0.0163 - val_mae: 0.0951
Epoch 6/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0084 - mae: 0.0666 - val_loss: 0.0170 - val_mae: 0.0957
Epoch 7/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0080 - mae: 0.0642 - val_loss: 0.0150 - val_mae: 0.0911
Epoch 8/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0076 - mae: 0.0626 - val_loss: 0.0138 - val_mae: 0.0871
Epoch 9/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0074 - mae: 0.0621 - 

#### 1 > r > 0.90

In [72]:
# Train LSTM model
lstm_model_1r90_1 = create_lstm_model(
    df_scaled=df_selected_1r90_1, 
    no_columns=len(selected_stations_1r90_1),
    n_steps=24,
    epochs=50,
    batch_size=32,
    units_1=50,
    units_2=50,
    dropout_rate=0.2,
    learning_rate=0.01
    )

lstm_model_1r90_1.save("lstm_models_v4/lstm_model_1r90_1.keras") # Save the trained model

Epoch 1/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - loss: 0.1270 - mae: 0.2507 - val_loss: 0.1194 - val_mae: 0.2803
Epoch 2/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0201 - mae: 0.1124 - val_loss: 0.0920 - val_mae: 0.2484
Epoch 3/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0148 - mae: 0.0950 - val_loss: 0.0807 - val_mae: 0.2340
Epoch 4/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0110 - mae: 0.0809 - val_loss: 0.0762 - val_mae: 0.2276
Epoch 5/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0107 - mae: 0.0779 - val_loss: 0.0593 - val_mae: 0.2033
Epoch 6/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0091 - mae: 0.0713 - val_loss: 0.0561 - val_mae: 0.1987
Epoch 7/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0083 - mae: 0.0700 - val_loss: 0.0398 - val_mae: 0.1696
Epoch 8/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0084 - mae: 0.0692 - val_loss: 0.0362 - val_mae: 0.1637
Epoch 9/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0080 - mae:

In [ ]:
# Train LSTM model
lstm_model_1r90_2 = create_lstm_model(
    df_scaled=df_selected_1r90_2, 
    no_columns=len(selected_stations_1r90_2),
    n_steps=24,
    epochs=50,
    batch_size=32,
    units_1=50,
    units_2=50,
    dropout_rate=0.2,
    learning_rate=0.01
    )

lstm_model_1r90_2.save("lstm_models_v4/lstm_model_1r90_2.keras") # Save the trained model

In [ ]:
# Train LSTM model
lstm_model_1r90_3 = create_lstm_model(
    df_scaled=df_selected_1r90_3, 
    no_columns=len(selected_stations_1r90_3),
    n_steps=24,
    epochs=50,
    batch_size=32,
    units_1=50,
    units_2=50,
    dropout_rate=0.2,
    learning_rate=0.01
    )

lstm_model_1r90_3.save("lstm_models_v4/lstm_model_1r90_3.keras") # Save the trained model

In [75]:
# Train LSTM model
lstm_model_1r90_4 = create_lstm_model(
    df_scaled=df_selected_1r90_4, 
    no_columns=len(selected_stations_1r90_4),
    n_steps=24,
    epochs=50,
    batch_size=32,
    units_1=50,
    units_2=50,
    dropout_rate=0.2,
    learning_rate=0.01
    )

lstm_model_1r90_4.save("lstm_models_v4/lstm_model_1r90_4.keras") # Save the trained model

Epoch 1/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - loss: 0.0960 - mae: 0.2270 - val_loss: 0.0823 - val_mae: 0.2323
Epoch 2/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0202 - mae: 0.1095 - val_loss: 0.0652 - val_mae: 0.2091
Epoch 3/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0144 - mae: 0.0916 - val_loss: 0.0644 - val_mae: 0.2084
Epoch 4/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0123 - mae: 0.0830 - val_loss: 0.0538 - val_mae: 0.1906
Epoch 5/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0110 - mae: 0.0798 - val_loss: 0.0392 - val_mae: 0.1637
Epoch 6/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0096 - mae: 0.0741 - val_loss: 0.0346 - val_mae: 0.1572
Epoch 7/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0083 - mae: 0.0693 - val_loss: 0.0285 - val_mae: 0.1450
Epoch 8/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0080 - mae: 0.0670 - val_loss: 0.0207 - val_mae: 0.1198
Epoch 9/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0076 - mae:

In [76]:
# Train LSTM model
lstm_model_1r90_5 = create_lstm_model(
    df_scaled=df_selected_1r90_5, 
    no_columns=len(selected_stations_1r90_5),
    n_steps=24,
    epochs=50,
    batch_size=32,
    units_1=50,
    units_2=50,
    dropout_rate=0.2,
    learning_rate=0.01
    )

lstm_model_1r90_5.save("lstm_models_v4/lstm_model_1r90_5.keras") # Save the trained model

Epoch 1/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - loss: 0.0888 - mae: 0.2092 - val_loss: 0.0771 - val_mae: 0.2272
Epoch 2/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0203 - mae: 0.1092 - val_loss: 0.0607 - val_mae: 0.2049
Epoch 3/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0134 - mae: 0.0884 - val_loss: 0.0525 - val_mae: 0.1896
Epoch 4/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0122 - mae: 0.0841 - val_loss: 0.0489 - val_mae: 0.1827
Epoch 5/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0103 - mae: 0.0758 - val_loss: 0.0425 - val_mae: 0.1741
Epoch 6/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0095 - mae: 0.0741 - val_loss: 0.0398 - val_mae: 0.1671
Epoch 7/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0097 - mae: 0.0745 - val_loss: 0.0300 - val_mae: 0.1463
Epoch 8/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0081 - mae: 0.0678 - val_loss: 0.0217 - val_mae: 0.1244
Epoch 9/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0079 - mae:

In [ ]:
# Train LSTM model
lstm_model_1r90_6 = create_lstm_model(
    df_scaled=df_selected_1r90_6, 
    no_columns=len(selected_stations_1r90_6),
    n_steps=24,
    epochs=50,
    batch_size=32,
    units_1=50,
    units_2=50,
    dropout_rate=0.2,
    learning_rate=0.01
    )

lstm_model_1r90_6.save("lstm_models_v4/lstm_model_1r90_6.keras") # Save the trained model

Epoch 1/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - loss: 0.0862 - mae: 0.2137 - val_loss: 0.0823 - val_mae: 0.2301
Epoch 2/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0215 - mae: 0.1115 - val_loss: 0.0587 - val_mae: 0.1975
Epoch 3/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0137 - mae: 0.0887 - val_loss: 0.0411 - val_mae: 0.1673
Epoch 4/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0126 - mae: 0.0845 - val_loss: 0.0380 - val_mae: 0.1579
Epoch 5/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0098 - mae: 0.0754 - val_loss: 0.0389 - val_mae: 0.1607
Epoch 6/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0090 - mae: 0.0715 - val_loss: 0.0309 - val_mae: 0.1429
Epoch 7/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0077 - mae: 0.0654 - val_loss: 0.0177 - val_mae: 0.1080
Epoch 8/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0078 - mae: 0.0658 - val_loss: 0.0200 - val_mae: 0.1171
Epoch 9/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.00

In [ ]:
# Train LSTM model
lstm_model_1r90_7 = create_lstm_model(
    df_scaled=df_selected_1r90_7, 
    no_columns=len(selected_stations_1r90_7),
    n_steps=24,
    epochs=100,
    batch_size=32,
    units_1=50,
    units_2=50,
    dropout_rate=0.2,
    learning_rate=0.01
    )

lstm_model_1r90_7.save("lstm_models_v4/lstm_model_1r90_7.keras") # Save the trained model

In [ ]:
# Train LSTM model
lstm_model_1r90_8 = create_lstm_model(
    df_scaled=df_selected_1r90_8, 
    no_columns=len(selected_stations_1r90_8),
    n_steps=24,
    epochs=100,
    batch_size=32,
    units_1=50,
    units_2=50,
    dropout_rate=0.2,
    learning_rate=0.01
    )

lstm_model_1r90_8.save("lstm_models_v4/lstm_model_1r90_8.keras") # Save the trained model

In [ ]:
# Train LSTM model
lstm_model_1r90_9 = create_lstm_model(
    df_scaled=df_selected_1r90_9, 
    no_columns=len(selected_stations_1r90_9),
    n_steps=24,
    epochs=100,
    batch_size=32,
    units_1=50,
    units_2=50,
    dropout_rate=0.2,
    learning_rate=0.01
    )

lstm_model_1r90_9.save("lstm_models_v4/lstm_model_1r90_9.keras") # Save the trained model

In [44]:
# Train LSTM model
lstm_model_1r90_10 = create_lstm_model(
    df_scaled=df_selected_1r90_10, 
    no_columns=len(selected_stations_1r90_10),
    n_steps=24,
    epochs=100,
    batch_size=32,
    units_1=50,
    units_2=50,
    dropout_rate=0.2,
    learning_rate=0.01
    )

lstm_model_1r90_10.save("lstm_models_v4/lstm_model_1r90_10.keras") # Save the trained model

Epoch 1/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - loss: 0.0984 - mae: 0.2266 - val_loss: 0.0932 - val_mae: 0.2485
Epoch 2/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0183 - mae: 0.1040 - val_loss: 0.0587 - val_mae: 0.2057
Epoch 3/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0133 - mae: 0.0884 - val_loss: 0.0506 - val_mae: 0.1932
Epoch 4/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0102 - mae: 0.0774 - val_loss: 0.0406 - val_mae: 0.1730
Epoch 5/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0103 - mae: 0.0763 - val_loss: 0.0334 - val_mae: 0.1557
Epoch 6/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0091 - mae: 0.0730 - val_loss: 0.0248 - val_mae: 0.1289
Epoch 7/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0091 - mae: 0.0735 - val_loss: 0.0267 - val_mae: 0.1398
Epoch 8/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0084 - mae: 0.0692 - val_loss: 0.0179 - val_mae: 0.1102
Epoch 9/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.00

In [48]:
# Train LSTM model
lstm_model_1r90_30 = create_lstm_model(
    df_scaled=df_selected_1r90_30, 
    no_columns=len(selected_stations_1r90_30),
    n_steps=24,
    epochs=50,
    batch_size=32,
    units_1= 100,
    units_2= 50,
    dropout_rate=0.2,
    learning_rate=0.01
    )

lstm_model_1r90_30.save("lstm_models_v4/lstm_model_1r90_30.keras") # Save the trained model

/Users/manojherath/PythonProjects/MadridTraffic/.venv/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 4s 18ms/step - loss: 0.1054 - mae: 0.2231 - val_loss: 0.0602 - val_mae: 0.1949
Epoch 2/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: 0.0148 - mae: 0.0930 - val_loss: 0.0497 - val_mae: 0.1739
Epoch 3/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 0.0117 - mae: 0.0818 - val_loss: 0.0405 - val_mae: 0.1561
Epoch 4/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 0.0104 - mae: 0.0765 - val_loss: 0.0428 - val_mae: 0.1622
Epoch 5/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 0.0093 - mae: 0.0708 - val_loss: 0.0335 - val_mae: 0.1432
Epoch 6/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 0.0086 - mae: 0.0687 - val_loss: 0.0244 - val_mae: 0.1228
Epoch 7/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: 0.0087 - mae: 0.0687 - val_loss: 0.0148 - val_mae: 0.0917
Epoch 8/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 0.0076 - mae: 0.0641 - val_loss: 0.0192 - val_mae: 0.1063
Epoch 9/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 0.007

#### 1 > r > 0.85

In [48]:
# Train LSTM model
lstm_model_1r85_1 = create_lstm_model(
    df_scaled=df_selected_1r85_1, 
    no_columns=len(selected_stations_1r85_1),
    n_steps=24,
    epochs=100,
    batch_size=32,
    units_1= 50,
    units_2= 50,
    dropout_rate=0.2,
    learning_rate=0.01
    )

lstm_model_1r85_1.save("lstm_models_v4/lstm_model_1r85_1.keras") # Save the trained model

Epoch 1/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - loss: 0.1564 - mae: 0.2849 - val_loss: 0.1459 - val_mae: 0.3180
Epoch 2/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0245 - mae: 0.1236 - val_loss: 0.1250 - val_mae: 0.2952
Epoch 3/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0207 - mae: 0.1150 - val_loss: 0.1020 - val_mae: 0.2699
Epoch 4/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0179 - mae: 0.1062 - val_loss: 0.0851 - val_mae: 0.2490
Epoch 5/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0131 - mae: 0.0890 - val_loss: 0.0756 - val_mae: 0.2350
Epoch 6/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0123 - mae: 0.0888 - val_loss: 0.0506 - val_mae: 0.1934
Epoch 7/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0134 - mae: 0.0900 - val_loss: 0.0526 - val_mae: 0.1979
Epoch 8/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0106 - mae: 0.0819 - val_loss: 0.0486 - val_mae: 0.1898
Epoch 9/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.00

In [ ]:
# Train LSTM model
lstm_model_1r85_2 = create_lstm_model(
    df_scaled=df_selected_1r85_2, 
    no_columns=len(selected_stations_1r85_2),
    n_steps=24,
    epochs=100,
    batch_size=32,
    units_1= 50,
    units_2= 50,
    dropout_rate=0.2,
    learning_rate=0.01
    )

lstm_model_1r85_2.save("lstm_models_v4/lstm_model_1r85_2.keras") # Save the trained model

In [ ]:
# Train LSTM model
lstm_model_1r85_3 = create_lstm_model(
    df_scaled=df_selected_1r85_3, 
    no_columns=len(selected_stations_1r85_3),
    n_steps=24,
    epochs=100,
    batch_size=32,
    units_1= 100,
    units_2= 50,
    dropout_rate=0.3,
    learning_rate=0.001
    )

lstm_model_1r85_3.save("lstm_models_v4/lstm_model_1r85_3.keras") # Save the trained model

In [ ]:
# Train LSTM model
lstm_model_1r85_4 = create_lstm_model(
    df_scaled=df_selected_1r85_4, 
    no_columns=len(selected_stations_1r85_4),
    n_steps=24,
    epochs=100,
    batch_size=32,
    units_1= 50,
    units_2= 50,
    dropout_rate=0.2,
    learning_rate=0.01
    )

lstm_model_1r85_4.save("lstm_models_v4/lstm_model_1r85_4.keras") # Save the trained model

In [ ]:
# Train LSTM model
lstm_model_1r85_5 = create_lstm_model(
    df_scaled=df_selected_1r85_5, 
    no_columns=len(selected_stations_1r85_5),
    n_steps=24,
    epochs=100,
    batch_size=32,
    units_1= 50,
    units_2= 50,
    dropout_rate=0.2,
    learning_rate=0.01
    )

lstm_model_1r85_5.save("lstm_models_v4/lstm_model_1r85_5.keras") # Save the trained model

In [ ]:
# Train LSTM model
lstm_model_1r85_6 = create_lstm_model(
    df_scaled=df_selected_1r85_6, 
    no_columns=len(selected_stations_1r85_6),
    n_steps=24,
    epochs=100,
    batch_size=32,
    units_1= 50,
    units_2= 50,
    dropout_rate=0.2,
    learning_rate=0.01
    )

lstm_model_1r85_6.save("lstm_models_v4/lstm_model_1r85_6.keras") # Save the trained model

In [65]:
# Train LSTM model
lstm_model_1r85_7 = create_lstm_model(
    df_scaled=df_selected_1r85_7, 
    no_columns=len(selected_stations_1r85_7),
    n_steps=24,
    epochs=100,
    batch_size=32,
    units_1= 50,
    units_2= 50,
    dropout_rate=0.2,
    learning_rate=0.01
    )

lstm_model_1r85_7.save("lstm_models_v4/lstm_model_1r85_7.keras") # Save the trained model

Epoch 1/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - loss: 0.1094 - mae: 0.2467 - val_loss: 0.0721 - val_mae: 0.2245
Epoch 2/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0210 - mae: 0.1130 - val_loss: 0.0634 - val_mae: 0.2084
Epoch 3/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0131 - mae: 0.0881 - val_loss: 0.0565 - val_mae: 0.1951
Epoch 4/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0105 - mae: 0.0772 - val_loss: 0.0481 - val_mae: 0.1801
Epoch 5/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0100 - mae: 0.0750 - val_loss: 0.0429 - val_mae: 0.1719
Epoch 6/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0089 - mae: 0.0723 - val_loss: 0.0325 - val_mae: 0.1505
Epoch 7/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0089 - mae: 0.0723 - val_loss: 0.0244 - val_mae: 0.1293
Epoch 8/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0083 - mae: 0.0697 - val_loss: 0.0258 - val_mae: 0.1324
Epoch 9/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 0.0

In [ ]:
# Train LSTM model
lstm_model_1r85_8 = create_lstm_model(
    df_scaled=df_selected_1r85_8, 
    no_columns=len(selected_stations_1r85_8),
    n_steps=24,
    epochs=100,
    batch_size=32,
    units_1= 50,
    units_2= 50,
    dropout_rate=0.2,
    learning_rate=0.01
    )

lstm_model_1r85_8.save("lstm_models_v4/lstm_model_1r85_8.keras") # Save the trained model

In [ ]:
# Train LSTM model
lstm_model_1r85_9 = create_lstm_model(
    df_scaled=df_selected_1r85_9, 
    no_columns=len(selected_stations_1r85_9),
    n_steps=24,
    epochs=100,
    batch_size=32,
    units_1= 50,
    units_2= 50,
    dropout_rate=0.2,
    learning_rate=0.01
    )

lstm_model_1r85_9.save("lstm_models_v4/lstm_model_1r85_9.keras") # Save the trained model

In [ ]:
# Train LSTM model
lstm_model_1r85_10 = create_lstm_model(
    df_scaled=df_selected_1r85_10, 
    no_columns=len(selected_stations_1r85_10),
    n_steps=24,
    epochs=100,
    batch_size=32,
    units_1= 50,
    units_2= 50,
    dropout_rate=0.2,
    learning_rate=0.01
    )

lstm_model_1r85_10.save("lstm_models_v4/lstm_model_1r85_10.keras") # Save the trained model

In [49]:
# Train LSTM model
lstm_model_1r85_30 = create_lstm_model(
    df_scaled=df_selected_1r85_30, 
    no_columns=len(selected_stations_1r85_30),
    n_steps=24,
    epochs=50,
    batch_size=32,
    units_1= 100,
    units_2= 50,
    dropout_rate=0.2,
    learning_rate=0.01
    )

lstm_model_1r85_30.save("lstm_models_v4/lstm_model_1r85_30.keras") # Save the trained model

Epoch 1/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 3s 19ms/step - loss: 0.1071 - mae: 0.2268 - val_loss: 0.0550 - val_mae: 0.1828
Epoch 2/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 0.0142 - mae: 0.0899 - val_loss: 0.0496 - val_mae: 0.1737
Epoch 3/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 0.0109 - mae: 0.0784 - val_loss: 0.0498 - val_mae: 0.1738
Epoch 4/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 0.0110 - mae: 0.0786 - val_loss: 0.0353 - val_mae: 0.1472
Epoch 5/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 0.0088 - mae: 0.0693 - val_loss: 0.0313 - val_mae: 0.1372
Epoch 6/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: 0.0093 - mae: 0.0706 - val_loss: 0.0257 - val_mae: 0.1249
Epoch 7/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 0.0084 - mae: 0.0668 - val_loss: 0.0208 - val_mae: 0.1099
Epoch 8/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 0.0076 - mae: 0.0628 - val_loss: 0.0102 - val_mae: 0.0761
Epoch 9/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 0.008

#### 1 > r > 0.80

In [93]:
# Train LSTM model
lstm_model_1r80_1 = create_lstm_model(
    df_scaled=df_selected_1r80_1, 
    no_columns=len(selected_stations_1r80_1),
    n_steps=24,
    epochs=50,
    batch_size=32,
    units_1= 128,
    units_2= 64,
    dropout_rate=0.2,
    learning_rate=0.001
    )

lstm_model_1r80_1.save("lstm_models_v4/lstm_model_1r80_1.keras") # Save the trained model

Epoch 1/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 4s 23ms/step - loss: 0.1208 - mae: 0.2589 - val_loss: 0.1750 - val_mae: 0.3542
Epoch 2/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: 0.0234 - mae: 0.1203 - val_loss: 0.1649 - val_mae: 0.3422
Epoch 3/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - loss: 0.0208 - mae: 0.1120 - val_loss: 0.1505 - val_mae: 0.3251
Epoch 4/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - loss: 0.0160 - mae: 0.0987 - val_loss: 0.1297 - val_mae: 0.3004
Epoch 5/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: 0.0158 - mae: 0.0975 - val_loss: 0.1209 - val_mae: 0.2893
Epoch 6/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - loss: 0.0147 - mae: 0.0942 - val_loss: 0.1107 - val_mae: 0.2765
Epoch 7/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - loss: 0.0132 - mae: 0.0891 - val_loss: 0.0950 - val_mae: 0.2555
Epoch 8/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - loss: 0.0121 - mae: 0.0845 - val_loss: 0.0827 - val_mae: 0.2380
Epoch 9/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - loss: 0.012

In [104]:
# Train LSTM model
lstm_model_1r80_2 = create_lstm_model(
    df_scaled=df_selected_1r80_2, 
    no_columns=len(selected_stations_1r80_2),
    n_steps=24,
    epochs=100,
    batch_size=32,
    units_1= 50,
    units_2= 50,
    dropout_rate=0.2,
    learning_rate=0.01
    )

lstm_model_1r80_2.save("lstm_models_v4/lstm_model_1r80_2.keras") # Save the trained model

Epoch 1/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - loss: 0.1121 - mae: 0.2324 - val_loss: 0.0865 - val_mae: 0.2445
Epoch 2/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0186 - mae: 0.1060 - val_loss: 0.0669 - val_mae: 0.2178
Epoch 3/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0146 - mae: 0.0914 - val_loss: 0.0589 - val_mae: 0.2057
Epoch 4/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0110 - mae: 0.0805 - val_loss: 0.0509 - val_mae: 0.1929
Epoch 5/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0120 - mae: 0.0825 - val_loss: 0.0445 - val_mae: 0.1799
Epoch 6/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0093 - mae: 0.0730 - val_loss: 0.0419 - val_mae: 0.1756
Epoch 7/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0079 - mae: 0.0671 - val_loss: 0.0338 - val_mae: 0.1567
Epoch 8/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0081 - mae: 0.0663 - val_loss: 0.0288 - val_mae: 0.1449
Epoch 9/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0074 - mae

In [95]:
# Train LSTM model
lstm_model_1r80_3 = create_lstm_model(
    df_scaled=df_selected_1r80_3, 
    no_columns=len(selected_stations_1r80_3),
    n_steps=24,
    epochs=50,
    batch_size=32,
    units_1= 50,
    units_2= 50,
    dropout_rate=0.2,
    learning_rate=0.01
    )

lstm_model_1r80_3.save("lstm_models_v4/lstm_model_1r80_3.keras") # Save the trained model

Epoch 1/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 13s 12ms/step - loss: 0.0848 - mae: 0.2163 - val_loss: 0.0905 - val_mae: 0.2484
Epoch 2/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0213 - mae: 0.1135 - val_loss: 0.0741 - val_mae: 0.2288
Epoch 3/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0139 - mae: 0.0902 - val_loss: 0.0610 - val_mae: 0.2100
Epoch 4/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0131 - mae: 0.0880 - val_loss: 0.0554 - val_mae: 0.2005
Epoch 5/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0102 - mae: 0.0762 - val_loss: 0.0440 - val_mae: 0.1805
Epoch 6/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0097 - mae: 0.0740 - val_loss: 0.0393 - val_mae: 0.1699
Epoch 7/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0089 - mae: 0.0713 - val_loss: 0.0331 - val_mae: 0.1564
Epoch 8/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0090 - mae: 0.0720 - val_loss: 0.0261 - val_mae: 0.1390
Epoch 9/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0083 - mae

In [96]:
# Train LSTM model
lstm_model_1r80_4 = create_lstm_model(
    df_scaled=df_selected_1r80_4, 
    no_columns=len(selected_stations_1r80_4),
    n_steps=24,
    epochs=50,
    batch_size=32,
    units_1= 50,
    units_2= 50,
    dropout_rate=0.2,
    learning_rate=0.01
    )

lstm_model_1r80_4.save("lstm_models_v4/lstm_model_1r80_4.keras") # Save the trained model

Epoch 1/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - loss: 0.0918 - mae: 0.2233 - val_loss: 0.0945 - val_mae: 0.2495
Epoch 2/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0223 - mae: 0.1158 - val_loss: 0.0744 - val_mae: 0.2265
Epoch 3/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0144 - mae: 0.0922 - val_loss: 0.0565 - val_mae: 0.2009
Epoch 4/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0117 - mae: 0.0818 - val_loss: 0.0466 - val_mae: 0.1846
Epoch 5/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0094 - mae: 0.0735 - val_loss: 0.0386 - val_mae: 0.1705
Epoch 6/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0092 - mae: 0.0730 - val_loss: 0.0313 - val_mae: 0.1526
Epoch 7/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0091 - mae: 0.0699 - val_loss: 0.0263 - val_mae: 0.1403
Epoch 8/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0082 - mae: 0.0689 - val_loss: 0.0214 - val_mae: 0.1265
Epoch 9/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0083 - mae:

In [97]:
# Train LSTM model
lstm_model_1r80_5 = create_lstm_model(
    df_scaled=df_selected_1r80_5, 
    no_columns=len(selected_stations_1r80_5),
    n_steps=24,
    epochs=50,
    batch_size=32,
    units_1= 50,
    units_2= 50,
    dropout_rate=0.2,
    learning_rate=0.01
    )

lstm_model_1r80_5.save("lstm_models_v4/lstm_model_1r80_5.keras") # Save the trained model

Epoch 1/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - loss: 0.1119 - mae: 0.2322 - val_loss: 0.0764 - val_mae: 0.2337
Epoch 2/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0189 - mae: 0.1052 - val_loss: 0.0586 - val_mae: 0.2094
Epoch 3/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0135 - mae: 0.0885 - val_loss: 0.0522 - val_mae: 0.1983
Epoch 4/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0120 - mae: 0.0829 - val_loss: 0.0449 - val_mae: 0.1837
Epoch 5/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0102 - mae: 0.0751 - val_loss: 0.0415 - val_mae: 0.1767
Epoch 6/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0094 - mae: 0.0728 - val_loss: 0.0341 - val_mae: 0.1597
Epoch 7/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0092 - mae: 0.0720 - val_loss: 0.0293 - val_mae: 0.1472
Epoch 8/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0090 - mae: 0.0709 - val_loss: 0.0211 - val_mae: 0.1245
Epoch 9/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0081 - mae:

In [98]:
# Train LSTM model
lstm_model_1r80_6 = create_lstm_model(
    df_scaled=df_selected_1r80_6, 
    no_columns=len(selected_stations_1r80_6),
    n_steps=24,
    epochs=50,
    batch_size=32,
    units_1= 50,
    units_2= 50,
    dropout_rate=0.2,
    learning_rate=0.01
    )

lstm_model_1r80_6.save("lstm_models_v4/lstm_model_1r80_6.keras") # Save the trained model

Epoch 1/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - loss: 0.0841 - mae: 0.2099 - val_loss: 0.0844 - val_mae: 0.2413
Epoch 2/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0187 - mae: 0.1046 - val_loss: 0.0590 - val_mae: 0.2089
Epoch 3/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0149 - mae: 0.0915 - val_loss: 0.0520 - val_mae: 0.1965
Epoch 4/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0111 - mae: 0.0786 - val_loss: 0.0445 - val_mae: 0.1831
Epoch 5/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0096 - mae: 0.0747 - val_loss: 0.0406 - val_mae: 0.1751
Epoch 6/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0092 - mae: 0.0725 - val_loss: 0.0315 - val_mae: 0.1531
Epoch 7/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0086 - mae: 0.0708 - val_loss: 0.0227 - val_mae: 0.1291
Epoch 8/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0087 - mae: 0.0709 - val_loss: 0.0206 - val_mae: 0.1224
Epoch 9/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0086 - mae:

In [105]:
# Train LSTM model
lstm_model_1r80_7 = create_lstm_model(
    df_scaled=df_selected_1r80_7, 
    no_columns=len(selected_stations_1r80_7),
    n_steps=24,
    epochs=50,
    batch_size=32,
    units_1= 50,
    units_2= 50,
    dropout_rate=0.2,
    learning_rate=0.01
    )

lstm_model_1r80_7.save("lstm_models_v4/lstm_model_1r80_7.keras") # Save the trained model

Epoch 1/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - loss: 0.0813 - mae: 0.2121 - val_loss: 0.1038 - val_mae: 0.2633
Epoch 2/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0179 - mae: 0.1029 - val_loss: 0.0701 - val_mae: 0.2230
Epoch 3/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0135 - mae: 0.0888 - val_loss: 0.0588 - val_mae: 0.2028
Epoch 4/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0111 - mae: 0.0795 - val_loss: 0.0451 - val_mae: 0.1800
Epoch 5/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0107 - mae: 0.0762 - val_loss: 0.0339 - val_mae: 0.1558
Epoch 6/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0094 - mae: 0.0729 - val_loss: 0.0289 - val_mae: 0.1439
Epoch 7/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0097 - mae: 0.0747 - val_loss: 0.0205 - val_mae: 0.1200
Epoch 8/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0090 - mae: 0.0709 - val_loss: 0.0201 - val_mae: 0.1191
Epoch 9/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0082 - mae:

In [100]:
# Train LSTM model
lstm_model_1r80_8 = create_lstm_model(
    df_scaled=df_selected_1r80_8, 
    no_columns=len(selected_stations_1r80_8),
    n_steps=24,
    epochs=50,
    batch_size=32,
    units_1= 50,
    units_2= 50,
    dropout_rate=0.2,
    learning_rate=0.01
    )

lstm_model_1r80_8.save("lstm_models_v4/lstm_model_1r80_8.keras") # Save the trained model

Epoch 1/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - loss: 0.1217 - mae: 0.2422 - val_loss: 0.0527 - val_mae: 0.1935
Epoch 2/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0171 - mae: 0.0995 - val_loss: 0.0572 - val_mae: 0.2005
Epoch 3/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0147 - mae: 0.0914 - val_loss: 0.0457 - val_mae: 0.1782
Epoch 4/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0123 - mae: 0.0839 - val_loss: 0.0439 - val_mae: 0.1730
Epoch 5/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0101 - mae: 0.0753 - val_loss: 0.0324 - val_mae: 0.1512
Epoch 6/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0099 - mae: 0.0757 - val_loss: 0.0223 - val_mae: 0.1238
Epoch 7/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0093 - mae: 0.0724 - val_loss: 0.0202 - val_mae: 0.1189
Epoch 8/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0088 - mae: 0.0701 - val_loss: 0.0172 - val_mae: 0.1028
Epoch 9/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0085 - mae:

In [101]:
# Train LSTM model
lstm_model_1r80_9 = create_lstm_model(
    df_scaled=df_selected_1r80_9, 
    no_columns=len(selected_stations_1r80_9),
    n_steps=24,
    epochs=50,
    batch_size=32,
    units_1= 50,
    units_2= 50,
    dropout_rate=0.2,
    learning_rate=0.01
    )

lstm_model_1r80_9.save("lstm_models_v4/lstm_model_1r80_9.keras") # Save the trained model

Epoch 1/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - loss: 0.0713 - mae: 0.1976 - val_loss: 0.0798 - val_mae: 0.2334
Epoch 2/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0154 - mae: 0.0948 - val_loss: 0.0490 - val_mae: 0.1894
Epoch 3/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0126 - mae: 0.0841 - val_loss: 0.0433 - val_mae: 0.1760
Epoch 4/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0099 - mae: 0.0750 - val_loss: 0.0321 - val_mae: 0.1516
Epoch 5/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0090 - mae: 0.0704 - val_loss: 0.0287 - val_mae: 0.1436
Epoch 6/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0080 - mae: 0.0671 - val_loss: 0.0232 - val_mae: 0.1271
Epoch 7/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0085 - mae: 0.0687 - val_loss: 0.0168 - val_mae: 0.1070
Epoch 8/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0088 - mae: 0.0693 - val_loss: 0.0131 - val_mae: 0.0933
Epoch 9/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0072 - mae:

In [102]:
# Train LSTM model
lstm_model_1r80_10 = create_lstm_model(
    df_scaled=df_selected_1r80_10, 
    no_columns=len(selected_stations_1r80_10),
    n_steps=24,
    epochs=50,
    batch_size=32,
    units_1= 50,
    units_2= 50,
    dropout_rate=0.2,
    learning_rate=0.01
    )

lstm_model_1r80_10.save("lstm_models_v4/lstm_model_1r80_10.keras") # Save the trained model

Epoch 1/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - loss: 0.1027 - mae: 0.2321 - val_loss: 0.0770 - val_mae: 0.2253
Epoch 2/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0186 - mae: 0.1052 - val_loss: 0.0665 - val_mae: 0.2073
Epoch 3/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0131 - mae: 0.0857 - val_loss: 0.0546 - val_mae: 0.1902
Epoch 4/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0117 - mae: 0.0801 - val_loss: 0.0415 - val_mae: 0.1707
Epoch 5/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0106 - mae: 0.0770 - val_loss: 0.0367 - val_mae: 0.1554
Epoch 6/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0105 - mae: 0.0756 - val_loss: 0.0293 - val_mae: 0.1396
Epoch 7/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0088 - mae: 0.0690 - val_loss: 0.0311 - val_mae: 0.1439
Epoch 8/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0088 - mae: 0.0699 - val_loss: 0.0195 - val_mae: 0.1112
Epoch 9/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0090 - mae:

In [50]:
# Train LSTM model
lstm_model_1r80_30 = create_lstm_model(
    df_scaled=df_selected_1r80_30, 
    no_columns=len(selected_stations_1r80_30),
    n_steps=24,
    epochs=50,
    batch_size=32,
    units_1= 100,
    units_2= 50,
    dropout_rate=0.2,
    learning_rate=0.01
    )

lstm_model_1r80_30.save("lstm_models_v4/lstm_model_1r80_30.keras") # Save the trained model

Epoch 1/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 5s 16ms/step - loss: 0.1104 - mae: 0.2236 - val_loss: 0.0463 - val_mae: 0.1734
Epoch 2/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 0.0146 - mae: 0.0912 - val_loss: 0.0435 - val_mae: 0.1671
Epoch 3/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 0.0114 - mae: 0.0804 - val_loss: 0.0308 - val_mae: 0.1397
Epoch 4/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 0.0108 - mae: 0.0777 - val_loss: 0.0409 - val_mae: 0.1561
Epoch 5/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 0.0100 - mae: 0.0739 - val_loss: 0.0294 - val_mae: 0.1320
Epoch 6/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 0.0099 - mae: 0.0740 - val_loss: 0.0293 - val_mae: 0.1310
Epoch 7/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 0.0093 - mae: 0.0716 - val_loss: 0.0189 - val_mae: 0.1038
Epoch 8/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 0.0088 - mae: 0.0698 - val_loss: 0.0163 - val_mae: 0.0949
Epoch 9/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 0.008

#### 1 > r > 0.75

In [78]:
# Train LSTM model
lstm_model_1r75_1 = create_lstm_model(
    df_scaled=df_selected_1r75_1, 
    no_columns=len(selected_stations_1r75_1),
    n_steps=24,
    epochs=50,
    batch_size=32,
    units_1= 50,
    units_2= 50,
    dropout_rate=0.2,
    learning_rate=0.01
    )

lstm_model_1r75_1.save("lstm_models_v4/lstm_model_1r75_1.keras") # Save the trained model

Epoch 1/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - loss: 0.1384 - mae: 0.2689 - val_loss: 0.1254 - val_mae: 0.2957
Epoch 2/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0254 - mae: 0.1270 - val_loss: 0.1193 - val_mae: 0.2890
Epoch 3/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0200 - mae: 0.1111 - val_loss: 0.1033 - val_mae: 0.2696
Epoch 4/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0176 - mae: 0.1049 - val_loss: 0.0879 - val_mae: 0.2517
Epoch 5/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0144 - mae: 0.0950 - val_loss: 0.0848 - val_mae: 0.2474
Epoch 6/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0128 - mae: 0.0880 - val_loss: 0.0703 - val_mae: 0.2266
Epoch 7/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0116 - mae: 0.0845 - val_loss: 0.0587 - val_mae: 0.2085
Epoch 8/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0113 - mae: 0.0820 - val_loss: 0.0525 - val_mae: 0.1960
Epoch 9/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0097 - mae

In [79]:
# Train LSTM model
lstm_model_1r75_2 = create_lstm_model(
    df_scaled=df_selected_1r75_2, 
    no_columns=len(selected_stations_1r75_2),
    n_steps=24,
    epochs=50,
    batch_size=32,
    units_1= 50,
    units_2= 50,
    dropout_rate=0.2,
    learning_rate=0.01
    )

lstm_model_1r75_2.save("lstm_models_v4/lstm_model_1r75_2.keras") # Save the trained model

Epoch 1/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - loss: 0.1249 - mae: 0.2652 - val_loss: 0.0899 - val_mae: 0.2576
Epoch 2/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0256 - mae: 0.1266 - val_loss: 0.0802 - val_mae: 0.2460
Epoch 3/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0186 - mae: 0.1069 - val_loss: 0.0723 - val_mae: 0.2346
Epoch 4/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0137 - mae: 0.0914 - val_loss: 0.0680 - val_mae: 0.2276
Epoch 5/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0130 - mae: 0.0906 - val_loss: 0.0591 - val_mae: 0.2129
Epoch 6/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0116 - mae: 0.0840 - val_loss: 0.0533 - val_mae: 0.2018
Epoch 7/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0106 - mae: 0.0808 - val_loss: 0.0398 - val_mae: 0.1741
Epoch 8/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0100 - mae: 0.0777 - val_loss: 0.0358 - val_mae: 0.1662
Epoch 9/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0096 - mae

In [80]:
# Train LSTM model
lstm_model_1r75_3 = create_lstm_model(
    df_scaled=df_selected_1r75_3, 
    no_columns=len(selected_stations_1r75_3),
    n_steps=24,
    epochs=50,
    batch_size=32,
    units_1= 50,
    units_2= 50,
    dropout_rate=0.2,
    learning_rate=0.01
    )

lstm_model_1r75_3.save("lstm_models_v4/lstm_model_1r75_3.keras") # Save the trained model

Epoch 1/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - loss: 0.1062 - mae: 0.2370 - val_loss: 0.1222 - val_mae: 0.2868
Epoch 2/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0197 - mae: 0.1095 - val_loss: 0.0903 - val_mae: 0.2503
Epoch 3/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0158 - mae: 0.0974 - val_loss: 0.0729 - val_mae: 0.2256
Epoch 4/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0135 - mae: 0.0908 - val_loss: 0.0710 - val_mae: 0.2224
Epoch 5/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0125 - mae: 0.0874 - val_loss: 0.0572 - val_mae: 0.2017
Epoch 6/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0104 - mae: 0.0790 - val_loss: 0.0430 - val_mae: 0.1779
Epoch 7/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0113 - mae: 0.0825 - val_loss: 0.0284 - val_mae: 0.1446
Epoch 8/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0094 - mae: 0.0760 - val_loss: 0.0259 - val_mae: 0.1372
Epoch 9/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0092 - mae:

In [81]:
# Train LSTM model
lstm_model_1r75_4 = create_lstm_model(
    df_scaled=df_selected_1r75_4, 
    no_columns=len(selected_stations_1r75_4),
    n_steps=24,
    epochs=50,
    batch_size=32,
    units_1= 50,
    units_2= 50,
    dropout_rate=0.2,
    learning_rate=0.01
    )

lstm_model_1r75_4.save("lstm_models_v4/lstm_model_1r75_4.keras") # Save the trained model

Epoch 1/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - loss: 0.1194 - mae: 0.2400 - val_loss: 0.1047 - val_mae: 0.2681
Epoch 2/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0191 - mae: 0.1080 - val_loss: 0.0807 - val_mae: 0.2364
Epoch 3/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0136 - mae: 0.0897 - val_loss: 0.0653 - val_mae: 0.2152
Epoch 4/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0122 - mae: 0.0847 - val_loss: 0.0524 - val_mae: 0.1947
Epoch 5/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0119 - mae: 0.0820 - val_loss: 0.0470 - val_mae: 0.1847
Epoch 6/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0105 - mae: 0.0778 - val_loss: 0.0378 - val_mae: 0.1660
Epoch 7/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0100 - mae: 0.0769 - val_loss: 0.0316 - val_mae: 0.1497
Epoch 8/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0098 - mae: 0.0757 - val_loss: 0.0241 - val_mae: 0.1341
Epoch 9/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0088 - mae:

In [92]:
# Train LSTM model
lstm_model_1r75_5 = create_lstm_model(
    df_scaled=df_selected_1r75_5, 
    no_columns=len(selected_stations_1r75_5),
    n_steps=24,
    epochs=50,
    batch_size=32,
    units_1= 100,
    units_2= 50,
    dropout_rate=0.2,
    learning_rate=0.01
    )

lstm_model_1r75_5.save("lstm_models_v4/lstm_model_1r75_5.keras") # Save the trained model

Epoch 1/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - loss: 0.1243 - mae: 0.2537 - val_loss: 0.0864 - val_mae: 0.2470
Epoch 2/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 0.0208 - mae: 0.1112 - val_loss: 0.0636 - val_mae: 0.2169
Epoch 3/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 0.0143 - mae: 0.0916 - val_loss: 0.0501 - val_mae: 0.1934
Epoch 4/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 0.0113 - mae: 0.0808 - val_loss: 0.0468 - val_mae: 0.1858
Epoch 5/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 0.0104 - mae: 0.0767 - val_loss: 0.0431 - val_mae: 0.1759
Epoch 6/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 0.0098 - mae: 0.0751 - val_loss: 0.0334 - val_mae: 0.1575
Epoch 7/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 0.0098 - mae: 0.0747 - val_loss: 0.0324 - val_mae: 0.1536
Epoch 8/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 0.0095 - mae: 0.0734 - val_loss: 0.0258 - val_mae: 0.1359
Epoch 9/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - loss: 0.008

In [83]:
# Train LSTM model
lstm_model_1r75_6 = create_lstm_model(
    df_scaled=df_selected_1r75_6, 
    no_columns=len(selected_stations_1r75_6),
    n_steps=24,
    epochs=50,
    batch_size=32,
    units_1= 50,
    units_2= 50,
    dropout_rate=0.2,
    learning_rate=0.01
    )

lstm_model_1r75_6.save("lstm_models_v4/lstm_model_1r75_6.keras") # Save the trained model

Epoch 1/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - loss: 0.0809 - mae: 0.2102 - val_loss: 0.1055 - val_mae: 0.2679
Epoch 2/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0187 - mae: 0.1056 - val_loss: 0.0795 - val_mae: 0.2367
Epoch 3/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0141 - mae: 0.0906 - val_loss: 0.0607 - val_mae: 0.2092
Epoch 4/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0118 - mae: 0.0827 - val_loss: 0.0471 - val_mae: 0.1865
Epoch 5/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0100 - mae: 0.0756 - val_loss: 0.0436 - val_mae: 0.1789
Epoch 6/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0098 - mae: 0.0741 - val_loss: 0.0282 - val_mae: 0.1423
Epoch 7/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0102 - mae: 0.0758 - val_loss: 0.0292 - val_mae: 0.1445
Epoch 8/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0105 - mae: 0.0783 - val_loss: 0.0203 - val_mae: 0.1189
Epoch 9/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0097 - mae:

In [84]:
# Train LSTM model
lstm_model_1r75_7 = create_lstm_model(
    df_scaled=df_selected_1r75_7, 
    no_columns=len(selected_stations_1r75_7),
    n_steps=24,
    epochs=50,
    batch_size=32,
    units_1= 50,
    units_2= 50,
    dropout_rate=0.2,
    learning_rate=0.01
    )

lstm_model_1r75_7.save("lstm_models_v4/lstm_model_1r75_7.keras") # Save the trained model

Epoch 1/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - loss: 0.1071 - mae: 0.2432 - val_loss: 0.0816 - val_mae: 0.2421
Epoch 2/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0204 - mae: 0.1121 - val_loss: 0.0669 - val_mae: 0.2210
Epoch 3/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0139 - mae: 0.0904 - val_loss: 0.0488 - val_mae: 0.1903
Epoch 4/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0133 - mae: 0.0872 - val_loss: 0.0410 - val_mae: 0.1756
Epoch 5/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0104 - mae: 0.0774 - val_loss: 0.0359 - val_mae: 0.1624
Epoch 6/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0101 - mae: 0.0753 - val_loss: 0.0301 - val_mae: 0.1488
Epoch 7/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0090 - mae: 0.0722 - val_loss: 0.0268 - val_mae: 0.1383
Epoch 8/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0091 - mae: 0.0728 - val_loss: 0.0166 - val_mae: 0.1086
Epoch 9/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0091 - mae:

In [85]:
# Train LSTM model
lstm_model_1r75_8 = create_lstm_model(
    df_scaled=df_selected_1r75_8, 
    no_columns=len(selected_stations_1r75_8),
    n_steps=24,
    epochs=50,
    batch_size=32,
    units_1= 50,
    units_2= 50,
    dropout_rate=0.2,
    learning_rate=0.01
    )

lstm_model_1r75_8.save("lstm_models_v4/lstm_model_1r75_8.keras") # Save the trained model

Epoch 1/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - loss: 0.1135 - mae: 0.2392 - val_loss: 0.0727 - val_mae: 0.2287
Epoch 2/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0185 - mae: 0.1041 - val_loss: 0.0573 - val_mae: 0.2051
Epoch 3/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0142 - mae: 0.0897 - val_loss: 0.0530 - val_mae: 0.1983
Epoch 4/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0109 - mae: 0.0790 - val_loss: 0.0486 - val_mae: 0.1910
Epoch 5/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0105 - mae: 0.0779 - val_loss: 0.0380 - val_mae: 0.1684
Epoch 6/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0106 - mae: 0.0778 - val_loss: 0.0379 - val_mae: 0.1685
Epoch 7/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0090 - mae: 0.0723 - val_loss: 0.0206 - val_mae: 0.1194
Epoch 8/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0086 - mae: 0.0701 - val_loss: 0.0162 - val_mae: 0.1060
Epoch 9/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0083 - mae:

In [86]:
# Train LSTM model
lstm_model_1r75_9 = create_lstm_model(
    df_scaled=df_selected_1r75_9, 
    no_columns=len(selected_stations_1r75_9),
    n_steps=24,
    epochs=50,
    batch_size=32,
    units_1= 50,
    units_2= 50,
    dropout_rate=0.2,
    learning_rate=0.01
    )

lstm_model_1r75_9.save("lstm_models_v4/lstm_model_1r75_9.keras") # Save the trained model

Epoch 1/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - loss: 0.0922 - mae: 0.2273 - val_loss: 0.0723 - val_mae: 0.2297
Epoch 2/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0186 - mae: 0.1056 - val_loss: 0.0597 - val_mae: 0.2111
Epoch 3/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0140 - mae: 0.0905 - val_loss: 0.0487 - val_mae: 0.1900
Epoch 4/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0116 - mae: 0.0819 - val_loss: 0.0432 - val_mae: 0.1779
Epoch 5/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0101 - mae: 0.0760 - val_loss: 0.0365 - val_mae: 0.1627
Epoch 6/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0100 - mae: 0.0760 - val_loss: 0.0298 - val_mae: 0.1457
Epoch 7/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0099 - mae: 0.0751 - val_loss: 0.0254 - val_mae: 0.1354
Epoch 8/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0085 - mae: 0.0695 - val_loss: 0.0202 - val_mae: 0.1185
Epoch 9/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0086 - mae:

In [87]:
# Train LSTM model
lstm_model_1r75_10 = create_lstm_model(
    df_scaled=df_selected_1r75_10, 
    no_columns=len(selected_stations_1r75_10),
    n_steps=24,
    epochs=50,
    batch_size=32,
    units_1= 50,
    units_2= 50,
    dropout_rate=0.2,
    learning_rate=0.01
    )

lstm_model_1r75_10.save("lstm_models_v4/lstm_model_1r75_10.keras") # Save the trained model

Epoch 1/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - loss: 0.1358 - mae: 0.2568 - val_loss: 0.0712 - val_mae: 0.2209
Epoch 2/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0198 - mae: 0.1092 - val_loss: 0.0640 - val_mae: 0.2090
Epoch 3/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0146 - mae: 0.0920 - val_loss: 0.0533 - val_mae: 0.1933
Epoch 4/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0115 - mae: 0.0802 - val_loss: 0.0422 - val_mae: 0.1717
Epoch 5/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0108 - mae: 0.0781 - val_loss: 0.0328 - val_mae: 0.1525
Epoch 6/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0114 - mae: 0.0796 - val_loss: 0.0296 - val_mae: 0.1431
Epoch 7/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0093 - mae: 0.0721 - val_loss: 0.0293 - val_mae: 0.1405
Epoch 8/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0092 - mae: 0.0712 - val_loss: 0.0205 - val_mae: 0.1164
Epoch 9/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0088 - mae:

In [51]:
# Train LSTM model
lstm_model_1r75_30 = create_lstm_model(
    df_scaled=df_selected_1r75_30, 
    no_columns=len(selected_stations_1r75_30),
    n_steps=24,
    epochs=50,
    batch_size=32,
    units_1= 100,
    units_2= 50,
    dropout_rate=0.2,
    learning_rate=0.01
    )

lstm_model_1r75_30.save("lstm_models_v4/lstm_model_1r75_30.keras") # Save the trained model

Epoch 1/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 3s 17ms/step - loss: 0.0879 - mae: 0.2091 - val_loss: 0.0568 - val_mae: 0.1879
Epoch 2/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 0.0147 - mae: 0.0924 - val_loss: 0.0555 - val_mae: 0.1867
Epoch 3/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 0.0115 - mae: 0.0807 - val_loss: 0.0473 - val_mae: 0.1698
Epoch 4/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - loss: 0.0106 - mae: 0.0768 - val_loss: 0.0461 - val_mae: 0.1673
Epoch 5/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 0.0096 - mae: 0.0726 - val_loss: 0.0341 - val_mae: 0.1448
Epoch 6/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 0.0087 - mae: 0.0690 - val_loss: 0.0279 - val_mae: 0.1270
Epoch 7/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 0.0085 - mae: 0.0677 - val_loss: 0.0294 - val_mae: 0.1318
Epoch 8/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 0.0085 - mae: 0.0672 - val_loss: 0.0269 - val_mae: 0.1265
Epoch 9/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 0.008

### GRU

#### 1 > r > 0.95

In [77]:
# Train GRU model
gru_model_1r95_1 = create_gru_model(
    df_scaled=df_selected_1r95_1, 
    no_columns=len(selected_stations_1r95_1),
    n_steps=24,
    epochs=100,
    batch_size=32,
    units_1=50,
    units_2=50,
    dropout_rate=0.2,
    learning_rate=0.01
)

gru_model_1r95_1.save("gru_models/gru_model_1r95_1.keras")  # Save the trained model


Epoch 1/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - loss: 0.0521 - mae: 0.1714 - val_loss: 0.0821 - val_mae: 0.2323
Epoch 2/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0125 - mae: 0.0865 - val_loss: 0.0685 - val_mae: 0.2173
Epoch 3/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0109 - mae: 0.0801 - val_loss: 0.0615 - val_mae: 0.2085
Epoch 4/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0088 - mae: 0.0725 - val_loss: 0.0530 - val_mae: 0.1933
Epoch 5/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0071 - mae: 0.0640 - val_loss: 0.0490 - val_mae: 0.1882
Epoch 6/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0075 - mae: 0.0646 - val_loss: 0.0483 - val_mae: 0.1861
Epoch 7/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0063 - mae: 0.0615 - val_loss: 0.0393 - val_mae: 0.1680
Epoch 8/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0063 - mae: 0.0599 - val_loss: 0.0254 - val_mae: 0.1312
Epoch 9/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.00

In [78]:
# Train GRU model
gru_model_1r95_2 = create_gru_model(
    df_scaled=df_selected_1r95_2, 
    no_columns=len(selected_stations_1r95_2),
    n_steps=24,
    epochs=100,
    batch_size=32,
    units_1=50,
    units_2=50,
    dropout_rate=0.2,
    learning_rate=0.01
)

gru_model_1r95_2.save("gru_models/gru_model_1r95_2.keras")  # Save the trained model


Epoch 1/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - loss: 0.0706 - mae: 0.1901 - val_loss: 0.0990 - val_mae: 0.2507
Epoch 2/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0193 - mae: 0.1074 - val_loss: 0.0929 - val_mae: 0.2438
Epoch 3/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0157 - mae: 0.0953 - val_loss: 0.0639 - val_mae: 0.2074
Epoch 4/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0132 - mae: 0.0860 - val_loss: 0.0612 - val_mae: 0.2042
Epoch 5/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0111 - mae: 0.0798 - val_loss: 0.0490 - val_mae: 0.1862
Epoch 6/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0094 - mae: 0.0742 - val_loss: 0.0440 - val_mae: 0.1784
Epoch 7/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0085 - mae: 0.0699 - val_loss: 0.0332 - val_mae: 0.1562
Epoch 8/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0082 - mae: 0.0697 - val_loss: 0.0352 - val_mae: 0.1579
Epoch 9/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0

In [135]:
# Train GRU model
gru_model_1r95_3 = create_gru_model(
    df_scaled=df_selected_1r95_3, 
    no_columns=len(selected_stations_1r95_3),
    n_steps=24,
    epochs=100,
    batch_size=32,
    units_1=50,
    units_2=50,
    dropout_rate=0.2,
    learning_rate=0.01
)

gru_model_1r95_3.save("gru_models/gru_model_1r95_3.keras")  # Save the trained model


Epoch 1/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - loss: 0.1110 - mae: 0.2329 - val_loss: 0.0693 - val_mae: 0.2193
Epoch 2/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0207 - mae: 0.1097 - val_loss: 0.0630 - val_mae: 0.2109
Epoch 3/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0134 - mae: 0.0879 - val_loss: 0.0474 - val_mae: 0.1872
Epoch 4/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0141 - mae: 0.0893 - val_loss: 0.0431 - val_mae: 0.1775
Epoch 5/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0103 - mae: 0.0768 - val_loss: 0.0459 - val_mae: 0.1830
Epoch 6/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0104 - mae: 0.0777 - val_loss: 0.0414 - val_mae: 0.1728
Epoch 7/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0100 - mae: 0.0758 - val_loss: 0.0286 - val_mae: 0.1449
Epoch 8/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0110 - mae: 0.0796 - val_loss: 0.0208 - val_mae: 0.1200
Epoch 9/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.00

In [138]:
# Train GRU model
gru_model_1r95_4 = create_gru_model(
    df_scaled=df_selected_1r95_4, 
    no_columns=len(selected_stations_1r95_4),
    n_steps=24,
    epochs=100,
    batch_size=32,
    units_1=50,
    units_2=50,
    dropout_rate=0.2,
    learning_rate=0.01
)

gru_model_1r95_4.save("gru_models/gru_model_1r95_4.keras")  # Save the trained model


Epoch 1/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - loss: 0.0687 - mae: 0.1869 - val_loss: 0.0598 - val_mae: 0.2026
Epoch 2/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0167 - mae: 0.0986 - val_loss: 0.0602 - val_mae: 0.2011
Epoch 3/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0129 - mae: 0.0853 - val_loss: 0.0556 - val_mae: 0.1965
Epoch 4/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0109 - mae: 0.0774 - val_loss: 0.0399 - val_mae: 0.1675
Epoch 5/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0095 - mae: 0.0730 - val_loss: 0.0366 - val_mae: 0.1625
Epoch 6/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0078 - mae: 0.0668 - val_loss: 0.0317 - val_mae: 0.1502
Epoch 7/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0084 - mae: 0.0680 - val_loss: 0.0202 - val_mae: 0.1180
Epoch 8/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0086 - mae: 0.0701 - val_loss: 0.0150 - val_mae: 0.1014
Epoch 9/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.

In [81]:
# Train GRU model
gru_model_1r95_5 = create_gru_model(
    df_scaled=df_selected_1r95_5, 
    no_columns=len(selected_stations_1r95_5),
    n_steps=24,
    epochs=100,
    batch_size=32,
    units_1=50,
    units_2=50,
    dropout_rate=0.2,
    learning_rate=0.01
)

gru_model_1r95_5.save("gru_models/gru_model_1r95_5.keras")  # Save the trained model


Epoch 1/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - loss: 0.0752 - mae: 0.2012 - val_loss: 0.0719 - val_mae: 0.2178
Epoch 2/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 0.0172 - mae: 0.0981 - val_loss: 0.0610 - val_mae: 0.2028
Epoch 3/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0125 - mae: 0.0845 - val_loss: 0.0476 - val_mae: 0.1810
Epoch 4/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0113 - mae: 0.0790 - val_loss: 0.0332 - val_mae: 0.1552
Epoch 5/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0102 - mae: 0.0762 - val_loss: 0.0286 - val_mae: 0.1440
Epoch 6/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0087 - mae: 0.0708 - val_loss: 0.0277 - val_mae: 0.1399
Epoch 7/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0075 - mae: 0.0662 - val_loss: 0.0147 - val_mae: 0.1026
Epoch 8/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0078 - mae: 0.0663 - val_loss: 0.0114 - val_mae: 0.0889
Epoch 9/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.

In [82]:
# Train GRU model
gru_model_1r95_6 = create_gru_model(
    df_scaled=df_selected_1r95_6, 
    no_columns=len(selected_stations_1r95_6),
    n_steps=24,
    epochs=100,
    batch_size=32,
    units_1=50,
    units_2=50,
    dropout_rate=0.2,
    learning_rate=0.01
)

gru_model_1r95_6.save("gru_models/gru_model_1r95_6.keras")  # Save the trained model


Epoch 1/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 5s 12ms/step - loss: 0.0774 - mae: 0.1929 - val_loss: 0.0496 - val_mae: 0.1884
Epoch 2/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0179 - mae: 0.1015 - val_loss: 0.0435 - val_mae: 0.1740
Epoch 3/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0116 - mae: 0.0806 - val_loss: 0.0443 - val_mae: 0.1741
Epoch 4/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0100 - mae: 0.0735 - val_loss: 0.0267 - val_mae: 0.1381
Epoch 5/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0093 - mae: 0.0720 - val_loss: 0.0270 - val_mae: 0.1364
Epoch 6/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0088 - mae: 0.0699 - val_loss: 0.0247 - val_mae: 0.1322
Epoch 7/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0083 - mae: 0.0681 - val_loss: 0.0161 - val_mae: 0.1055
Epoch 8/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0079 - mae: 0.0685 - val_loss: 0.0152 - val_mae: 0.0987
Epoch 9/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.00

In [136]:
# Train GRU model
gru_model_1r95_7 = create_gru_model(
    df_scaled=df_selected_1r95_7, 
    no_columns=len(selected_stations_1r95_7),
    n_steps=24,
    epochs=100,
    batch_size=32,
    units_1=50,
    units_2=50,
    dropout_rate=0.2,
    learning_rate=0.01
)

gru_model_1r95_7.save("gru_models/gru_model_1r95_7.keras")  # Save the trained model


Epoch 1/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - loss: 0.0717 - mae: 0.1942 - val_loss: 0.0554 - val_mae: 0.1945
Epoch 2/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0161 - mae: 0.0965 - val_loss: 0.0455 - val_mae: 0.1823
Epoch 3/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0136 - mae: 0.0883 - val_loss: 0.0425 - val_mae: 0.1745
Epoch 4/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0108 - mae: 0.0780 - val_loss: 0.0397 - val_mae: 0.1651
Epoch 5/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0103 - mae: 0.0753 - val_loss: 0.0253 - val_mae: 0.1319
Epoch 6/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0105 - mae: 0.0764 - val_loss: 0.0345 - val_mae: 0.1514
Epoch 7/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0093 - mae: 0.0728 - val_loss: 0.0199 - val_mae: 0.1115
Epoch 8/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0082 - mae: 0.0676 - val_loss: 0.0115 - val_mae: 0.0855
Epoch 9/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.00

In [84]:
# Train GRU model
gru_model_1r95_8 = create_gru_model(
    df_scaled=df_selected_1r95_8, 
    no_columns=len(selected_stations_1r95_8),
    n_steps=24,
    epochs=100,
    batch_size=32,
    units_1=50,
    units_2=50,
    dropout_rate=0.2,
    learning_rate=0.01
)

gru_model_1r95_8.save("gru_models/gru_model_1r95_8.keras")  # Save the trained model


Epoch 1/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - loss: 0.0994 - mae: 0.2205 - val_loss: 0.0443 - val_mae: 0.1816
Epoch 2/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0181 - mae: 0.1005 - val_loss: 0.0387 - val_mae: 0.1671
Epoch 3/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0123 - mae: 0.0836 - val_loss: 0.0282 - val_mae: 0.1415
Epoch 4/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0111 - mae: 0.0787 - val_loss: 0.0210 - val_mae: 0.1220
Epoch 5/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0117 - mae: 0.0814 - val_loss: 0.0158 - val_mae: 0.1025
Epoch 6/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0093 - mae: 0.0731 - val_loss: 0.0120 - val_mae: 0.0876
Epoch 7/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0092 - mae: 0.0710 - val_loss: 0.0129 - val_mae: 0.0954
Epoch 8/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0085 - mae: 0.0699 - val_loss: 0.0100 - val_mae: 0.0800
Epoch 9/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.

In [85]:
# Train GRU model
gru_model_1r95_9 = create_gru_model(
    df_scaled=df_selected_1r95_9, 
    no_columns=len(selected_stations_1r95_9),
    n_steps=24,
    epochs=100,
    batch_size=32,
    units_1=50,
    units_2=50,
    dropout_rate=0.2,
    learning_rate=0.01
)

gru_model_1r95_9.save("gru_models/gru_model_1r95_9.keras")  # Save the trained model


Epoch 1/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - loss: 0.0980 - mae: 0.2192 - val_loss: 0.0583 - val_mae: 0.2004
Epoch 2/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0188 - mae: 0.1046 - val_loss: 0.0385 - val_mae: 0.1676
Epoch 3/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0144 - mae: 0.0911 - val_loss: 0.0315 - val_mae: 0.1477
Epoch 4/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0127 - mae: 0.0842 - val_loss: 0.0240 - val_mae: 0.1290
Epoch 5/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0102 - mae: 0.0759 - val_loss: 0.0246 - val_mae: 0.1274
Epoch 6/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0095 - mae: 0.0730 - val_loss: 0.0153 - val_mae: 0.0989
Epoch 7/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0091 - mae: 0.0708 - val_loss: 0.0158 - val_mae: 0.0959
Epoch 8/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0088 - mae: 0.0706 - val_loss: 0.0142 - val_mae: 0.0899
Epoch 9/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.01

In [137]:
# Train GRU model
gru_model_1r95_10 = create_gru_model(
    df_scaled=df_selected_1r95_10, 
    no_columns=len(selected_stations_1r95_10),
    n_steps=24,
    epochs=100,
    batch_size=32,
    units_1=50,
    units_2=50,
    dropout_rate=0.2,
    learning_rate=0.01
)

gru_model_1r95_10.save("gru_models/gru_model_1r95_10.keras")  # Save the trained model


Epoch 1/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - loss: 0.1172 - mae: 0.2353 - val_loss: 0.0423 - val_mae: 0.1752
Epoch 2/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0190 - mae: 0.1028 - val_loss: 0.0319 - val_mae: 0.1534
Epoch 3/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0149 - mae: 0.0903 - val_loss: 0.0286 - val_mae: 0.1433
Epoch 4/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0135 - mae: 0.0858 - val_loss: 0.0207 - val_mae: 0.1210
Epoch 5/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0120 - mae: 0.0813 - val_loss: 0.0181 - val_mae: 0.1115
Epoch 6/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0106 - mae: 0.0773 - val_loss: 0.0163 - val_mae: 0.1037
Epoch 7/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0102 - mae: 0.0749 - val_loss: 0.0095 - val_mae: 0.0766
Epoch 8/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0094 - mae: 0.0732 - val_loss: 0.0105 - val_mae: 0.0844
Epoch 9/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.00

#### 1 > r > 0.90

In [129]:
# Train GRU model
gru_model_1r90_1 = create_gru_model(
    df_scaled=df_selected_1r90_1, 
    no_columns=len(selected_stations_1r90_1),
    n_steps=24,
    epochs=50,
    batch_size=32,
    units_1=50,
    units_2=50,
    dropout_rate=0.2,
    learning_rate=0.01
)

gru_model_1r90_1.save("gru_models/gru_model_1r90_1.keras")  # Save the trained model


Epoch 1/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - loss: 0.0792 - mae: 0.2039 - val_loss: 0.1120 - val_mae: 0.2716
Epoch 2/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0169 - mae: 0.1029 - val_loss: 0.0907 - val_mae: 0.2470
Epoch 3/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0121 - mae: 0.0862 - val_loss: 0.0746 - val_mae: 0.2271
Epoch 4/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0099 - mae: 0.0767 - val_loss: 0.0734 - val_mae: 0.2247
Epoch 5/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0091 - mae: 0.0748 - val_loss: 0.0471 - val_mae: 0.1850
Epoch 6/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0097 - mae: 0.0757 - val_loss: 0.0549 - val_mae: 0.1966
Epoch 7/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0080 - mae: 0.0687 - val_loss: 0.0362 - val_mae: 0.1623
Epoch 8/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0078 - mae: 0.0678 - val_loss: 0.0329 - val_mae: 0.1546
Epoch 9/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0082 - mae:

In [127]:
# Train GRU model
gru_model_1r90_2 = create_gru_model(
    df_scaled=df_selected_1r90_2, 
    no_columns=len(selected_stations_1r90_2),
    n_steps=24,
    epochs=50,
    batch_size=32,
    units_1=50,
    units_2=50,
    dropout_rate=0.2,
    learning_rate=0.01
)

gru_model_1r90_2.save("gru_models/gru_model_1r90_2.keras")  # Save the trained model


Epoch 1/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 4s 14ms/step - loss: 0.0670 - mae: 0.1907 - val_loss: 0.0784 - val_mae: 0.2287
Epoch 2/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0144 - mae: 0.0935 - val_loss: 0.0635 - val_mae: 0.2069
Epoch 3/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0108 - mae: 0.0785 - val_loss: 0.0553 - val_mae: 0.1949
Epoch 4/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0092 - mae: 0.0725 - val_loss: 0.0350 - val_mae: 0.1571
Epoch 5/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0100 - mae: 0.0761 - val_loss: 0.0381 - val_mae: 0.1641
Epoch 6/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0079 - mae: 0.0676 - val_loss: 0.0354 - val_mae: 0.1593
Epoch 7/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0071 - mae: 0.0631 - val_loss: 0.0306 - val_mae: 0.1485
Epoch 8/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0065 - mae: 0.0604 - val_loss: 0.0294 - val_mae: 0.1428
Epoch 9/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0076 - mae:

In [148]:
# Train GRU model
gru_model_1r90_3 = create_gru_model(
    df_scaled=df_selected_1r90_3, 
    no_columns=len(selected_stations_1r90_3),
    n_steps=24,
    epochs=50,
    batch_size=32,
    units_1=50,
    units_2=50,
    dropout_rate=0.2,
    learning_rate=0.01
)

gru_model_1r90_3.save("gru_models/gru_model_1r90_3.keras")  # Save the trained model


Epoch 1/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 4s 16ms/step - loss: 0.0804 - mae: 0.2050 - val_loss: 0.0744 - val_mae: 0.2226
Epoch 2/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0164 - mae: 0.0989 - val_loss: 0.0639 - val_mae: 0.2075
Epoch 3/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0130 - mae: 0.0860 - val_loss: 0.0505 - val_mae: 0.1893
Epoch 4/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0099 - mae: 0.0737 - val_loss: 0.0535 - val_mae: 0.1943
Epoch 5/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0099 - mae: 0.0751 - val_loss: 0.0453 - val_mae: 0.1813
Epoch 6/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0088 - mae: 0.0708 - val_loss: 0.0309 - val_mae: 0.1509
Epoch 7/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0078 - mae: 0.0669 - val_loss: 0.0302 - val_mae: 0.1467
Epoch 8/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0076 - mae: 0.0653 - val_loss: 0.0188 - val_mae: 0.1175
Epoch 9/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0077 - m

In [147]:
# Train GRU model
gru_model_1r90_4 = create_gru_model(
    df_scaled=df_selected_1r90_4, 
    no_columns=len(selected_stations_1r90_4),
    n_steps=24,
    epochs=50,
    batch_size=32,
    units_1=50,
    units_2=50,
    dropout_rate=0.2,
    learning_rate=0.01
)

gru_model_1r90_4.save("gru_models/gru_model_1r90_4.keras")  # Save the trained model


Epoch 1/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 4s 18ms/step - loss: 0.0769 - mae: 0.1968 - val_loss: 0.0782 - val_mae: 0.2239
Epoch 2/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0180 - mae: 0.1020 - val_loss: 0.0515 - val_mae: 0.1940
Epoch 3/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0126 - mae: 0.0845 - val_loss: 0.0454 - val_mae: 0.1804
Epoch 4/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0102 - mae: 0.0761 - val_loss: 0.0330 - val_mae: 0.1569
Epoch 5/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0094 - mae: 0.0725 - val_loss: 0.0348 - val_mae: 0.1583
Epoch 6/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0098 - mae: 0.0745 - val_loss: 0.0271 - val_mae: 0.1408
Epoch 7/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0075 - mae: 0.0663 - val_loss: 0.0177 - val_mae: 0.1111
Epoch 8/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0074 - mae: 0.0647 - val_loss: 0.0164 - val_mae: 0.1058
Epoch 9/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0075 - mae

In [128]:
# Train GRU model
gru_model_1r90_5 = create_gru_model(
    df_scaled=df_selected_1r90_5, 
    no_columns=len(selected_stations_1r90_5),
    n_steps=24,
    epochs=50,
    batch_size=32,
    units_1=50,
    units_2=50,
    dropout_rate=0.2,
    learning_rate=0.01
)

gru_model_1r90_5.save("gru_models/gru_model_1r90_5.keras")  # Save the trained model


Epoch 1/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 4s 13ms/step - loss: 0.0738 - mae: 0.1944 - val_loss: 0.0654 - val_mae: 0.2097
Epoch 2/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0153 - mae: 0.0929 - val_loss: 0.0485 - val_mae: 0.1863
Epoch 3/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0126 - mae: 0.0838 - val_loss: 0.0415 - val_mae: 0.1721
Epoch 4/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0094 - mae: 0.0729 - val_loss: 0.0315 - val_mae: 0.1497
Epoch 5/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0111 - mae: 0.0782 - val_loss: 0.0291 - val_mae: 0.1427
Epoch 6/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0093 - mae: 0.0728 - val_loss: 0.0289 - val_mae: 0.1405
Epoch 7/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0089 - mae: 0.0725 - val_loss: 0.0113 - val_mae: 0.0857
Epoch 8/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0080 - mae: 0.0672 - val_loss: 0.0118 - val_mae: 0.0861
Epoch 9/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0081 - mae:

In [142]:
# Train GRU model
gru_model_1r90_6 = create_gru_model(
    df_scaled=df_selected_1r90_6, 
    no_columns=len(selected_stations_1r90_6),
    n_steps=24,
    epochs=50,
    batch_size=32,
    units_1=50,
    units_2=50,
    dropout_rate=0.2,
    learning_rate=0.01
)

gru_model_1r90_6.save("gru_models/gru_model_1r90_6.keras")  # Save the trained model


Epoch 1/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 4s 14ms/step - loss: 0.1184 - mae: 0.2314 - val_loss: 0.0466 - val_mae: 0.1840
Epoch 2/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0222 - mae: 0.1117 - val_loss: 0.0428 - val_mae: 0.1775
Epoch 3/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0132 - mae: 0.0861 - val_loss: 0.0344 - val_mae: 0.1582
Epoch 4/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0105 - mae: 0.0767 - val_loss: 0.0386 - val_mae: 0.1672
Epoch 5/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0107 - mae: 0.0769 - val_loss: 0.0304 - val_mae: 0.1460
Epoch 6/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0098 - mae: 0.0758 - val_loss: 0.0241 - val_mae: 0.1294
Epoch 7/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0084 - mae: 0.0695 - val_loss: 0.0106 - val_mae: 0.0831
Epoch 8/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0085 - mae: 0.0701 - val_loss: 0.0121 - val_mae: 0.0877
Epoch 9/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0077 - mae:

In [144]:
# Train GRU model
gru_model_1r90_7 = create_gru_model(
    df_scaled=df_selected_1r90_7, 
    no_columns=len(selected_stations_1r90_7),
    n_steps=24,
    epochs=50,
    batch_size=32,
    units_1=50,
    units_2=50,
    dropout_rate=0.2,
    learning_rate=0.01
)

gru_model_1r90_7.save("gru_models/gru_model_1r90_7.keras")  # Save the trained model


Epoch 1/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - loss: 0.0795 - mae: 0.2049 - val_loss: 0.0470 - val_mae: 0.1859
Epoch 2/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0164 - mae: 0.0961 - val_loss: 0.0428 - val_mae: 0.1759
Epoch 3/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0118 - mae: 0.0818 - val_loss: 0.0312 - val_mae: 0.1502
Epoch 4/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0109 - mae: 0.0790 - val_loss: 0.0237 - val_mae: 0.1289
Epoch 5/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0100 - mae: 0.0749 - val_loss: 0.0239 - val_mae: 0.1285
Epoch 6/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0093 - mae: 0.0715 - val_loss: 0.0223 - val_mae: 0.1241
Epoch 7/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0091 - mae: 0.0725 - val_loss: 0.0199 - val_mae: 0.1103
Epoch 8/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0079 - mae: 0.0667 - val_loss: 0.0084 - val_mae: 0.0708
Epoch 9/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0087 - mae

In [94]:
# Train GRU model
gru_model_1r90_8 = create_gru_model(
    df_scaled=df_selected_1r90_8, 
    no_columns=len(selected_stations_1r90_8),
    n_steps=24,
    epochs=50,
    batch_size=32,
    units_1=50,
    units_2=50,
    dropout_rate=0.2,
    learning_rate=0.01
)

gru_model_1r90_8.save("gru_models/gru_model_1r90_8.keras")  # Save the trained model


Epoch 1/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - loss: 0.0843 - mae: 0.2043 - val_loss: 0.0637 - val_mae: 0.2080
Epoch 2/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0163 - mae: 0.0971 - val_loss: 0.0381 - val_mae: 0.1656
Epoch 3/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0108 - mae: 0.0783 - val_loss: 0.0282 - val_mae: 0.1415
Epoch 4/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0091 - mae: 0.0721 - val_loss: 0.0296 - val_mae: 0.1459
Epoch 5/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0096 - mae: 0.0733 - val_loss: 0.0221 - val_mae: 0.1246
Epoch 6/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0085 - mae: 0.0694 - val_loss: 0.0191 - val_mae: 0.1136
Epoch 7/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0079 - mae: 0.0661 - val_loss: 0.0132 - val_mae: 0.0931
Epoch 8/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0082 - mae: 0.0682 - val_loss: 0.0114 - val_mae: 0.0823
Epoch 9/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0073 - mae:

In [95]:
# Train GRU model
gru_model_1r90_9 = create_gru_model(
    df_scaled=df_selected_1r90_9, 
    no_columns=len(selected_stations_1r90_9),
    n_steps=24,
    epochs=50,
    batch_size=32,
    units_1=50,
    units_2=50,
    dropout_rate=0.2,
    learning_rate=0.01
)

gru_model_1r90_9.save("gru_models/gru_model_1r90_9.keras")  # Save the trained model


Epoch 1/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - loss: 0.0995 - mae: 0.2122 - val_loss: 0.0439 - val_mae: 0.1804
Epoch 2/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0155 - mae: 0.0953 - val_loss: 0.0332 - val_mae: 0.1521
Epoch 3/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0137 - mae: 0.0896 - val_loss: 0.0346 - val_mae: 0.1533
Epoch 4/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0111 - mae: 0.0799 - val_loss: 0.0337 - val_mae: 0.1482
Epoch 5/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0097 - mae: 0.0739 - val_loss: 0.0216 - val_mae: 0.1181
Epoch 6/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0086 - mae: 0.0707 - val_loss: 0.0303 - val_mae: 0.1398
Epoch 7/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0087 - mae: 0.0709 - val_loss: 0.0173 - val_mae: 0.1013
Epoch 8/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0082 - mae: 0.0691 - val_loss: 0.0089 - val_mae: 0.0713
Epoch 9/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0081 - mae:

In [96]:
# Train GRU model
gru_model_1r90_10 = create_gru_model(
    df_scaled=df_selected_1r90_10, 
    no_columns=len(selected_stations_1r90_10),
    n_steps=24,
    epochs=50,
    batch_size=32,
    units_1=50,
    units_2=50,
    dropout_rate=0.2,
    learning_rate=0.01
)

gru_model_1r90_10.save("gru_models/gru_model_1r90_10.keras")  # Save the trained model


Epoch 1/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - loss: 0.0795 - mae: 0.2034 - val_loss: 0.0469 - val_mae: 0.1828
Epoch 2/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0149 - mae: 0.0943 - val_loss: 0.0270 - val_mae: 0.1373
Epoch 3/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0117 - mae: 0.0819 - val_loss: 0.0263 - val_mae: 0.1357
Epoch 4/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0109 - mae: 0.0778 - val_loss: 0.0174 - val_mae: 0.1059
Epoch 5/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0087 - mae: 0.0706 - val_loss: 0.0139 - val_mae: 0.0955
Epoch 6/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0089 - mae: 0.0699 - val_loss: 0.0131 - val_mae: 0.0911
Epoch 7/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0082 - mae: 0.0680 - val_loss: 0.0074 - val_mae: 0.0647
Epoch 8/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0080 - mae: 0.0669 - val_loss: 0.0074 - val_mae: 0.0650
Epoch 9/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0078 - mae:

#### 1 > r > 0.85

In [97]:
# Train GRU model
gru_model_1r85_1 = create_gru_model(
    df_scaled=df_selected_1r85_1, 
    no_columns=len(selected_stations_1r85_1),
    n_steps=24,
    epochs=50,
    batch_size=32,
    units_1=50,
    units_2=50,
    dropout_rate=0.2,
    learning_rate=0.01
)

gru_model_1r85_1.save("gru_models/gru_model_1r85_1.keras")  # Save the trained model


Epoch 1/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - loss: 0.1819 - mae: 0.2869 - val_loss: 0.1500 - val_mae: 0.3217
Epoch 2/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0207 - mae: 0.1129 - val_loss: 0.1181 - val_mae: 0.2867
Epoch 3/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0171 - mae: 0.1009 - val_loss: 0.1013 - val_mae: 0.2696
Epoch 4/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0140 - mae: 0.0923 - val_loss: 0.0814 - val_mae: 0.2421
Epoch 5/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0131 - mae: 0.0907 - val_loss: 0.0756 - val_mae: 0.2339
Epoch 6/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0108 - mae: 0.0804 - val_loss: 0.0619 - val_mae: 0.2137
Epoch 7/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0104 - mae: 0.0795 - val_loss: 0.0582 - val_mae: 0.2072
Epoch 8/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0104 - mae: 0.0782 - val_loss: 0.0488 - val_mae: 0.1898
Epoch 9/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0091 - mae

In [130]:
# Train GRU model
gru_model_1r85_2 = create_gru_model(
    df_scaled=df_selected_1r85_2, 
    no_columns=len(selected_stations_1r85_2),
    n_steps=24,
    epochs=50,
    batch_size=32,
    units_1=50,
    units_2=50,
    dropout_rate=0.2,
    learning_rate=0.01
)

gru_model_1r85_2.save("gru_models/gru_model_1r85_2.keras")  # Save the trained model


Epoch 1/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - loss: 0.1007 - mae: 0.2190 - val_loss: 0.0971 - val_mae: 0.2577
Epoch 2/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0162 - mae: 0.0978 - val_loss: 0.0842 - val_mae: 0.2420
Epoch 3/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0136 - mae: 0.0886 - val_loss: 0.0773 - val_mae: 0.2336
Epoch 4/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0112 - mae: 0.0807 - val_loss: 0.0605 - val_mae: 0.2085
Epoch 5/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0096 - mae: 0.0748 - val_loss: 0.0546 - val_mae: 0.1988
Epoch 6/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0081 - mae: 0.0693 - val_loss: 0.0417 - val_mae: 0.1742
Epoch 7/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0094 - mae: 0.0756 - val_loss: 0.0391 - val_mae: 0.1689
Epoch 8/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0089 - mae: 0.0721 - val_loss: 0.0310 - val_mae: 0.1498
Epoch 9/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0081 - mae:

In [99]:
# Train GRU model
gru_model_1r85_3 = create_gru_model(
    df_scaled=df_selected_1r85_3, 
    no_columns=len(selected_stations_1r85_3),
    n_steps=24,
    epochs=50,
    batch_size=32,
    units_1=50,
    units_2=50,
    dropout_rate=0.2,
    learning_rate=0.01
)

gru_model_1r85_3.save("gru_models/gru_model_1r85_3.keras")  # Save the trained model


Epoch 1/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step - loss: 0.0876 - mae: 0.2104 - val_loss: 0.0788 - val_mae: 0.2368
Epoch 2/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0171 - mae: 0.1003 - val_loss: 0.0756 - val_mae: 0.2291
Epoch 3/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0141 - mae: 0.0910 - val_loss: 0.0708 - val_mae: 0.2228
Epoch 4/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0128 - mae: 0.0858 - val_loss: 0.0527 - val_mae: 0.1954
Epoch 5/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0111 - mae: 0.0803 - val_loss: 0.0489 - val_mae: 0.1864
Epoch 6/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0104 - mae: 0.0792 - val_loss: 0.0476 - val_mae: 0.1850
Epoch 7/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0094 - mae: 0.0748 - val_loss: 0.0297 - val_mae: 0.1475
Epoch 8/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0091 - mae: 0.0730 - val_loss: 0.0265 - val_mae: 0.1383
Epoch 9/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0093 - mae:

In [100]:
# Train GRU model
gru_model_1r85_4 = create_gru_model(
    df_scaled=df_selected_1r85_4, 
    no_columns=len(selected_stations_1r85_4),
    n_steps=24,
    epochs=50,
    batch_size=32,
    units_1=50,
    units_2=50,
    dropout_rate=0.2,
    learning_rate=0.01
)

gru_model_1r85_4.save("gru_models/gru_model_1r85_4.keras")  # Save the trained model


Epoch 1/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - loss: 0.1046 - mae: 0.2298 - val_loss: 0.0661 - val_mae: 0.2145
Epoch 2/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0207 - mae: 0.1100 - val_loss: 0.0505 - val_mae: 0.1912
Epoch 3/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0146 - mae: 0.0914 - val_loss: 0.0520 - val_mae: 0.1919
Epoch 4/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0125 - mae: 0.0848 - val_loss: 0.0452 - val_mae: 0.1787
Epoch 5/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0110 - mae: 0.0799 - val_loss: 0.0340 - val_mae: 0.1557
Epoch 6/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0099 - mae: 0.0765 - val_loss: 0.0369 - val_mae: 0.1608
Epoch 7/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0113 - mae: 0.0799 - val_loss: 0.0153 - val_mae: 0.1027
Epoch 8/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0100 - mae: 0.0757 - val_loss: 0.0124 - val_mae: 0.0922
Epoch 9/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0098 - mae:

In [101]:
# Train GRU model
gru_model_1r85_5 = create_gru_model(
    df_scaled=df_selected_1r85_5, 
    no_columns=len(selected_stations_1r85_5),
    n_steps=24,
    epochs=50,
    batch_size=32,
    units_1=50,
    units_2=50,
    dropout_rate=0.2,
    learning_rate=0.01
)

gru_model_1r85_5.save("gru_models/gru_model_1r85_5.keras")  # Save the trained model


Epoch 1/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - loss: 0.0896 - mae: 0.2148 - val_loss: 0.0612 - val_mae: 0.2042
Epoch 2/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0173 - mae: 0.0989 - val_loss: 0.0444 - val_mae: 0.1806
Epoch 3/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0129 - mae: 0.0852 - val_loss: 0.0400 - val_mae: 0.1726
Epoch 4/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0106 - mae: 0.0781 - val_loss: 0.0279 - val_mae: 0.1432
Epoch 5/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0098 - mae: 0.0747 - val_loss: 0.0165 - val_mae: 0.1062
Epoch 6/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0104 - mae: 0.0775 - val_loss: 0.0190 - val_mae: 0.1163
Epoch 7/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0082 - mae: 0.0690 - val_loss: 0.0117 - val_mae: 0.0907
Epoch 8/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0089 - mae: 0.0713 - val_loss: 0.0119 - val_mae: 0.0872
Epoch 9/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0078 - 

In [102]:
# Train GRU model
gru_model_1r85_6 = create_gru_model(
    df_scaled=df_selected_1r85_6, 
    no_columns=len(selected_stations_1r85_6),
    n_steps=24,
    epochs=50,
    batch_size=32,
    units_1=50,
    units_2=50,
    dropout_rate=0.2,
    learning_rate=0.01
)

gru_model_1r85_6.save("gru_models/gru_model_1r85_6.keras")  # Save the trained model


Epoch 1/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - loss: 0.0988 - mae: 0.2300 - val_loss: 0.0556 - val_mae: 0.1998
Epoch 2/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0167 - mae: 0.0991 - val_loss: 0.0489 - val_mae: 0.1836
Epoch 3/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0133 - mae: 0.0882 - val_loss: 0.0351 - val_mae: 0.1576
Epoch 4/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0113 - mae: 0.0800 - val_loss: 0.0380 - val_mae: 0.1619
Epoch 5/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0098 - mae: 0.0741 - val_loss: 0.0207 - val_mae: 0.1196
Epoch 6/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0085 - mae: 0.0689 - val_loss: 0.0159 - val_mae: 0.1047
Epoch 7/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0086 - mae: 0.0701 - val_loss: 0.0132 - val_mae: 0.0952
Epoch 8/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0082 - mae: 0.0680 - val_loss: 0.0116 - val_mae: 0.0872
Epoch 9/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0080 - mae:

In [131]:
# Train GRU model
gru_model_1r85_7 = create_gru_model(
    df_scaled=df_selected_1r85_7, 
    no_columns=len(selected_stations_1r85_7),
    n_steps=24,
    epochs=50,
    batch_size=32,
    units_1=50,
    units_2=50,
    dropout_rate=0.2,
    learning_rate=0.01
)

gru_model_1r85_7.save("gru_models/gru_model_1r85_7.keras")  # Save the trained model


Epoch 1/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - loss: 0.0896 - mae: 0.2108 - val_loss: 0.0635 - val_mae: 0.2115
Epoch 2/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0161 - mae: 0.0985 - val_loss: 0.0529 - val_mae: 0.1940
Epoch 3/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0125 - mae: 0.0848 - val_loss: 0.0336 - val_mae: 0.1540
Epoch 4/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0110 - mae: 0.0794 - val_loss: 0.0308 - val_mae: 0.1491
Epoch 5/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0096 - mae: 0.0744 - val_loss: 0.0255 - val_mae: 0.1330
Epoch 6/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0091 - mae: 0.0721 - val_loss: 0.0171 - val_mae: 0.1093
Epoch 7/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0085 - mae: 0.0703 - val_loss: 0.0117 - val_mae: 0.0889
Epoch 8/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0083 - mae: 0.0693 - val_loss: 0.0113 - val_mae: 0.0834
Epoch 9/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0078 - mae:

In [133]:
# Train GRU model
gru_model_1r85_8 = create_gru_model(
    df_scaled=df_selected_1r85_8, 
    no_columns=len(selected_stations_1r85_8),
    n_steps=24,
    epochs=50,
    batch_size=32,
    units_1=50,
    units_2=50,
    dropout_rate=0.2,
    learning_rate=0.01
)

gru_model_1r85_8.save("gru_models/gru_model_1r85_8.keras")  # Save the trained model


Epoch 1/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - loss: 0.0816 - mae: 0.1959 - val_loss: 0.0486 - val_mae: 0.1814
Epoch 2/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0146 - mae: 0.0903 - val_loss: 0.0349 - val_mae: 0.1555
Epoch 3/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0122 - mae: 0.0824 - val_loss: 0.0338 - val_mae: 0.1493
Epoch 4/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0098 - mae: 0.0737 - val_loss: 0.0338 - val_mae: 0.1496
Epoch 5/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0092 - mae: 0.0711 - val_loss: 0.0233 - val_mae: 0.1244
Epoch 6/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0083 - mae: 0.0671 - val_loss: 0.0137 - val_mae: 0.0948
Epoch 7/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0079 - mae: 0.0662 - val_loss: 0.0120 - val_mae: 0.0874
Epoch 8/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0080 - mae: 0.0664 - val_loss: 0.0134 - val_mae: 0.0862
Epoch 9/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0083 - mae:

In [134]:
# Train GRU model
gru_model_1r85_9 = create_gru_model(
    df_scaled=df_selected_1r85_9, 
    no_columns=len(selected_stations_1r85_9),
    n_steps=24,
    epochs=50,
    batch_size=32,
    units_1=50,
    units_2=50,
    dropout_rate=0.2,
    learning_rate=0.01
)

gru_model_1r85_9.save("gru_models/gru_model_1r85_9.keras")  # Save the trained model


Epoch 1/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - loss: 0.0822 - mae: 0.1987 - val_loss: 0.0427 - val_mae: 0.1754
Epoch 2/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0143 - mae: 0.0883 - val_loss: 0.0362 - val_mae: 0.1603
Epoch 3/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0117 - mae: 0.0793 - val_loss: 0.0327 - val_mae: 0.1518
Epoch 4/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 0.0094 - mae: 0.0714 - val_loss: 0.0236 - val_mae: 0.1275
Epoch 5/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0092 - mae: 0.0714 - val_loss: 0.0240 - val_mae: 0.1280
Epoch 6/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0087 - mae: 0.0690 - val_loss: 0.0188 - val_mae: 0.1113
Epoch 7/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0086 - mae: 0.0684 - val_loss: 0.0175 - val_mae: 0.1034
Epoch 8/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0085 - mae: 0.0689 - val_loss: 0.0091 - val_mae: 0.0703
Epoch 9/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0082 - mae

In [132]:
# Train GRU model
gru_model_1r85_10 = create_gru_model(
    df_scaled=df_selected_1r85_10, 
    no_columns=len(selected_stations_1r85_10),
    n_steps=24,
    epochs=50,
    batch_size=32,
    units_1=50,
    units_2=50,
    dropout_rate=0.2,
    learning_rate=0.01
)

gru_model_1r85_10.save("gru_models/gru_model_1r85_10.keras")  # Save the trained model


Epoch 1/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - loss: 0.0707 - mae: 0.1917 - val_loss: 0.0460 - val_mae: 0.1783
Epoch 2/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0156 - mae: 0.0951 - val_loss: 0.0382 - val_mae: 0.1621
Epoch 3/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0135 - mae: 0.0867 - val_loss: 0.0286 - val_mae: 0.1411
Epoch 4/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0108 - mae: 0.0773 - val_loss: 0.0275 - val_mae: 0.1377
Epoch 5/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0102 - mae: 0.0739 - val_loss: 0.0242 - val_mae: 0.1277
Epoch 6/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0092 - mae: 0.0715 - val_loss: 0.0161 - val_mae: 0.1028
Epoch 7/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0089 - mae: 0.0696 - val_loss: 0.0091 - val_mae: 0.0715
Epoch 8/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0083 - mae: 0.0678 - val_loss: 0.0108 - val_mae: 0.0807
Epoch 9/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0084 - mae:

#### 1 > r > 0.80

In [107]:
# Train GRU model
gru_model_1r80_1 = create_gru_model(
    df_scaled=df_selected_1r80_1, 
    no_columns=len(selected_stations_1r80_1),
    n_steps=24,
    epochs=50,
    batch_size=32,
    units_1=50,
    units_2=50,
    dropout_rate=0.2,
    learning_rate=0.01
)

gru_model_1r80_1.save("gru_models/gru_model_1r80_1.keras")  # Save the trained model


Epoch 1/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - loss: 0.0702 - mae: 0.1959 - val_loss: 0.0964 - val_mae: 0.2613
Epoch 2/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0185 - mae: 0.1052 - val_loss: 0.0754 - val_mae: 0.2337
Epoch 3/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0125 - mae: 0.0861 - val_loss: 0.0674 - val_mae: 0.2228
Epoch 4/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0114 - mae: 0.0830 - val_loss: 0.0604 - val_mae: 0.2112
Epoch 5/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0096 - mae: 0.0759 - val_loss: 0.0581 - val_mae: 0.2067
Epoch 6/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0093 - mae: 0.0753 - val_loss: 0.0423 - val_mae: 0.1771
Epoch 7/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0086 - mae: 0.0708 - val_loss: 0.0291 - val_mae: 0.1450
Epoch 8/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0089 - mae: 0.0726 - val_loss: 0.0269 - val_mae: 0.1401
Epoch 9/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0073 - mae:

In [108]:
# Train GRU model
gru_model_1r80_2 = create_gru_model(
    df_scaled=df_selected_1r80_2, 
    no_columns=len(selected_stations_1r80_2),
    n_steps=24,
    epochs=50,
    batch_size=32,
    units_1=50,
    units_2=50,
    dropout_rate=0.2,
    learning_rate=0.01
)

gru_model_1r80_2.save("gru_models/gru_model_1r80_2.keras")  # Save the trained model


Epoch 1/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - loss: 0.0979 - mae: 0.2118 - val_loss: 0.0791 - val_mae: 0.2327
Epoch 2/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0164 - mae: 0.0977 - val_loss: 0.0599 - val_mae: 0.2050
Epoch 3/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0131 - mae: 0.0848 - val_loss: 0.0533 - val_mae: 0.1942
Epoch 4/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0112 - mae: 0.0804 - val_loss: 0.0487 - val_mae: 0.1869
Epoch 5/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0101 - mae: 0.0738 - val_loss: 0.0402 - val_mae: 0.1712
Epoch 6/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0090 - mae: 0.0712 - val_loss: 0.0297 - val_mae: 0.1484
Epoch 7/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0084 - mae: 0.0688 - val_loss: 0.0305 - val_mae: 0.1497
Epoch 8/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0081 - mae: 0.0673 - val_loss: 0.0218 - val_mae: 0.1275
Epoch 9/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0076 - mae:

In [109]:
# Train GRU model
gru_model_1r80_3 = create_gru_model(
    df_scaled=df_selected_1r80_3, 
    no_columns=len(selected_stations_1r80_3),
    n_steps=24,
    epochs=50,
    batch_size=32,
    units_1=50,
    units_2=50,
    dropout_rate=0.2,
    learning_rate=0.01
)

gru_model_1r80_3.save("gru_models/gru_model_1r80_3.keras")  # Save the trained model


Epoch 1/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - loss: 0.0707 - mae: 0.1910 - val_loss: 0.0794 - val_mae: 0.2328
Epoch 2/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0158 - mae: 0.0932 - val_loss: 0.0571 - val_mae: 0.2003
Epoch 3/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0109 - mae: 0.0778 - val_loss: 0.0476 - val_mae: 0.1849
Epoch 4/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0104 - mae: 0.0749 - val_loss: 0.0430 - val_mae: 0.1769
Epoch 5/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0094 - mae: 0.0737 - val_loss: 0.0388 - val_mae: 0.1691
Epoch 6/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0082 - mae: 0.0679 - val_loss: 0.0310 - val_mae: 0.1517
Epoch 7/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0082 - mae: 0.0672 - val_loss: 0.0208 - val_mae: 0.1211
Epoch 8/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0084 - mae: 0.0684 - val_loss: 0.0179 - val_mae: 0.1115
Epoch 9/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0075 - mae:

In [110]:
# Train GRU model
gru_model_1r80_4 = create_gru_model(
    df_scaled=df_selected_1r80_4, 
    no_columns=len(selected_stations_1r80_4),
    n_steps=24,
    epochs=50,
    batch_size=32,
    units_1=50,
    units_2=50,
    dropout_rate=0.2,
    learning_rate=0.01
)

gru_model_1r80_4.save("gru_models/gru_model_1r80_4.keras")  # Save the trained model


Epoch 1/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - loss: 0.0758 - mae: 0.1973 - val_loss: 0.0577 - val_mae: 0.2048
Epoch 2/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0158 - mae: 0.0960 - val_loss: 0.0472 - val_mae: 0.1862
Epoch 3/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0112 - mae: 0.0805 - val_loss: 0.0387 - val_mae: 0.1695
Epoch 4/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0110 - mae: 0.0787 - val_loss: 0.0357 - val_mae: 0.1640
Epoch 5/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0091 - mae: 0.0716 - val_loss: 0.0330 - val_mae: 0.1571
Epoch 6/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0098 - mae: 0.0737 - val_loss: 0.0251 - val_mae: 0.1357
Epoch 7/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0084 - mae: 0.0696 - val_loss: 0.0204 - val_mae: 0.1218
Epoch 8/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0087 - mae: 0.0708 - val_loss: 0.0140 - val_mae: 0.0987
Epoch 9/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0084 - mae:

In [111]:
# Train GRU model
gru_model_1r80_5 = create_gru_model(
    df_scaled=df_selected_1r80_5, 
    no_columns=len(selected_stations_1r80_5),
    n_steps=24,
    epochs=50,
    batch_size=32,
    units_1=50,
    units_2=50,
    dropout_rate=0.2,
    learning_rate=0.01
)

gru_model_1r80_5.save("gru_models/gru_model_1r80_5.keras")  # Save the trained model


Epoch 1/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - loss: 0.1242 - mae: 0.2360 - val_loss: 0.0465 - val_mae: 0.1864
Epoch 2/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0192 - mae: 0.1068 - val_loss: 0.0426 - val_mae: 0.1787
Epoch 3/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0136 - mae: 0.0864 - val_loss: 0.0347 - val_mae: 0.1602
Epoch 4/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0115 - mae: 0.0802 - val_loss: 0.0366 - val_mae: 0.1642
Epoch 5/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0104 - mae: 0.0761 - val_loss: 0.0287 - val_mae: 0.1450
Epoch 6/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0094 - mae: 0.0721 - val_loss: 0.0267 - val_mae: 0.1408
Epoch 7/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0094 - mae: 0.0734 - val_loss: 0.0176 - val_mae: 0.1123
Epoch 8/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0090 - mae: 0.0708 - val_loss: 0.0148 - val_mae: 0.1021
Epoch 9/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0095 - mae:

In [112]:
# Train GRU model
gru_model_1r80_6 = create_gru_model(
    df_scaled=df_selected_1r80_6, 
    no_columns=len(selected_stations_1r80_6),
    n_steps=24,
    epochs=50,
    batch_size=32,
    units_1=50,
    units_2=50,
    dropout_rate=0.2,
    learning_rate=0.01
)

gru_model_1r80_6.save("gru_models/gru_model_1r80_6.keras")  # Save the trained model


Epoch 1/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - loss: 0.0721 - mae: 0.1953 - val_loss: 0.0479 - val_mae: 0.1870
Epoch 2/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0160 - mae: 0.0947 - val_loss: 0.0458 - val_mae: 0.1813
Epoch 3/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0120 - mae: 0.0819 - val_loss: 0.0338 - val_mae: 0.1583
Epoch 4/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0109 - mae: 0.0774 - val_loss: 0.0265 - val_mae: 0.1399
Epoch 5/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0095 - mae: 0.0734 - val_loss: 0.0158 - val_mae: 0.1065
Epoch 6/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0087 - mae: 0.0696 - val_loss: 0.0199 - val_mae: 0.1178
Epoch 7/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0087 - mae: 0.0697 - val_loss: 0.0135 - val_mae: 0.0960
Epoch 8/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0081 - mae: 0.0683 - val_loss: 0.0145 - val_mae: 0.0951
Epoch 9/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0079 - mae:

In [146]:
# Train GRU model
gru_model_1r80_7 = create_gru_model(
    df_scaled=df_selected_1r80_7, 
    no_columns=len(selected_stations_1r80_7),
    n_steps=24,
    epochs=50,
    batch_size=32,
    units_1=50,
    units_2=50,
    dropout_rate=0.2,
    learning_rate=0.01
)

gru_model_1r80_7.save("gru_models/gru_model_1r80_7.keras")  # Save the trained model


Epoch 1/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 5s 15ms/step - loss: 0.0797 - mae: 0.2001 - val_loss: 0.0684 - val_mae: 0.2142
Epoch 2/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0153 - mae: 0.0928 - val_loss: 0.0611 - val_mae: 0.2037
Epoch 3/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0116 - mae: 0.0817 - val_loss: 0.0499 - val_mae: 0.1866
Epoch 4/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0115 - mae: 0.0806 - val_loss: 0.0445 - val_mae: 0.1786
Epoch 5/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0102 - mae: 0.0757 - val_loss: 0.0301 - val_mae: 0.1468
Epoch 6/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0096 - mae: 0.0737 - val_loss: 0.0205 - val_mae: 0.1218
Epoch 7/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0092 - mae: 0.0721 - val_loss: 0.0220 - val_mae: 0.1211
Epoch 8/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0080 - mae: 0.0675 - val_loss: 0.0143 - val_mae: 0.0976
Epoch 9/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0082 - mae

In [114]:
# Train GRU model
gru_model_1r80_8 = create_gru_model(
    df_scaled=df_selected_1r80_8, 
    no_columns=len(selected_stations_1r80_8),
    n_steps=24,
    epochs=50,
    batch_size=32,
    units_1=50,
    units_2=50,
    dropout_rate=0.2,
    learning_rate=0.01
)

gru_model_1r80_8.save("gru_models/gru_model_1r80_8.keras")  # Save the trained model


Epoch 1/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - loss: 0.1250 - mae: 0.2371 - val_loss: 0.0463 - val_mae: 0.1859
Epoch 2/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0163 - mae: 0.0984 - val_loss: 0.0407 - val_mae: 0.1724
Epoch 3/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0142 - mae: 0.0907 - val_loss: 0.0465 - val_mae: 0.1839
Epoch 4/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0115 - mae: 0.0801 - val_loss: 0.0303 - val_mae: 0.1502
Epoch 5/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 0.0107 - mae: 0.0777 - val_loss: 0.0312 - val_mae: 0.1495
Epoch 6/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0096 - mae: 0.0735 - val_loss: 0.0267 - val_mae: 0.1359
Epoch 7/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0098 - mae: 0.0748 - val_loss: 0.0194 - val_mae: 0.1144
Epoch 8/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0101 - mae: 0.0752 - val_loss: 0.0161 - val_mae: 0.1033
Epoch 9/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0091 - m

In [115]:
# Train GRU model
gru_model_1r80_9 = create_gru_model(
    df_scaled=df_selected_1r80_9, 
    no_columns=len(selected_stations_1r80_9),
    n_steps=24,
    epochs=50,
    batch_size=32,
    units_1=50,
    units_2=50,
    dropout_rate=0.2,
    learning_rate=0.01
)

gru_model_1r80_9.save("gru_models/gru_model_1r80_9.keras")  # Save the trained model


Epoch 1/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - loss: 0.0909 - mae: 0.2125 - val_loss: 0.0665 - val_mae: 0.2122
Epoch 2/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0153 - mae: 0.0927 - val_loss: 0.0398 - val_mae: 0.1716
Epoch 3/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 0.0119 - mae: 0.0813 - val_loss: 0.0392 - val_mae: 0.1682
Epoch 4/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0103 - mae: 0.0750 - val_loss: 0.0259 - val_mae: 0.1355
Epoch 5/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0095 - mae: 0.0736 - val_loss: 0.0240 - val_mae: 0.1313
Epoch 6/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0083 - mae: 0.0686 - val_loss: 0.0164 - val_mae: 0.1065
Epoch 7/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0080 - mae: 0.0669 - val_loss: 0.0122 - val_mae: 0.0886
Epoch 8/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0081 - mae: 0.0673 - val_loss: 0.0097 - val_mae: 0.0777
Epoch 9/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0077 - mae

In [141]:
# Train GRU model
gru_model_1r80_10 = create_gru_model(
    df_scaled=df_selected_1r80_10, 
    no_columns=len(selected_stations_1r80_10),
    n_steps=24,
    epochs=50,
    batch_size=32,
    units_1=50,
    units_2=50,
    dropout_rate=0.2,
    learning_rate=0.01
)

gru_model_1r80_10.save("gru_models/gru_model_1r80_10.keras")  # Save the trained model


Epoch 1/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - loss: 0.0876 - mae: 0.2049 - val_loss: 0.0412 - val_mae: 0.1709
Epoch 2/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0154 - mae: 0.0927 - val_loss: 0.0390 - val_mae: 0.1665
Epoch 3/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0109 - mae: 0.0774 - val_loss: 0.0312 - val_mae: 0.1478
Epoch 4/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0094 - mae: 0.0717 - val_loss: 0.0223 - val_mae: 0.1238
Epoch 5/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0090 - mae: 0.0713 - val_loss: 0.0217 - val_mae: 0.1219
Epoch 6/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0088 - mae: 0.0697 - val_loss: 0.0163 - val_mae: 0.1047
Epoch 7/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 0.0082 - mae: 0.0672 - val_loss: 0.0149 - val_mae: 0.0980
Epoch 8/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0078 - mae: 0.0659 - val_loss: 0.0106 - val_mae: 0.0823
Epoch 9/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 0.0076 - 

#### 1 > r > 0.75

In [117]:
# Train GRU model
gru_model_1r75_1 = create_gru_model(
    df_scaled=df_selected_1r75_1, 
    no_columns=len(selected_stations_1r75_1),
    n_steps=24,
    epochs=50,
    batch_size=32,
    units_1=50,
    units_2=50,
    dropout_rate=0.2,
    learning_rate=0.01
)

gru_model_1r75_1.save("gru_models/gru_model_1r75_1.keras")  # Save the trained model


Epoch 1/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - loss: 0.0933 - mae: 0.2278 - val_loss: 0.1540 - val_mae: 0.3253
Epoch 2/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0205 - mae: 0.1117 - val_loss: 0.1156 - val_mae: 0.2851
Epoch 3/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 0.0167 - mae: 0.0991 - val_loss: 0.1048 - val_mae: 0.2732
Epoch 4/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 0.0144 - mae: 0.0932 - val_loss: 0.0933 - val_mae: 0.2575
Epoch 5/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0116 - mae: 0.0837 - val_loss: 0.0770 - val_mae: 0.2363
Epoch 6/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0106 - mae: 0.0794 - val_loss: 0.0590 - val_mae: 0.2079
Epoch 7/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 0.0103 - mae: 0.0780 - val_loss: 0.0536 - val_mae: 0.1993
Epoch 8/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0102 - mae: 0.0767 - val_loss: 0.0510 - val_mae: 0.1953
Epoch 9/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0087 - m

In [118]:
# Train GRU model
gru_model_1r75_2 = create_gru_model(
    df_scaled=df_selected_1r75_2, 
    no_columns=len(selected_stations_1r75_2),
    n_steps=24,
    epochs=50,
    batch_size=32,
    units_1=50,
    units_2=50,
    dropout_rate=0.2,
    learning_rate=0.01
)

gru_model_1r75_2.save("gru_models/gru_model_1r75_2.keras")  # Save the trained model


Epoch 1/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - loss: 0.1116 - mae: 0.2403 - val_loss: 0.1035 - val_mae: 0.2699
Epoch 2/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0213 - mae: 0.1142 - val_loss: 0.0797 - val_mae: 0.2408
Epoch 3/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0157 - mae: 0.0973 - val_loss: 0.0833 - val_mae: 0.2460
Epoch 4/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0135 - mae: 0.0917 - val_loss: 0.0677 - val_mae: 0.2220
Epoch 5/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0133 - mae: 0.0887 - val_loss: 0.0551 - val_mae: 0.2032
Epoch 6/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0110 - mae: 0.0808 - val_loss: 0.0563 - val_mae: 0.2037
Epoch 7/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0092 - mae: 0.0744 - val_loss: 0.0435 - val_mae: 0.1770
Epoch 8/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0086 - mae: 0.0724 - val_loss: 0.0430 - val_mae: 0.1766
Epoch 9/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0088 - mae:

In [119]:
# Train GRU model
gru_model_1r75_3 = create_gru_model(
    df_scaled=df_selected_1r75_3, 
    no_columns=len(selected_stations_1r75_3),
    n_steps=24,
    epochs=50,
    batch_size=32,
    units_1=50,
    units_2=50,
    dropout_rate=0.2,
    learning_rate=0.01
)

gru_model_1r75_3.save("gru_models/gru_model_1r75_3.keras")  # Save the trained model


Epoch 1/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - loss: 0.1044 - mae: 0.2292 - val_loss: 0.0744 - val_mae: 0.2312
Epoch 2/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0187 - mae: 0.1058 - val_loss: 0.0571 - val_mae: 0.2059
Epoch 3/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0131 - mae: 0.0879 - val_loss: 0.0492 - val_mae: 0.1925
Epoch 4/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0121 - mae: 0.0835 - val_loss: 0.0588 - val_mae: 0.2051
Epoch 5/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0105 - mae: 0.0779 - val_loss: 0.0489 - val_mae: 0.1905
Epoch 6/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 0.0090 - mae: 0.0729 - val_loss: 0.0377 - val_mae: 0.1663
Epoch 7/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0091 - mae: 0.0727 - val_loss: 0.0308 - val_mae: 0.1494
Epoch 8/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0085 - mae: 0.0712 - val_loss: 0.0240 - val_mae: 0.1299
Epoch 9/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0090 - ma

In [140]:
# Train GRU model
gru_model_1r75_4 = create_gru_model(
    df_scaled=df_selected_1r75_4, 
    no_columns=len(selected_stations_1r75_4),
    n_steps=24,
    epochs=50,
    batch_size=32,
    units_1=50,
    units_2=50,
    dropout_rate=0.2,
    learning_rate=0.01
)

gru_model_1r75_4.save("gru_models/gru_model_1r75_4.keras")  # Save the trained model


Epoch 1/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - loss: 0.0775 - mae: 0.2008 - val_loss: 0.0728 - val_mae: 0.2270
Epoch 2/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0162 - mae: 0.0961 - val_loss: 0.0465 - val_mae: 0.1871
Epoch 3/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0130 - mae: 0.0878 - val_loss: 0.0483 - val_mae: 0.1881
Epoch 4/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0104 - mae: 0.0781 - val_loss: 0.0410 - val_mae: 0.1750
Epoch 5/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0100 - mae: 0.0756 - val_loss: 0.0293 - val_mae: 0.1478
Epoch 6/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0092 - mae: 0.0725 - val_loss: 0.0291 - val_mae: 0.1469
Epoch 7/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0082 - mae: 0.0683 - val_loss: 0.0232 - val_mae: 0.1293
Epoch 8/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0081 - mae: 0.0689 - val_loss: 0.0132 - val_mae: 0.0974
Epoch 9/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0084 - mae

In [121]:
# Train GRU model
gru_model_1r75_5 = create_gru_model(
    df_scaled=df_selected_1r75_5, 
    no_columns=len(selected_stations_1r75_5),
    n_steps=24,
    epochs=50,
    batch_size=32,
    units_1=50,
    units_2=50,
    dropout_rate=0.2,
    learning_rate=0.01
)

gru_model_1r75_5.save("gru_models/gru_model_1r75_5.keras")  # Save the trained model


Epoch 1/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - loss: 0.1032 - mae: 0.2279 - val_loss: 0.0616 - val_mae: 0.2093
Epoch 2/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0174 - mae: 0.1005 - val_loss: 0.0434 - val_mae: 0.1782
Epoch 3/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0147 - mae: 0.0919 - val_loss: 0.0354 - val_mae: 0.1629
Epoch 4/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0133 - mae: 0.0860 - val_loss: 0.0287 - val_mae: 0.1465
Epoch 5/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0110 - mae: 0.0791 - val_loss: 0.0268 - val_mae: 0.1389
Epoch 6/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0098 - mae: 0.0747 - val_loss: 0.0183 - val_mae: 0.1145
Epoch 7/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0099 - mae: 0.0751 - val_loss: 0.0115 - val_mae: 0.0873
Epoch 8/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0098 - mae: 0.0758 - val_loss: 0.0126 - val_mae: 0.0916
Epoch 9/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0092 - mae:

In [122]:
# Train GRU model
gru_model_1r75_6 = create_gru_model(
    df_scaled=df_selected_1r75_6, 
    no_columns=len(selected_stations_1r75_6),
    n_steps=24,
    epochs=50,
    batch_size=32,
    units_1=50,
    units_2=50,
    dropout_rate=0.2,
    learning_rate=0.01
)

gru_model_1r75_6.save("gru_models/gru_model_1r75_6.keras")  # Save the trained model


Epoch 1/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - loss: 0.1237 - mae: 0.2397 - val_loss: 0.0732 - val_mae: 0.2272
Epoch 2/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0170 - mae: 0.0988 - val_loss: 0.0566 - val_mae: 0.2044
Epoch 3/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0146 - mae: 0.0919 - val_loss: 0.0418 - val_mae: 0.1769
Epoch 4/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0118 - mae: 0.0816 - val_loss: 0.0373 - val_mae: 0.1660
Epoch 5/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0105 - mae: 0.0774 - val_loss: 0.0347 - val_mae: 0.1603
Epoch 6/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0103 - mae: 0.0775 - val_loss: 0.0243 - val_mae: 0.1330
Epoch 7/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0095 - mae: 0.0728 - val_loss: 0.0163 - val_mae: 0.1077
Epoch 8/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0099 - mae: 0.0751 - val_loss: 0.0189 - val_mae: 0.1131
Epoch 9/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0093 - mae:

In [123]:
# Train GRU model
gru_model_1r75_7 = create_gru_model(
    df_scaled=df_selected_1r75_7, 
    no_columns=len(selected_stations_1r75_7),
    n_steps=24,
    epochs=50,
    batch_size=32,
    units_1=50,
    units_2=50,
    dropout_rate=0.2,
    learning_rate=0.01
)

gru_model_1r75_7.save("gru_models/gru_model_1r75_7.keras")  # Save the trained model


Epoch 1/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - loss: 0.0773 - mae: 0.2039 - val_loss: 0.0676 - val_mae: 0.2167
Epoch 2/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0171 - mae: 0.0988 - val_loss: 0.0514 - val_mae: 0.1938
Epoch 3/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0119 - mae: 0.0826 - val_loss: 0.0442 - val_mae: 0.1796
Epoch 4/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0113 - mae: 0.0802 - val_loss: 0.0329 - val_mae: 0.1556
Epoch 5/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0098 - mae: 0.0746 - val_loss: 0.0243 - val_mae: 0.1337
Epoch 6/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0100 - mae: 0.0756 - val_loss: 0.0279 - val_mae: 0.1419
Epoch 7/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0095 - mae: 0.0736 - val_loss: 0.0132 - val_mae: 0.0957
Epoch 8/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0091 - mae: 0.0717 - val_loss: 0.0118 - val_mae: 0.0871
Epoch 9/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0089 - mae:

In [124]:
# Train GRU model
gru_model_1r75_8 = create_gru_model(
    df_scaled=df_selected_1r75_8, 
    no_columns=len(selected_stations_1r75_8),
    n_steps=24,
    epochs=50,
    batch_size=32,
    units_1=50,
    units_2=50,
    dropout_rate=0.2,
    learning_rate=0.01
)

gru_model_1r75_8.save("gru_models/gru_model_1r75_8.keras")  # Save the trained model


Epoch 1/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - loss: 0.0982 - mae: 0.2204 - val_loss: 0.0445 - val_mae: 0.1806
Epoch 2/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0170 - mae: 0.0983 - val_loss: 0.0389 - val_mae: 0.1701
Epoch 3/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0139 - mae: 0.0890 - val_loss: 0.0367 - val_mae: 0.1659
Epoch 4/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0112 - mae: 0.0802 - val_loss: 0.0284 - val_mae: 0.1440
Epoch 5/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0099 - mae: 0.0761 - val_loss: 0.0316 - val_mae: 0.1520
Epoch 6/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0100 - mae: 0.0751 - val_loss: 0.0199 - val_mae: 0.1195
Epoch 7/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0097 - mae: 0.0748 - val_loss: 0.0141 - val_mae: 0.0968
Epoch 8/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0089 - mae: 0.0716 - val_loss: 0.0159 - val_mae: 0.1033
Epoch 9/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0091 - mae:

In [125]:
# Train GRU model
gru_model_1r75_9 = create_gru_model(
    df_scaled=df_selected_1r75_9, 
    no_columns=len(selected_stations_1r75_9),
    n_steps=24,
    epochs=50,
    batch_size=32,
    units_1=50,
    units_2=50,
    dropout_rate=0.2,
    learning_rate=0.01
)

gru_model_1r75_9.save("gru_models/gru_model_1r75_9.keras")  # Save the trained model


Epoch 1/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - loss: 0.1065 - mae: 0.2253 - val_loss: 0.0438 - val_mae: 0.1810
Epoch 2/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0182 - mae: 0.1033 - val_loss: 0.0375 - val_mae: 0.1683
Epoch 3/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0141 - mae: 0.0897 - val_loss: 0.0328 - val_mae: 0.1579
Epoch 4/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0115 - mae: 0.0812 - val_loss: 0.0358 - val_mae: 0.1635
Epoch 5/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0110 - mae: 0.0797 - val_loss: 0.0198 - val_mae: 0.1198
Epoch 6/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0095 - mae: 0.0744 - val_loss: 0.0227 - val_mae: 0.1246
Epoch 7/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0102 - mae: 0.0760 - val_loss: 0.0150 - val_mae: 0.1017
Epoch 8/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0094 - mae: 0.0732 - val_loss: 0.0147 - val_mae: 0.0973
Epoch 9/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0090 - mae:

In [126]:
# Train GRU model
gru_model_1r75_10 = create_gru_model(
    df_scaled=df_selected_1r75_10, 
    no_columns=len(selected_stations_1r75_10),
    n_steps=24,
    epochs=50,
    batch_size=32,
    units_1=50,
    units_2=50,
    dropout_rate=0.2,
    learning_rate=0.01
)

gru_model_1r75_10.save("gru_models/gru_model_1r75_10.keras")  # Save the trained model


Epoch 1/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - loss: 0.1251 - mae: 0.2410 - val_loss: 0.0528 - val_mae: 0.1943
Epoch 2/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0160 - mae: 0.0957 - val_loss: 0.0394 - val_mae: 0.1667
Epoch 3/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0133 - mae: 0.0853 - val_loss: 0.0299 - val_mae: 0.1443
Epoch 4/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0117 - mae: 0.0799 - val_loss: 0.0218 - val_mae: 0.1220
Epoch 5/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0107 - mae: 0.0766 - val_loss: 0.0212 - val_mae: 0.1213
Epoch 6/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0101 - mae: 0.0749 - val_loss: 0.0159 - val_mae: 0.1022
Epoch 7/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0089 - mae: 0.0710 - val_loss: 0.0148 - val_mae: 0.0965
Epoch 8/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0081 - mae: 0.0669 - val_loss: 0.0099 - val_mae: 0.0767
Epoch 9/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0088 - mae:

### Transformer

#### 1 > r > 0.95

In [14]:
transformer_model_1r95_1 = create_transformer_model(
    df_scaled=df_selected_1r95_1,
    no_columns=len(selected_stations_1r95_1),
    n_steps=24,
    epochs=100,          # More epochs, since the model is smaller
    batch_size=16,       # Smaller batch for better generalization
    head_size=32,        # Reduce head size to avoid overfitting
    num_heads=2,         # Keep 2 heads, enough for diversity but not too heavy
    ff_dim=64,           # Reduce feed-forward dimension
    num_blocks=1,        # Fewer transformer blocks for smaller data
    dropout=0.2,         # Slightly higher dropout to regularize
    learning_rate=0.0005 # Lower LR for stable training
)


Epoch 1/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.1832 - mae: 0.3516 - val_loss: 0.1127 - val_mae: 0.2661
Epoch 2/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.1148 - mae: 0.2807 - val_loss: 0.0762 - val_mae: 0.2304
Epoch 3/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0821 - mae: 0.2518 - val_loss: 0.0632 - val_mae: 0.2197
Epoch 4/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0642 - mae: 0.2306 - val_loss: 0.0608 - val_mae: 0.2185
Epoch 5/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0620 - mae: 0.2283 - val_loss: 0.0610 - val_mae: 0.2193
Epoch 6/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0610 - mae: 0.2261 - val_loss: 0.0613 - val_mae: 0.2197
Epoch 7/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0626 - mae: 0.2292 - val_loss: 0.0615 - val_mae: 0.2200
Epoch 8/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0615 - mae: 0.2266 - val_loss: 0.0616 - val_mae: 0.2202
Epoch 9/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/

In [19]:
# Train Transformer model
transformer_model_1r95_2 = create_transformer_model(
    df_scaled=df_selected_1r95_2,
    no_columns=len(selected_stations_1r95_2),
    n_steps=24,
    epochs=100,          # More epochs, since the model is smaller
    batch_size=16,       # Smaller batch for better generalization
    head_size=32,        # Reduce head size to avoid overfitting
    num_heads=2,         # Keep 2 heads, enough for diversity but not too heavy
    ff_dim=64,           # Reduce feed-forward dimension
    num_blocks=1,        # Fewer transformer blocks for smaller data
    dropout=0.2,         # Slightly higher dropout to regularize
    learning_rate=0.0005 # Lower LR for stable training
)

# Save Transformer model
transformer_model_1r95_2.save("transformer_models/transformer_model_1r95_2.keras")


Epoch 1/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.8521 - mae: 0.8793 - val_loss: 0.5706 - val_mae: 0.7049
Epoch 2/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5196 - mae: 0.6648 - val_loss: 0.3560 - val_mae: 0.5330
Epoch 3/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.3286 - mae: 0.5027 - val_loss: 0.2342 - val_mae: 0.4055
Epoch 4/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2296 - mae: 0.3901 - val_loss: 0.1646 - val_mae: 0.3285
Epoch 5/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1635 - mae: 0.3192 - val_loss: 0.1246 - val_mae: 0.2878
Epoch 6/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1271 - mae: 0.2869 - val_loss: 0.1015 - val_mae: 0.2651
Epoch 7/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1055 - mae: 0.2699 - val_loss: 0.0881 - val_mae: 0.2515
Epoch 8/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0949 - mae: 0.2635 - val_loss: 0.0804 - val_mae: 0.2439
Epoch 9/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/

In [20]:
# Train Transformer model
transformer_model_1r95_3 = create_transformer_model(
    df_scaled=df_selected_1r95_3,
    no_columns=len(selected_stations_1r95_3),
    n_steps=24,
    epochs=100,          # More epochs, since the model is smaller
    batch_size=16,       # Smaller batch for better generalization
    head_size=32,        # Reduce head size to avoid overfitting
    num_heads=2,         # Keep 2 heads, enough for diversity but not too heavy
    ff_dim=64,           # Reduce feed-forward dimension
    num_blocks=1,        # Fewer transformer blocks for smaller data
    dropout=0.2,         # Slightly higher dropout to regularize
    learning_rate=0.0005 # Lower LR for stable training
)

# Save Transformer model
transformer_model_1r95_3.save("transformer_models/transformer_model_1r95_3.keras")


Epoch 1/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.5579 - mae: 0.5458 - val_loss: 0.1317 - val_mae: 0.2905
Epoch 2/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1298 - mae: 0.2998 - val_loss: 0.0881 - val_mae: 0.2540
Epoch 3/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0914 - mae: 0.2655 - val_loss: 0.0790 - val_mae: 0.2460
Epoch 4/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0824 - mae: 0.2552 - val_loss: 0.0772 - val_mae: 0.2438
Epoch 5/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0809 - mae: 0.2526 - val_loss: 0.0766 - val_mae: 0.2428
Epoch 6/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0793 - mae: 0.2523 - val_loss: 0.0762 - val_mae: 0.2420
Epoch 7/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0807 - mae: 0.2536 - val_loss: 0.0759 - val_mae: 0.2415
Epoch 8/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0772 - mae: 0.2463 - val_loss: 0.0753 - val_mae: 0.2405
Epoch 9/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/

In [30]:
# Train Transformer model
transformer_model_1r95_4 = create_transformer_model(
    df_scaled=df_selected_1r95_4,
    no_columns=len(selected_stations_1r95_4),
    n_steps=24,
    epochs=100,          # More epochs, since the model is smaller
    batch_size=16,       # Smaller batch for better generalization
    head_size=32,        # Reduce head size to avoid overfitting
    num_heads=2,         # Keep 2 heads, enough for diversity but not too heavy
    ff_dim=64,           # Reduce feed-forward dimension
    num_blocks=1,        # Fewer transformer blocks for smaller data
    dropout=0.2,         # Slightly higher dropout to regularize
    learning_rate=0.01 # Lower LR for stable training
)

# Save Transformer model
transformer_model_1r95_4.save("transformer_models/transformer_model_1r95_4.keras")


Epoch 1/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.2214 - mae: 0.3418 - val_loss: 0.0678 - val_mae: 0.2275
Epoch 2/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0587 - mae: 0.2059 - val_loss: 0.0183 - val_mae: 0.0980
Epoch 3/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0172 - mae: 0.0954 - val_loss: 0.0180 - val_mae: 0.0936
Epoch 4/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0162 - mae: 0.0928 - val_loss: 0.0174 - val_mae: 0.0910
Epoch 5/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0153 - mae: 0.0892 - val_loss: 0.0195 - val_mae: 0.1130
Epoch 6/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0158 - mae: 0.0933 - val_loss: 0.0146 - val_mae: 0.0850
Epoch 7/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0151 - mae: 0.0898 - val_loss: 0.0141 - val_mae: 0.0857
Epoch 8/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0140 - mae: 0.0851 - val_loss: 0.0143 - val_mae: 0.0854
Epoch 9/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/

In [21]:
# Train Transformer model
transformer_model_1r95_5 = create_transformer_model(
    df_scaled=df_selected_1r95_5,
    no_columns=len(selected_stations_1r95_5),
    n_steps=24,
    epochs=100,          # More epochs, since the model is smaller
    batch_size=16,       # Smaller batch for better generalization
    head_size=32,        # Reduce head size to avoid overfitting
    num_heads=2,         # Keep 2 heads, enough for diversity but not too heavy
    ff_dim=64,           # Reduce feed-forward dimension
    num_blocks=1,        # Fewer transformer blocks for smaller data
    dropout=0.2,         # Slightly higher dropout to regularize
    learning_rate=0.0005 # Lower LR for stable training
)

# Save Transformer model
transformer_model_1r95_5.save("transformer_models/transformer_model_1r95_5.keras")


Epoch 1/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.4261 - mae: 0.4942 - val_loss: 0.1180 - val_mae: 0.2809
Epoch 2/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0971 - mae: 0.2560 - val_loss: 0.0815 - val_mae: 0.2427
Epoch 3/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0737 - mae: 0.2342 - val_loss: 0.0677 - val_mae: 0.2213
Epoch 4/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0664 - mae: 0.2234 - val_loss: 0.0592 - val_mae: 0.2025
Epoch 5/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0588 - mae: 0.2060 - val_loss: 0.0504 - val_mae: 0.1798
Epoch 6/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0470 - mae: 0.1728 - val_loss: 0.0402 - val_mae: 0.1525
Epoch 7/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0371 - mae: 0.1507 - val_loss: 0.0335 - val_mae: 0.1389
Epoch 8/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0313 - mae: 0.1380 - val_loss: 0.0274 - val_mae: 0.1246
Epoch 9/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/

In [23]:
# Train Transformer model
transformer_model_1r95_6 = create_transformer_model(
    df_scaled=df_selected_1r95_6,
    no_columns=len(selected_stations_1r95_6),
    n_steps=24,
    epochs=100,          # More epochs, since the model is smaller
    batch_size=16,       # Smaller batch for better generalization
    head_size=32,        # Reduce head size to avoid overfitting
    num_heads=2,         # Keep 2 heads, enough for diversity but not too heavy
    ff_dim=64,           # Reduce feed-forward dimension
    num_blocks=1,        # Fewer transformer blocks for smaller data
    dropout=0.2,         # Slightly higher dropout to regularize
    learning_rate=0.0005 # Lower LR for stable training
)

# Save Transformer model
transformer_model_1r95_6.save("transformer_models/transformer_model_1r95_6.keras")


Epoch 1/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.5766 - mae: 0.5716 - val_loss: 0.1337 - val_mae: 0.3055
Epoch 2/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1162 - mae: 0.2891 - val_loss: 0.0849 - val_mae: 0.2505
Epoch 3/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0834 - mae: 0.2541 - val_loss: 0.0756 - val_mae: 0.2390
Epoch 4/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0768 - mae: 0.2463 - val_loss: 0.0666 - val_mae: 0.2264
Epoch 5/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0614 - mae: 0.2180 - val_loss: 0.0404 - val_mae: 0.1671
Epoch 6/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0376 - mae: 0.1587 - val_loss: 0.0314 - val_mae: 0.1392
Epoch 7/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0313 - mae: 0.1376 - val_loss: 0.0284 - val_mae: 0.1317
Epoch 8/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0265 - mae: 0.1231 - val_loss: 0.0246 - val_mae: 0.1192
Epoch 9/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/

In [28]:
# Train Transformer model
transformer_model_1r95_7 = create_transformer_model(
    df_scaled=df_selected_1r95_7,
    no_columns=len(selected_stations_1r95_7),
    n_steps=24,
    epochs=100,          # More epochs, since the model is smaller
    batch_size=16,       # Smaller batch for better generalization
    head_size=32,        # Reduce head size to avoid overfitting
    num_heads=2,         # Keep 2 heads, enough for diversity but not too heavy
    ff_dim=64,           # Reduce feed-forward dimension
    num_blocks=1,        # Fewer transformer blocks for smaller data
    dropout=0.2,         # Slightly higher dropout to regularize
    learning_rate=0.01 # Lower LR for stable training
)

# Save Transformer model
transformer_model_1r95_7.save("transformer_models/transformer_model_1r95_7.keras")


Epoch 1/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.2705 - mae: 0.3729 - val_loss: 0.0770 - val_mae: 0.2406
Epoch 2/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0686 - mae: 0.2239 - val_loss: 0.0146 - val_mae: 0.0912
Epoch 3/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0143 - mae: 0.0884 - val_loss: 0.0121 - val_mae: 0.0802
Epoch 4/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0129 - mae: 0.0838 - val_loss: 0.0122 - val_mae: 0.0808
Epoch 5/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0148 - mae: 0.0897 - val_loss: 0.0113 - val_mae: 0.0763
Epoch 6/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0111 - mae: 0.0768 - val_loss: 0.0119 - val_mae: 0.0828
Epoch 7/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0110 - mae: 0.0762 - val_loss: 0.0117 - val_mae: 0.0792
Epoch 8/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0108 - mae: 0.0758 - val_loss: 0.0110 - val_mae: 0.0756
Epoch 9/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/

In [25]:
# Train Transformer model
transformer_model_1r95_8 = create_transformer_model(
    df_scaled=df_selected_1r95_8,
    no_columns=len(selected_stations_1r95_8),
    n_steps=24,
    epochs=100,          # More epochs, since the model is smaller
    batch_size=16,       # Smaller batch for better generalization
    head_size=32,        # Reduce head size to avoid overfitting
    num_heads=2,         # Keep 2 heads, enough for diversity but not too heavy
    ff_dim=64,           # Reduce feed-forward dimension
    num_blocks=1,        # Fewer transformer blocks for smaller data
    dropout=0.2,         # Slightly higher dropout to regularize
    learning_rate=0.0005 # Lower LR for stable training
)

# Save Transformer model
transformer_model_1r95_8.save("transformer_models/transformer_model_1r95_8.keras")


Epoch 1/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.4070 - mae: 0.4974 - val_loss: 0.0933 - val_mae: 0.2487
Epoch 2/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0778 - mae: 0.2289 - val_loss: 0.0601 - val_mae: 0.1995
Epoch 3/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0524 - mae: 0.1867 - val_loss: 0.0498 - val_mae: 0.1725
Epoch 4/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0443 - mae: 0.1641 - val_loss: 0.0437 - val_mae: 0.1590
Epoch 5/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0401 - mae: 0.1540 - val_loss: 0.0392 - val_mae: 0.1501
Epoch 6/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0344 - mae: 0.1423 - val_loss: 0.0354 - val_mae: 0.1416
Epoch 7/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0315 - mae: 0.1352 - val_loss: 0.0308 - val_mae: 0.1328
Epoch 8/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0264 - mae: 0.1240 - val_loss: 0.0267 - val_mae: 0.1227
Epoch 9/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/

In [26]:
# Train Transformer model
transformer_model_1r95_9 = create_transformer_model(
    df_scaled=df_selected_1r95_9,
    no_columns=len(selected_stations_1r95_9),
    n_steps=24,
    epochs=100,          # More epochs, since the model is smaller
    batch_size=16,       # Smaller batch for better generalization
    head_size=32,        # Reduce head size to avoid overfitting
    num_heads=2,         # Keep 2 heads, enough for diversity but not too heavy
    ff_dim=64,           # Reduce feed-forward dimension
    num_blocks=1,        # Fewer transformer blocks for smaller data
    dropout=0.2,         # Slightly higher dropout to regularize
    learning_rate=0.0005 # Lower LR for stable training
)

# Save Transformer model
transformer_model_1r95_9.save("transformer_models/transformer_model_1r95_9.keras")


Epoch 1/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.2903 - mae: 0.3995 - val_loss: 0.0804 - val_mae: 0.2475
Epoch 2/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0800 - mae: 0.2506 - val_loss: 0.0718 - val_mae: 0.2339
Epoch 3/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0734 - mae: 0.2396 - val_loss: 0.0640 - val_mae: 0.2185
Epoch 4/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0596 - mae: 0.2096 - val_loss: 0.0425 - val_mae: 0.1708
Epoch 5/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0368 - mae: 0.1586 - val_loss: 0.0287 - val_mae: 0.1394
Epoch 6/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0250 - mae: 0.1291 - val_loss: 0.0222 - val_mae: 0.1194
Epoch 7/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0195 - mae: 0.1093 - val_loss: 0.0186 - val_mae: 0.1072
Epoch 8/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0167 - mae: 0.0993 - val_loss: 0.0174 - val_mae: 0.1010
Epoch 9/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/

In [27]:
# Train Transformer model
transformer_model_1r95_10 = create_transformer_model(
    df_scaled=df_selected_1r95_10,
    no_columns=len(selected_stations_1r95_10),
    n_steps=24,
    epochs=100,          # More epochs, since the model is smaller
    batch_size=16,       # Smaller batch for better generalization
    head_size=32,        # Reduce head size to avoid overfitting
    num_heads=2,         # Keep 2 heads, enough for diversity but not too heavy
    ff_dim=64,           # Reduce feed-forward dimension
    num_blocks=1,        # Fewer transformer blocks for smaller data
    dropout=0.2,         # Slightly higher dropout to regularize
    learning_rate=0.0005 # Lower LR for stable training
)

# Save Transformer model
transformer_model_1r95_10.save("transformer_models/transformer_model_1r95_10.keras")


Epoch 1/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.3750 - mae: 0.4940 - val_loss: 0.1024 - val_mae: 0.2703
Epoch 2/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0942 - mae: 0.2649 - val_loss: 0.0751 - val_mae: 0.2373
Epoch 3/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0704 - mae: 0.2319 - val_loss: 0.0539 - val_mae: 0.1940
Epoch 4/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0477 - mae: 0.1812 - val_loss: 0.0359 - val_mae: 0.1530
Epoch 5/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0314 - mae: 0.1437 - val_loss: 0.0241 - val_mae: 0.1224
Epoch 6/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0212 - mae: 0.1136 - val_loss: 0.0184 - val_mae: 0.1013
Epoch 7/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0160 - mae: 0.0950 - val_loss: 0.0158 - val_mae: 0.0945
Epoch 8/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0133 - mae: 0.0851 - val_loss: 0.0152 - val_mae: 0.0901
Epoch 9/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/

#### 1 > r > 0.90

In [216]:
# Train Transformer model
transformer_model_1r90_1 = create_transformer_model(
    df_scaled=df_selected_1r90_1,
    no_columns=len(selected_stations_1r90_1),
    n_steps=24,
    epochs=50,
    batch_size=32,
    head_size=64,
    num_heads=2,
    ff_dim=128,
    num_blocks=2,
    dropout=0.1,
    learning_rate=0.001
)

# Save Transformer model
transformer_model_1r90_1.save("transformer_models/transformer_model_1r90_1.keras")


Epoch 1/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 7s 10ms/step - loss: 0.2644 - mae: 0.4445 - val_loss: 0.1730 - val_mae: 0.3449
Epoch 2/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.1789 - mae: 0.3529 - val_loss: 0.1152 - val_mae: 0.2758
Epoch 3/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.1256 - mae: 0.2962 - val_loss: 0.0809 - val_mae: 0.2374
Epoch 4/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0903 - mae: 0.2605 - val_loss: 0.0646 - val_mae: 0.2173
Epoch 5/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0749 - mae: 0.2413 - val_loss: 0.0592 - val_mae: 0.2104
Epoch 6/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0665 - mae: 0.2286 - val_loss: 0.0583 - val_mae: 0.2098
Epoch 7/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0662 - mae: 0.2292 - val_loss: 0.0585 - val_mae: 0.2103
Epoch 8/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0642 - mae: 0.2241 - val_loss: 0.0588 - val_mae: 0.2107
Epoch 9/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0634 - mae:

In [217]:
# Train Transformer model
transformer_model_1r90_2 = create_transformer_model(
    df_scaled=df_selected_1r90_2,
    no_columns=len(selected_stations_1r90_2),
    n_steps=24,
    epochs=50,
    batch_size=32,
    head_size=64,
    num_heads=2,
    ff_dim=128,
    num_blocks=2,
    dropout=0.1,
    learning_rate=0.001
)

# Save Transformer model
transformer_model_1r90_2.save("transformer_models/transformer_model_1r90_2.keras")


Epoch 1/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - loss: 0.8392 - mae: 0.8119 - val_loss: 0.5754 - val_mae: 0.6428
Epoch 2/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.5629 - mae: 0.6370 - val_loss: 0.2074 - val_mae: 0.3842
Epoch 3/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.1888 - mae: 0.3652 - val_loss: 0.1413 - val_mae: 0.3089
Epoch 4/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.1289 - mae: 0.2956 - val_loss: 0.1060 - val_mae: 0.2661
Epoch 5/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 0.0959 - mae: 0.2517 - val_loss: 0.0862 - val_mae: 0.2442
Epoch 6/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0841 - mae: 0.2391 - val_loss: 0.0749 - val_mae: 0.2322
Epoch 7/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0741 - mae: 0.2302 - val_loss: 0.0681 - val_mae: 0.2245
Epoch 8/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0695 - mae: 0.2284 - val_loss: 0.0645 - val_mae: 0.2204
Epoch 9/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0645 - mae

In [218]:
# Train Transformer model
transformer_model_1r90_3 = create_transformer_model(
    df_scaled=df_selected_1r90_3,
    no_columns=len(selected_stations_1r90_3),
    n_steps=24,
    epochs=50,
    batch_size=32,
    head_size=64,
    num_heads=2,
    ff_dim=128,
    num_blocks=2,
    dropout=0.1,
    learning_rate=0.001
)

# Save Transformer model
transformer_model_1r90_3.save("transformer_models/transformer_model_1r90_3.keras")


Epoch 1/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - loss: 1.3802 - mae: 0.8969 - val_loss: 0.8156 - val_mae: 0.6650
Epoch 2/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.8015 - mae: 0.6560 - val_loss: 0.5730 - val_mae: 0.5597
Epoch 3/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.5790 - mae: 0.5648 - val_loss: 0.4169 - val_mae: 0.4845
Epoch 4/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.4263 - mae: 0.4940 - val_loss: 0.3131 - val_mae: 0.4244
Epoch 5/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.3301 - mae: 0.4407 - val_loss: 0.2404 - val_mae: 0.3795
Epoch 6/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.2564 - mae: 0.3945 - val_loss: 0.1905 - val_mae: 0.3383
Epoch 7/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 0.1976 - mae: 0.3472 - val_loss: 0.1547 - val_mae: 0.3024
Epoch 8/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - loss: 0.1667 - mae: 0.3160 - val_loss: 0.1273 - val_mae: 0.2721
Epoch 9/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.1343 - 

In [219]:
# Train Transformer model
transformer_model_1r90_4 = create_transformer_model(
    df_scaled=df_selected_1r90_4,
    no_columns=len(selected_stations_1r90_4),
    n_steps=24,
    epochs=50,
    batch_size=32,
    head_size=64,
    num_heads=2,
    ff_dim=128,
    num_blocks=2,
    dropout=0.1,
    learning_rate=0.001
)

# Save Transformer model
transformer_model_1r90_4.save("transformer_models/transformer_model_1r90_4.keras")


Epoch 1/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 6s 15ms/step - loss: 0.3407 - mae: 0.4266 - val_loss: 0.0846 - val_mae: 0.2446
Epoch 2/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0840 - mae: 0.2487 - val_loss: 0.0663 - val_mae: 0.2224
Epoch 3/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0669 - mae: 0.2265 - val_loss: 0.0622 - val_mae: 0.2160
Epoch 4/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0651 - mae: 0.2242 - val_loss: 0.0596 - val_mae: 0.2089
Epoch 5/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0594 - mae: 0.2126 - val_loss: 0.0523 - val_mae: 0.1911
Epoch 6/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0463 - mae: 0.1825 - val_loss: 0.0306 - val_mae: 0.1438
Epoch 7/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0309 - mae: 0.1412 - val_loss: 0.0231 - val_mae: 0.1224
Epoch 8/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0237 - mae: 0.1218 - val_loss: 0.0192 - val_mae: 0.1097
Epoch 9/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 0.0211 - mae

In [220]:
# Train Transformer model
transformer_model_1r90_5 = create_transformer_model(
    df_scaled=df_selected_1r90_5,
    no_columns=len(selected_stations_1r90_5),
    n_steps=24,
    epochs=50,
    batch_size=32,
    head_size=64,
    num_heads=2,
    ff_dim=128,
    num_blocks=2,
    dropout=0.1,
    learning_rate=0.001
)

# Save Transformer model
transformer_model_1r90_5.save("transformer_models/transformer_model_1r90_5.keras")


Epoch 1/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step - loss: 0.3114 - mae: 0.4029 - val_loss: 0.0748 - val_mae: 0.2332
Epoch 2/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0703 - mae: 0.2294 - val_loss: 0.0618 - val_mae: 0.2163
Epoch 3/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0645 - mae: 0.2257 - val_loss: 0.0590 - val_mae: 0.2095
Epoch 4/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0612 - mae: 0.2176 - val_loss: 0.0554 - val_mae: 0.2020
Epoch 5/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0568 - mae: 0.2068 - val_loss: 0.0492 - val_mae: 0.1859
Epoch 6/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0469 - mae: 0.1797 - val_loss: 0.0383 - val_mae: 0.1581
Epoch 7/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0358 - mae: 0.1494 - val_loss: 0.0294 - val_mae: 0.1303
Epoch 8/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 0.0269 - mae: 0.1241 - val_loss: 0.0234 - val_mae: 0.1173
Epoch 9/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0213 - ma

In [221]:
# Train Transformer model
transformer_model_1r90_6 = create_transformer_model(
    df_scaled=df_selected_1r90_6,
    no_columns=len(selected_stations_1r90_6),
    n_steps=24,
    epochs=50,
    batch_size=32,
    head_size=64,
    num_heads=2,
    ff_dim=128,
    num_blocks=2,
    dropout=0.1,
    learning_rate=0.001
)

# Save Transformer model
transformer_model_1r90_6.save("transformer_models/transformer_model_1r90_6.keras")


Epoch 1/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 5s 24ms/step - loss: 0.6584 - mae: 0.6488 - val_loss: 0.1588 - val_mae: 0.3249
Epoch 2/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.1577 - mae: 0.3302 - val_loss: 0.0915 - val_mae: 0.2495
Epoch 3/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0948 - mae: 0.2617 - val_loss: 0.0674 - val_mae: 0.2220
Epoch 4/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0740 - mae: 0.2396 - val_loss: 0.0578 - val_mae: 0.2068
Epoch 5/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0610 - mae: 0.2139 - val_loss: 0.0503 - val_mae: 0.1888
Epoch 6/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0514 - mae: 0.1906 - val_loss: 0.0415 - val_mae: 0.1657
Epoch 7/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0409 - mae: 0.1639 - val_loss: 0.0361 - val_mae: 0.1520
Epoch 8/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0352 - mae: 0.1487 - val_loss: 0.0315 - val_mae: 0.1402
Epoch 9/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0301 - mae:

In [32]:
# Train Transformer model
transformer_model_1r90_7 = create_transformer_model(
    df_scaled=df_selected_1r90_7,
    no_columns=len(selected_stations_1r90_7),
    n_steps=24,
    epochs=100,          # More epochs, since the model is smaller
    batch_size=16,       # Smaller batch for better generalization
    head_size=32,        # Reduce head size to avoid overfitting
    num_heads=2,         # Keep 2 heads, enough for diversity but not too heavy
    ff_dim=64,           # Reduce feed-forward dimension
    num_blocks=1,        # Fewer transformer blocks for smaller data
    dropout=0.2,         # Slightly higher dropout to regularize
    learning_rate=0.01 # Lower LR for stable training
)

# Save Transformer model
transformer_model_1r90_7.save("transformer_models/transformer_model_1r90_7.keras")


Epoch 1/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.1548 - mae: 0.2945 - val_loss: 0.0149 - val_mae: 0.0913
Epoch 2/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0171 - mae: 0.0977 - val_loss: 0.0137 - val_mae: 0.0912
Epoch 3/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0124 - mae: 0.0828 - val_loss: 0.0118 - val_mae: 0.0835
Epoch 4/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0112 - mae: 0.0775 - val_loss: 0.0121 - val_mae: 0.0796
Epoch 5/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0118 - mae: 0.0807 - val_loss: 0.0107 - val_mae: 0.0735
Epoch 6/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0106 - mae: 0.0763 - val_loss: 0.0102 - val_mae: 0.0732
Epoch 7/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0095 - mae: 0.0716 - val_loss: 0.0116 - val_mae: 0.0816
Epoch 8/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0097 - mae: 0.0722 - val_loss: 0.0101 - val_mae: 0.0703
Epoch 9/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/

In [223]:
# Train Transformer model
transformer_model_1r90_8 = create_transformer_model(
    df_scaled=df_selected_1r90_8,
    no_columns=len(selected_stations_1r90_8),
    n_steps=24,
    epochs=50,
    batch_size=32,
    head_size=64,
    num_heads=2,
    ff_dim=128,
    num_blocks=2,
    dropout=0.1,
    learning_rate=0.001
)

# Save Transformer model
transformer_model_1r90_8.save("transformer_models/transformer_model_1r90_8.keras")


Epoch 1/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 5s 13ms/step - loss: 0.3388 - mae: 0.4199 - val_loss: 0.0721 - val_mae: 0.2330
Epoch 2/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0722 - mae: 0.2369 - val_loss: 0.0656 - val_mae: 0.2225
Epoch 3/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0709 - mae: 0.2360 - val_loss: 0.0623 - val_mae: 0.2160
Epoch 4/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 0.0657 - mae: 0.2256 - val_loss: 0.0519 - val_mae: 0.1921
Epoch 5/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 0.0456 - mae: 0.1790 - val_loss: 0.0261 - val_mae: 0.1299
Epoch 6/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0252 - mae: 0.1286 - val_loss: 0.0190 - val_mae: 0.1086
Epoch 7/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0189 - mae: 0.1083 - val_loss: 0.0154 - val_mae: 0.0952
Epoch 8/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0145 - mae: 0.0932 - val_loss: 0.0154 - val_mae: 0.0977
Epoch 9/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0135 - m

In [224]:
# Train Transformer model
transformer_model_1r90_9 = create_transformer_model(
    df_scaled=df_selected_1r90_9,
    no_columns=len(selected_stations_1r90_9),
    n_steps=24,
    epochs=50,
    batch_size=32,
    head_size=64,
    num_heads=2,
    ff_dim=128,
    num_blocks=2,
    dropout=0.1,
    learning_rate=0.001
)

# Save Transformer model
transformer_model_1r90_9.save("transformer_models/transformer_model_1r90_9.keras")


Epoch 1/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 6s 21ms/step - loss: 0.6007 - mae: 0.5853 - val_loss: 0.1781 - val_mae: 0.3344
Epoch 2/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.1482 - mae: 0.3129 - val_loss: 0.0726 - val_mae: 0.2336
Epoch 3/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 0.0745 - mae: 0.2416 - val_loss: 0.0681 - val_mae: 0.2270
Epoch 4/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0702 - mae: 0.2335 - val_loss: 0.0638 - val_mae: 0.2175
Epoch 5/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0642 - mae: 0.2185 - val_loss: 0.0531 - val_mae: 0.1881
Epoch 6/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0516 - mae: 0.1851 - val_loss: 0.0447 - val_mae: 0.1669
Epoch 7/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0443 - mae: 0.1657 - val_loss: 0.0371 - val_mae: 0.1500
Epoch 8/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 0.0345 - mae: 0.1436 - val_loss: 0.0243 - val_mae: 0.1226
Epoch 9/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 0.0190 - 

In [246]:
# Train Transformer model
transformer_model_1r90_10 = create_transformer_model(
    df_scaled=df_selected_1r90_10,
    no_columns=len(selected_stations_1r90_10),
    n_steps=24,
    epochs=50,
    batch_size=32,
    head_size=64,
    num_heads=2,
    ff_dim=128,
    num_blocks=2,
    dropout=0.1,
    learning_rate=0.001
)

# Save Transformer model
transformer_model_1r90_10.save("transformer_models/transformer_model_1r90_10.keras")


Epoch 1/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 7s 11ms/step - loss: 0.5651 - mae: 0.5472 - val_loss: 0.0745 - val_mae: 0.2355
Epoch 2/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0778 - mae: 0.2468 - val_loss: 0.0675 - val_mae: 0.2245
Epoch 3/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0720 - mae: 0.2363 - val_loss: 0.0644 - val_mae: 0.2189
Epoch 4/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0645 - mae: 0.2207 - val_loss: 0.0501 - val_mae: 0.1838
Epoch 5/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0471 - mae: 0.1763 - val_loss: 0.0296 - val_mae: 0.1380
Epoch 6/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0232 - mae: 0.1222 - val_loss: 0.0156 - val_mae: 0.0969
Epoch 7/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0114 - mae: 0.0824 - val_loss: 0.0126 - val_mae: 0.0839
Epoch 8/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0093 - mae: 0.0728 - val_loss: 0.0119 - val_mae: 0.0807
Epoch 9/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0088 - mae:

#### 1 > r > 0.85

In [226]:
# Train Transformer model
transformer_model_1r85_1 = create_transformer_model(
    df_scaled=df_selected_1r85_1,
    no_columns=len(selected_stations_1r85_1),
    n_steps=24,
    epochs=50,
    batch_size=32,
    head_size=64,
    num_heads=2,
    ff_dim=128,
    num_blocks=2,
    dropout=0.1,
    learning_rate=0.001
)

# Save Transformer model
transformer_model_1r85_1.save("transformer_models/transformer_model_1r85_1.keras")


Epoch 1/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 31s 11ms/step - loss: 0.3434 - mae: 0.5097 - val_loss: 0.2358 - val_mae: 0.4058
Epoch 2/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.2553 - mae: 0.4303 - val_loss: 0.1636 - val_mae: 0.3365
Epoch 3/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.1757 - mae: 0.3620 - val_loss: 0.1177 - val_mae: 0.2913
Epoch 4/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.1249 - mae: 0.3084 - val_loss: 0.0931 - val_mae: 0.2634
Epoch 5/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0992 - mae: 0.2809 - val_loss: 0.0832 - val_mae: 0.2506
Epoch 6/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0914 - mae: 0.2698 - val_loss: 0.0806 - val_mae: 0.2463
Epoch 7/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0878 - mae: 0.2619 - val_loss: 0.0806 - val_mae: 0.2454
Epoch 8/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0850 - mae: 0.2568 - val_loss: 0.0810 - val_mae: 0.2454
Epoch 9/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0863 - mae

In [227]:
# Train Transformer model
transformer_model_1r85_2 = create_transformer_model(
    df_scaled=df_selected_1r85_2,
    no_columns=len(selected_stations_1r85_2),
    n_steps=24,
    epochs=50,
    batch_size=32,
    head_size=64,
    num_heads=2,
    ff_dim=128,
    num_blocks=2,
    dropout=0.1,
    learning_rate=0.001
)

# Save Transformer model
transformer_model_1r85_2.save("transformer_models/transformer_model_1r85_2.keras")


Epoch 1/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 6s 15ms/step - loss: 0.3912 - mae: 0.5215 - val_loss: 0.2223 - val_mae: 0.3951
Epoch 2/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.2153 - mae: 0.3862 - val_loss: 0.1319 - val_mae: 0.2947
Epoch 3/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.1338 - mae: 0.3025 - val_loss: 0.0965 - val_mae: 0.2543
Epoch 4/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.1044 - mae: 0.2703 - val_loss: 0.0859 - val_mae: 0.2417
Epoch 5/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0960 - mae: 0.2607 - val_loss: 0.0833 - val_mae: 0.2393
Epoch 6/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0862 - mae: 0.2505 - val_loss: 0.0826 - val_mae: 0.2392
Epoch 7/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0888 - mae: 0.2534 - val_loss: 0.0816 - val_mae: 0.2388
Epoch 8/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0873 - mae: 0.2514 - val_loss: 0.0806 - val_mae: 0.2383
Epoch 9/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0886 - mae:

In [228]:
# Train Transformer model
transformer_model_1r85_3 = create_transformer_model(
    df_scaled=df_selected_1r85_3,
    no_columns=len(selected_stations_1r85_3),
    n_steps=24,
    epochs=50,
    batch_size=32,
    head_size=64,
    num_heads=2,
    ff_dim=128,
    num_blocks=2,
    dropout=0.1,
    learning_rate=0.001
)

# Save Transformer model
transformer_model_1r85_3.save("transformer_models/transformer_model_1r85_3.keras")


Epoch 1/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 7s 11ms/step - loss: 0.3201 - mae: 0.4600 - val_loss: 0.1713 - val_mae: 0.3285
Epoch 2/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.1750 - mae: 0.3398 - val_loss: 0.1061 - val_mae: 0.2637
Epoch 3/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.1083 - mae: 0.2766 - val_loss: 0.0752 - val_mae: 0.2311
Epoch 4/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 0.0831 - mae: 0.2499 - val_loss: 0.0587 - val_mae: 0.2035
Epoch 5/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0631 - mae: 0.2116 - val_loss: 0.0476 - val_mae: 0.1772
Epoch 6/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0509 - mae: 0.1828 - val_loss: 0.0427 - val_mae: 0.1634
Epoch 7/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0428 - mae: 0.1626 - val_loss: 0.0391 - val_mae: 0.1532
Epoch 8/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0411 - mae: 0.1572 - val_loss: 0.0365 - val_mae: 0.1472
Epoch 9/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0374 - ma

In [229]:
# Train Transformer model
transformer_model_1r85_4 = create_transformer_model(
    df_scaled=df_selected_1r85_4,
    no_columns=len(selected_stations_1r85_4),
    n_steps=24,
    epochs=50,
    batch_size=32,
    head_size=64,
    num_heads=2,
    ff_dim=128,
    num_blocks=2,
    dropout=0.1,
    learning_rate=0.001
)

# Save Transformer model
transformer_model_1r85_4.save("transformer_models/transformer_model_1r85_4.keras")


Epoch 1/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 5s 14ms/step - loss: 0.4806 - mae: 0.5365 - val_loss: 0.0852 - val_mae: 0.2461
Epoch 2/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0894 - mae: 0.2603 - val_loss: 0.0760 - val_mae: 0.2395
Epoch 3/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0827 - mae: 0.2558 - val_loss: 0.0756 - val_mae: 0.2397
Epoch 4/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0793 - mae: 0.2483 - val_loss: 0.0749 - val_mae: 0.2394
Epoch 5/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0802 - mae: 0.2515 - val_loss: 0.0745 - val_mae: 0.2383
Epoch 6/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0800 - mae: 0.2519 - val_loss: 0.0739 - val_mae: 0.2374
Epoch 7/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0777 - mae: 0.2483 - val_loss: 0.0721 - val_mae: 0.2346
Epoch 8/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0785 - mae: 0.2498 - val_loss: 0.0687 - val_mae: 0.2263
Epoch 9/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0726 - mae:

In [230]:
# Train Transformer model
transformer_model_1r85_5 = create_transformer_model(
    df_scaled=df_selected_1r85_5,
    no_columns=len(selected_stations_1r85_5),
    n_steps=24,
    epochs=50,
    batch_size=32,
    head_size=64,
    num_heads=2,
    ff_dim=128,
    num_blocks=2,
    dropout=0.1,
    learning_rate=0.001
)

# Save Transformer model
transformer_model_1r85_5.save("transformer_models/transformer_model_1r85_5.keras")


Epoch 1/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 5s 15ms/step - loss: 0.2447 - mae: 0.3903 - val_loss: 0.0819 - val_mae: 0.2410
Epoch 2/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0796 - mae: 0.2432 - val_loss: 0.0655 - val_mae: 0.2189
Epoch 3/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0639 - mae: 0.2197 - val_loss: 0.0548 - val_mae: 0.1988
Epoch 4/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0494 - mae: 0.1913 - val_loss: 0.0401 - val_mae: 0.1660
Epoch 5/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0367 - mae: 0.1609 - val_loss: 0.0329 - val_mae: 0.1489
Epoch 6/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0278 - mae: 0.1379 - val_loss: 0.0264 - val_mae: 0.1287
Epoch 7/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0231 - mae: 0.1235 - val_loss: 0.0226 - val_mae: 0.1170
Epoch 8/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0202 - mae: 0.1131 - val_loss: 0.0202 - val_mae: 0.1090
Epoch 9/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0177 - mae

In [231]:
# Train Transformer model
transformer_model_1r85_6 = create_transformer_model(
    df_scaled=df_selected_1r85_6,
    no_columns=len(selected_stations_1r85_6),
    n_steps=24,
    epochs=50,
    batch_size=32,
    head_size=64,
    num_heads=2,
    ff_dim=128,
    num_blocks=2,
    dropout=0.1,
    learning_rate=0.001
)

# Save Transformer model
transformer_model_1r85_6.save("transformer_models/transformer_model_1r85_6.keras")


Epoch 1/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 5s 12ms/step - loss: 0.2094 - mae: 0.3620 - val_loss: 0.0775 - val_mae: 0.2405
Epoch 2/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0770 - mae: 0.2464 - val_loss: 0.0695 - val_mae: 0.2277
Epoch 3/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0716 - mae: 0.2369 - val_loss: 0.0679 - val_mae: 0.2271
Epoch 4/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0697 - mae: 0.2326 - val_loss: 0.0541 - val_mae: 0.1963
Epoch 5/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0486 - mae: 0.1883 - val_loss: 0.0336 - val_mae: 0.1524
Epoch 6/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0304 - mae: 0.1458 - val_loss: 0.0270 - val_mae: 0.1336
Epoch 7/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0243 - mae: 0.1277 - val_loss: 0.0242 - val_mae: 0.1253
Epoch 8/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0193 - mae: 0.1116 - val_loss: 0.0190 - val_mae: 0.1077
Epoch 9/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0181 - mae

In [232]:
# Train Transformer model
transformer_model_1r85_7 = create_transformer_model(
    df_scaled=df_selected_1r85_7,
    no_columns=len(selected_stations_1r85_7),
    n_steps=24,
    epochs=50,
    batch_size=32,
    head_size=64,
    num_heads=2,
    ff_dim=128,
    num_blocks=2,
    dropout=0.1,
    learning_rate=0.001
)

# Save Transformer model
transformer_model_1r85_7.save("transformer_models/transformer_model_1r85_7.keras")


Epoch 1/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 6s 26ms/step - loss: 0.4095 - mae: 0.5039 - val_loss: 0.1024 - val_mae: 0.2657
Epoch 2/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0911 - mae: 0.2550 - val_loss: 0.0655 - val_mae: 0.2187
Epoch 3/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0604 - mae: 0.2096 - val_loss: 0.0384 - val_mae: 0.1582
Epoch 4/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0300 - mae: 0.1403 - val_loss: 0.0233 - val_mae: 0.1202
Epoch 5/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0166 - mae: 0.1025 - val_loss: 0.0206 - val_mae: 0.1111
Epoch 6/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 0.0129 - mae: 0.0888 - val_loss: 0.0167 - val_mae: 0.0980
Epoch 7/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0117 - mae: 0.0827 - val_loss: 0.0173 - val_mae: 0.0998
Epoch 8/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0115 - mae: 0.0826 - val_loss: 0.0152 - val_mae: 0.0926
Epoch 9/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0110 - mae

In [233]:
# Train Transformer model
transformer_model_1r85_8 = create_transformer_model(
    df_scaled=df_selected_1r85_8,
    no_columns=len(selected_stations_1r85_8),
    n_steps=24,
    epochs=50,
    batch_size=32,
    head_size=64,
    num_heads=2,
    ff_dim=128,
    num_blocks=2,
    dropout=0.1,
    learning_rate=0.001
)

# Save Transformer model
transformer_model_1r85_8.save("transformer_models/transformer_model_1r85_8.keras")


Epoch 1/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 5s 13ms/step - loss: 0.2110 - mae: 0.3315 - val_loss: 0.0488 - val_mae: 0.1867
Epoch 2/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0448 - mae: 0.1775 - val_loss: 0.0308 - val_mae: 0.1347
Epoch 3/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0243 - mae: 0.1205 - val_loss: 0.0204 - val_mae: 0.1098
Epoch 4/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0150 - mae: 0.0938 - val_loss: 0.0157 - val_mae: 0.0924
Epoch 5/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0122 - mae: 0.0825 - val_loss: 0.0146 - val_mae: 0.0875
Epoch 6/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0123 - mae: 0.0816 - val_loss: 0.0137 - val_mae: 0.0839
Epoch 7/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0110 - mae: 0.0767 - val_loss: 0.0132 - val_mae: 0.0823
Epoch 8/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0100 - mae: 0.0723 - val_loss: 0.0122 - val_mae: 0.0791
Epoch 9/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 0.0094 - ma

In [234]:
# Train Transformer model
transformer_model_1r85_9 = create_transformer_model(
    df_scaled=df_selected_1r85_9,
    no_columns=len(selected_stations_1r85_9),
    n_steps=24,
    epochs=50,
    batch_size=32,
    head_size=64,
    num_heads=2,
    ff_dim=128,
    num_blocks=2,
    dropout=0.1,
    learning_rate=0.001
)

# Save Transformer model
transformer_model_1r85_9.save("transformer_models/transformer_model_1r85_9.keras")


Epoch 1/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 5s 19ms/step - loss: 0.6792 - mae: 0.6279 - val_loss: 0.1097 - val_mae: 0.2661
Epoch 2/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0902 - mae: 0.2447 - val_loss: 0.0645 - val_mae: 0.2147
Epoch 3/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0611 - mae: 0.2125 - val_loss: 0.0416 - val_mae: 0.1729
Epoch 4/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0343 - mae: 0.1552 - val_loss: 0.0268 - val_mae: 0.1300
Epoch 5/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0221 - mae: 0.1183 - val_loss: 0.0215 - val_mae: 0.1156
Epoch 6/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0182 - mae: 0.1067 - val_loss: 0.0185 - val_mae: 0.1057
Epoch 7/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0167 - mae: 0.1012 - val_loss: 0.0167 - val_mae: 0.0976
Epoch 8/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 0.0127 - mae: 0.0864 - val_loss: 0.0155 - val_mae: 0.0945
Epoch 9/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0128 - ma

In [41]:
# Train Transformer model
transformer_model_1r85_10 = create_transformer_model(
    df_scaled=df_selected_1r85_10,
    no_columns=len(selected_stations_1r85_10),
    n_steps=24,
    epochs=100,          # More epochs, since the model is smaller
    batch_size=16,       # Smaller batch for better generalization
    head_size=32,        # Reduce head size to avoid overfitting
    num_heads=2,         # Keep 2 heads, enough for diversity but not too heavy
    ff_dim=64,           # Reduce feed-forward dimension
    num_blocks=1,        # Fewer transformer blocks for smaller data
    dropout=0.2,         # Slightly higher dropout to regularize
    learning_rate=0.01 # Lower LR for stable training
)

# Save Transformer model
transformer_model_1r85_10.save("transformer_models/transformer_model_1r85_10.keras")


Epoch 1/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.1895 - mae: 0.3107 - val_loss: 0.0252 - val_mae: 0.1247
Epoch 2/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0168 - mae: 0.0992 - val_loss: 0.0145 - val_mae: 0.0907
Epoch 3/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0110 - mae: 0.0791 - val_loss: 0.0116 - val_mae: 0.0814
Epoch 4/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0095 - mae: 0.0737 - val_loss: 0.0169 - val_mae: 0.1018
Epoch 5/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0114 - mae: 0.0804 - val_loss: 0.0106 - val_mae: 0.0735
Epoch 6/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0084 - mae: 0.0682 - val_loss: 0.0099 - val_mae: 0.0710
Epoch 7/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0081 - mae: 0.0672 - val_loss: 0.0096 - val_mae: 0.0704
Epoch 8/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0079 - mae: 0.0663 - val_loss: 0.0097 - val_mae: 0.0706
Epoch 9/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/

#### 1 > r > 0.80

In [247]:
# Train Transformer model
transformer_model_1r80_1 = create_transformer_model(
    df_scaled=df_selected_1r80_1,
    no_columns=len(selected_stations_1r80_1),
    n_steps=24,
    epochs=50,
    batch_size=32,
    head_size=64,
    num_heads=2,
    ff_dim=128,
    num_blocks=2,
    dropout=0.1,
    learning_rate=0.001
)

# Save Transformer model
transformer_model_1r80_1.save("transformer_models/transformer_model_1r80_1.keras")


Epoch 1/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 8s 29ms/step - loss: 0.2378 - mae: 0.4228 - val_loss: 0.1442 - val_mae: 0.3173
Epoch 2/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.1459 - mae: 0.3232 - val_loss: 0.0898 - val_mae: 0.2547
Epoch 3/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0913 - mae: 0.2584 - val_loss: 0.0647 - val_mae: 0.2215
Epoch 4/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0702 - mae: 0.2352 - val_loss: 0.0560 - val_mae: 0.2072
Epoch 5/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0643 - mae: 0.2267 - val_loss: 0.0546 - val_mae: 0.2025
Epoch 6/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0617 - mae: 0.2216 - val_loss: 0.0546 - val_mae: 0.2011
Epoch 7/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0604 - mae: 0.2180 - val_loss: 0.0549 - val_mae: 0.2008
Epoch 8/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0613 - mae: 0.2192 - val_loss: 0.0549 - val_mae: 0.2008
Epoch 9/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0617 - mae:

In [248]:
# Train Transformer model
transformer_model_1r80_2 = create_transformer_model(
    df_scaled=df_selected_1r80_2,
    no_columns=len(selected_stations_1r80_2),
    n_steps=24,
    epochs=50,
    batch_size=32,
    head_size=64,
    num_heads=2,
    ff_dim=128,
    num_blocks=2,
    dropout=0.1,
    learning_rate=0.001
)

# Save Transformer model
transformer_model_1r80_2.save("transformer_models/transformer_model_1r80_2.keras")


Epoch 1/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - loss: 0.0973 - mae: 0.2603 - val_loss: 0.0691 - val_mae: 0.2185
Epoch 2/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0809 - mae: 0.2404 - val_loss: 0.0682 - val_mae: 0.2155
Epoch 3/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0771 - mae: 0.2344 - val_loss: 0.0670 - val_mae: 0.2148
Epoch 4/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0747 - mae: 0.2327 - val_loss: 0.0663 - val_mae: 0.2145
Epoch 5/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0767 - mae: 0.2339 - val_loss: 0.0654 - val_mae: 0.2139
Epoch 6/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0706 - mae: 0.2274 - val_loss: 0.0646 - val_mae: 0.2134
Epoch 7/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0732 - mae: 0.2336 - val_loss: 0.0639 - val_mae: 0.2129
Epoch 8/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0734 - mae: 0.2330 - val_loss: 0.0631 - val_mae: 0.2124
Epoch 9/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0692 - mae:

In [249]:
# Train Transformer model
transformer_model_1r80_3 = create_transformer_model(
    df_scaled=df_selected_1r80_3,
    no_columns=len(selected_stations_1r80_3),
    n_steps=24,
    epochs=50,
    batch_size=32,
    head_size=64,
    num_heads=2,
    ff_dim=128,
    num_blocks=2,
    dropout=0.1,
    learning_rate=0.001
)

# Save Transformer model
transformer_model_1r80_3.save("transformer_models/transformer_model_1r80_3.keras")


Epoch 1/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 6s 18ms/step - loss: 0.6290 - mae: 0.6094 - val_loss: 0.3836 - val_mae: 0.4677
Epoch 2/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.3242 - mae: 0.4406 - val_loss: 0.2378 - val_mae: 0.3813
Epoch 3/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.2014 - mae: 0.3584 - val_loss: 0.1561 - val_mae: 0.3166
Epoch 4/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.1372 - mae: 0.3026 - val_loss: 0.1127 - val_mae: 0.2736
Epoch 5/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.1017 - mae: 0.2615 - val_loss: 0.0878 - val_mae: 0.2441
Epoch 6/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 0.0853 - mae: 0.2430 - val_loss: 0.0781 - val_mae: 0.2324
Epoch 7/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0765 - mae: 0.2339 - val_loss: 0.0673 - val_mae: 0.2196
Epoch 8/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0716 - mae: 0.2316 - val_loss: 0.0642 - val_mae: 0.2149
Epoch 9/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0672 - mae

In [250]:
# Train Transformer model
transformer_model_1r80_4 = create_transformer_model(
    df_scaled=df_selected_1r80_4,
    no_columns=len(selected_stations_1r80_4),
    n_steps=24,
    epochs=50,
    batch_size=32,
    head_size=64,
    num_heads=2,
    ff_dim=128,
    num_blocks=2,
    dropout=0.1,
    learning_rate=0.001
)

# Save Transformer model
transformer_model_1r80_4.save("transformer_models/transformer_model_1r80_4.keras")


Epoch 1/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - loss: 0.7673 - mae: 0.7476 - val_loss: 0.3802 - val_mae: 0.5349
Epoch 2/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.3422 - mae: 0.4956 - val_loss: 0.1990 - val_mae: 0.3720
Epoch 3/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.1775 - mae: 0.3453 - val_loss: 0.1076 - val_mae: 0.2664
Epoch 4/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0998 - mae: 0.2583 - val_loss: 0.0654 - val_mae: 0.2096
Epoch 5/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0602 - mae: 0.2014 - val_loss: 0.0410 - val_mae: 0.1625
Epoch 6/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0332 - mae: 0.1466 - val_loss: 0.0310 - val_mae: 0.1365
Epoch 7/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0268 - mae: 0.1278 - val_loss: 0.0258 - val_mae: 0.1254
Epoch 8/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0249 - mae: 0.1217 - val_loss: 0.0245 - val_mae: 0.1229
Epoch 9/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0241 - mae:

In [251]:
# Train Transformer model
transformer_model_1r80_5 = create_transformer_model(
    df_scaled=df_selected_1r80_5,
    no_columns=len(selected_stations_1r80_5),
    n_steps=24,
    epochs=50,
    batch_size=32,
    head_size=64,
    num_heads=2,
    ff_dim=128,
    num_blocks=2,
    dropout=0.1,
    learning_rate=0.001
)

# Save Transformer model
transformer_model_1r80_5.save("transformer_models/transformer_model_1r80_5.keras")


Epoch 1/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - loss: 0.2758 - mae: 0.3984 - val_loss: 0.0694 - val_mae: 0.2279
Epoch 2/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0774 - mae: 0.2477 - val_loss: 0.0696 - val_mae: 0.2259
Epoch 3/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0738 - mae: 0.2403 - val_loss: 0.0664 - val_mae: 0.2210
Epoch 4/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0728 - mae: 0.2382 - val_loss: 0.0651 - val_mae: 0.2171
Epoch 5/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0693 - mae: 0.2317 - val_loss: 0.0583 - val_mae: 0.2076
Epoch 6/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0600 - mae: 0.2120 - val_loss: 0.0345 - val_mae: 0.1478
Epoch 7/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0310 - mae: 0.1420 - val_loss: 0.0256 - val_mae: 0.1268
Epoch 8/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0227 - mae: 0.1207 - val_loss: 0.0212 - val_mae: 0.1159
Epoch 9/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0207 - ma

In [252]:
# Train Transformer model
transformer_model_1r80_6 = create_transformer_model(
    df_scaled=df_selected_1r80_6,
    no_columns=len(selected_stations_1r80_6),
    n_steps=24,
    epochs=50,
    batch_size=32,
    head_size=64,
    num_heads=2,
    ff_dim=128,
    num_blocks=2,
    dropout=0.1,
    learning_rate=0.001
)

# Save Transformer model
transformer_model_1r80_6.save("transformer_models/transformer_model_1r80_6.keras")


Epoch 1/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - loss: 0.1828 - mae: 0.3258 - val_loss: 0.0640 - val_mae: 0.2193
Epoch 2/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0704 - mae: 0.2359 - val_loss: 0.0616 - val_mae: 0.2150
Epoch 3/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0685 - mae: 0.2325 - val_loss: 0.0598 - val_mae: 0.2098
Epoch 4/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0656 - mae: 0.2248 - val_loss: 0.0548 - val_mae: 0.1992
Epoch 5/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0592 - mae: 0.2108 - val_loss: 0.0437 - val_mae: 0.1734
Epoch 6/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0420 - mae: 0.1685 - val_loss: 0.0288 - val_mae: 0.1353
Epoch 7/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0313 - mae: 0.1399 - val_loss: 0.0226 - val_mae: 0.1185
Epoch 8/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0227 - mae: 0.1153 - val_loss: 0.0185 - val_mae: 0.1045
Epoch 9/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0210 - mae:

In [253]:
# Train Transformer model
transformer_model_1r80_7 = create_transformer_model(
    df_scaled=df_selected_1r80_7,
    no_columns=len(selected_stations_1r80_7),
    n_steps=24,
    epochs=50,
    batch_size=32,
    head_size=64,
    num_heads=2,
    ff_dim=128,
    num_blocks=2,
    dropout=0.1,
    learning_rate=0.001
)

# Save Transformer model
transformer_model_1r80_7.save("transformer_models/transformer_model_1r80_7.keras")


Epoch 1/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 5s 23ms/step - loss: 0.2284 - mae: 0.3460 - val_loss: 0.0664 - val_mae: 0.2216
Epoch 2/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0714 - mae: 0.2331 - val_loss: 0.0627 - val_mae: 0.2163
Epoch 3/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0670 - mae: 0.2252 - val_loss: 0.0581 - val_mae: 0.2054
Epoch 4/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0571 - mae: 0.2022 - val_loss: 0.0402 - val_mae: 0.1610
Epoch 5/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0339 - mae: 0.1452 - val_loss: 0.0257 - val_mae: 0.1245
Epoch 6/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0235 - mae: 0.1154 - val_loss: 0.0203 - val_mae: 0.1087
Epoch 7/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0188 - mae: 0.1006 - val_loss: 0.0183 - val_mae: 0.1034
Epoch 8/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0162 - mae: 0.0929 - val_loss: 0.0164 - val_mae: 0.0964
Epoch 9/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0153 - mae:

In [254]:
# Train Transformer model
transformer_model_1r80_8 = create_transformer_model(
    df_scaled=df_selected_1r80_8,
    no_columns=len(selected_stations_1r80_8),
    n_steps=24,
    epochs=50,
    batch_size=32,
    head_size=64,
    num_heads=2,
    ff_dim=128,
    num_blocks=2,
    dropout=0.1,
    learning_rate=0.001
)

# Save Transformer model
transformer_model_1r80_8.save("transformer_models/transformer_model_1r80_8.keras")


Epoch 1/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 4s 14ms/step - loss: 0.3345 - mae: 0.4439 - val_loss: 0.0705 - val_mae: 0.2262
Epoch 2/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0702 - mae: 0.2298 - val_loss: 0.0612 - val_mae: 0.2122
Epoch 3/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0659 - mae: 0.2243 - val_loss: 0.0596 - val_mae: 0.2051
Epoch 4/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0575 - mae: 0.2042 - val_loss: 0.0491 - val_mae: 0.1854
Epoch 5/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0468 - mae: 0.1798 - val_loss: 0.0363 - val_mae: 0.1550
Epoch 6/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0349 - mae: 0.1505 - val_loss: 0.0264 - val_mae: 0.1294
Epoch 7/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0253 - mae: 0.1257 - val_loss: 0.0197 - val_mae: 0.1104
Epoch 8/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 0.0174 - mae: 0.1019 - val_loss: 0.0167 - val_mae: 0.0991
Epoch 9/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0142 - mae

In [255]:
# Train Transformer model
transformer_model_1r80_9 = create_transformer_model(
    df_scaled=df_selected_1r80_9,
    no_columns=len(selected_stations_1r80_9),
    n_steps=24,
    epochs=50,
    batch_size=32,
    head_size=64,
    num_heads=2,
    ff_dim=128,
    num_blocks=2,
    dropout=0.1,
    learning_rate=0.001
)

# Save Transformer model
transformer_model_1r80_9.save("transformer_models/transformer_model_1r80_9.keras")


Epoch 1/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 4s 15ms/step - loss: 0.3719 - mae: 0.4626 - val_loss: 0.0666 - val_mae: 0.2159
Epoch 2/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0678 - mae: 0.2247 - val_loss: 0.0543 - val_mae: 0.1920
Epoch 3/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0533 - mae: 0.1915 - val_loss: 0.0481 - val_mae: 0.1772
Epoch 4/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0463 - mae: 0.1745 - val_loss: 0.0351 - val_mae: 0.1503
Epoch 5/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0308 - mae: 0.1404 - val_loss: 0.0179 - val_mae: 0.1032
Epoch 6/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0150 - mae: 0.0933 - val_loss: 0.0123 - val_mae: 0.0806
Epoch 7/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0110 - mae: 0.0758 - val_loss: 0.0118 - val_mae: 0.0795
Epoch 8/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0101 - mae: 0.0728 - val_loss: 0.0118 - val_mae: 0.0794
Epoch 9/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0091 - ma

In [256]:
# Train Transformer model
transformer_model_1r80_10 = create_transformer_model(
    df_scaled=df_selected_1r80_10,
    no_columns=len(selected_stations_1r80_10),
    n_steps=24,
    epochs=50,
    batch_size=32,
    head_size=64,
    num_heads=2,
    ff_dim=128,
    num_blocks=2,
    dropout=0.1,
    learning_rate=0.001
)

# Save Transformer model
transformer_model_1r80_10.save("transformer_models/transformer_model_1r80_10.keras")


Epoch 1/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - loss: 0.4942 - mae: 0.5376 - val_loss: 0.1101 - val_mae: 0.2596
Epoch 2/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0979 - mae: 0.2513 - val_loss: 0.0520 - val_mae: 0.1877
Epoch 3/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0519 - mae: 0.1868 - val_loss: 0.0409 - val_mae: 0.1560
Epoch 4/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0411 - mae: 0.1569 - val_loss: 0.0351 - val_mae: 0.1428
Epoch 5/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0359 - mae: 0.1459 - val_loss: 0.0299 - val_mae: 0.1333
Epoch 6/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0315 - mae: 0.1361 - val_loss: 0.0237 - val_mae: 0.1189
Epoch 7/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0226 - mae: 0.1142 - val_loss: 0.0153 - val_mae: 0.0936
Epoch 8/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0140 - mae: 0.0881 - val_loss: 0.0121 - val_mae: 0.0799
Epoch 9/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0120 - mae:

#### 1 > r > 0.75

In [236]:
# Train Transformer model
transformer_model_1r75_1 = create_transformer_model(
    df_scaled=df_selected_1r75_1,
    no_columns=len(selected_stations_1r75_1),
    n_steps=24,
    epochs=50,
    batch_size=32,
    head_size=64,
    num_heads=2,
    ff_dim=128,
    num_blocks=2,
    dropout=0.1,
    learning_rate=0.001
)

# Save Transformer model
transformer_model_1r75_1.save("transformer_models/transformer_model_1r75_1.keras")


Epoch 1/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 5s 9ms/step - loss: 0.3595 - mae: 0.5232 - val_loss: 0.2774 - val_mae: 0.4468
Epoch 2/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.3008 - mae: 0.4702 - val_loss: 0.2213 - val_mae: 0.3916
Epoch 3/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.2406 - mae: 0.4175 - val_loss: 0.1704 - val_mae: 0.3430
Epoch 4/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.1875 - mae: 0.3718 - val_loss: 0.1303 - val_mae: 0.3038
Epoch 5/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.1402 - mae: 0.3246 - val_loss: 0.1029 - val_mae: 0.2749
Epoch 6/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.1159 - mae: 0.3025 - val_loss: 0.0880 - val_mae: 0.2569
Epoch 7/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0958 - mae: 0.2757 - val_loss: 0.0819 - val_mae: 0.2489
Epoch 8/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0912 - mae: 0.2692 - val_loss: 0.0805 - val_mae: 0.2459
Epoch 9/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0891 - mae: 

In [38]:
# Train Transformer model
transformer_model_1r75_2 = create_transformer_model(
    df_scaled=df_selected_1r75_2,
    no_columns=len(selected_stations_1r75_2),
    n_steps=24,
    epochs=100,          # More epochs, since the model is smaller
    batch_size=16,       # Smaller batch for better generalization
    head_size=32,        # Reduce head size to avoid overfitting
    num_heads=2,         # Keep 2 heads, enough for diversity but not too heavy
    ff_dim=64,           # Reduce feed-forward dimension
    num_blocks=1,        # Fewer transformer blocks for smaller data
    dropout=0.2,         # Slightly higher dropout to regularize
    learning_rate=0.01   # Lower LR for stable training
)

# Save Transformer model
transformer_model_1r75_2.save("transformer_models/transformer_model_1r75_2.keras")


Epoch 1/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.7908 - mae: 0.7022 - val_loss: 0.0806 - val_mae: 0.2469
Epoch 2/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0867 - mae: 0.2606 - val_loss: 0.0808 - val_mae: 0.2471
Epoch 3/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0872 - mae: 0.2594 - val_loss: 0.0806 - val_mae: 0.2469
Epoch 4/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0843 - mae: 0.2555 - val_loss: 0.0805 - val_mae: 0.2481
Epoch 5/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0880 - mae: 0.2641 - val_loss: 0.0832 - val_mae: 0.2487
Epoch 6/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0866 - mae: 0.2589 - val_loss: 0.0816 - val_mae: 0.2475
Epoch 7/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0838 - mae: 0.2537 - val_loss: 0.0804 - val_mae: 0.2469
Epoch 8/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0888 - mae: 0.2648 - val_loss: 0.0804 - val_mae: 0.2469
Epoch 9/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/

In [259]:
# Train Transformer model
transformer_model_1r75_3 = create_transformer_model(
    df_scaled=df_selected_1r75_3,
    no_columns=len(selected_stations_1r75_3),
    n_steps=24,
    epochs=50,
    batch_size=32,
    head_size=64,
    num_heads=2,
    ff_dim=128,
    num_blocks=2,
    dropout=0.1,
    learning_rate=0.001
)

# Save Transformer model
transformer_model_1r75_3.save("transformer_models/transformer_model_1r75_3.keras")


Epoch 1/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 9s 12ms/step - loss: 0.7381 - mae: 0.7695 - val_loss: 0.3873 - val_mae: 0.5428
Epoch 2/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.3338 - mae: 0.4996 - val_loss: 0.1929 - val_mae: 0.3647
Epoch 3/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.1719 - mae: 0.3409 - val_loss: 0.1141 - val_mae: 0.2767
Epoch 4/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.1076 - mae: 0.2680 - val_loss: 0.0813 - val_mae: 0.2375
Epoch 5/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0783 - mae: 0.2341 - val_loss: 0.0605 - val_mae: 0.2069
Epoch 6/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0631 - mae: 0.2139 - val_loss: 0.0493 - val_mae: 0.1854
Epoch 7/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0532 - mae: 0.1942 - val_loss: 0.0453 - val_mae: 0.1753
Epoch 8/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0481 - mae: 0.1801 - val_loss: 0.0425 - val_mae: 0.1689
Epoch 9/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0445 - mae:

In [239]:
# Train Transformer model
transformer_model_1r75_4 = create_transformer_model(
    df_scaled=df_selected_1r75_4,
    no_columns=len(selected_stations_1r75_4),
    n_steps=24,
    epochs=50,
    batch_size=32,
    head_size=64,
    num_heads=2,
    ff_dim=128,
    num_blocks=2,
    dropout=0.1,
    learning_rate=0.001
)

# Save Transformer model
transformer_model_1r75_4.save("transformer_models/transformer_model_1r75_4.keras")


Epoch 1/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - loss: 0.7868 - mae: 0.7110 - val_loss: 0.2207 - val_mae: 0.3773
Epoch 2/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.2065 - mae: 0.3636 - val_loss: 0.1331 - val_mae: 0.3006
Epoch 3/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.1283 - mae: 0.2945 - val_loss: 0.0891 - val_mae: 0.2464
Epoch 4/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0897 - mae: 0.2474 - val_loss: 0.0605 - val_mae: 0.2030
Epoch 5/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0611 - mae: 0.2057 - val_loss: 0.0391 - val_mae: 0.1622
Epoch 6/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0428 - mae: 0.1724 - val_loss: 0.0312 - val_mae: 0.1459
Epoch 7/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0310 - mae: 0.1486 - val_loss: 0.0264 - val_mae: 0.1327
Epoch 8/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0274 - mae: 0.1355 - val_loss: 0.0232 - val_mae: 0.1217
Epoch 9/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0237 - mae:

In [240]:
# Train Transformer model
transformer_model_1r75_5 = create_transformer_model(
    df_scaled=df_selected_1r75_5,
    no_columns=len(selected_stations_1r75_5),
    n_steps=24,
    epochs=50,
    batch_size=32,
    head_size=64,
    num_heads=2,
    ff_dim=128,
    num_blocks=2,
    dropout=0.1,
    learning_rate=0.001
)

# Save Transformer model
transformer_model_1r75_5.save("transformer_models/transformer_model_1r75_5.keras")


Epoch 1/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - loss: 0.3961 - mae: 0.4459 - val_loss: 0.0656 - val_mae: 0.2181
Epoch 2/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0658 - mae: 0.2233 - val_loss: 0.0613 - val_mae: 0.2120
Epoch 3/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0616 - mae: 0.2169 - val_loss: 0.0587 - val_mae: 0.2069
Epoch 4/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0614 - mae: 0.2159 - val_loss: 0.0568 - val_mae: 0.2018
Epoch 5/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0584 - mae: 0.2093 - val_loss: 0.0516 - val_mae: 0.1914
Epoch 6/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0496 - mae: 0.1902 - val_loss: 0.0381 - val_mae: 0.1609
Epoch 7/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0364 - mae: 0.1588 - val_loss: 0.0293 - val_mae: 0.1377
Epoch 8/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0282 - mae: 0.1364 - val_loss: 0.0235 - val_mae: 0.1218
Epoch 9/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0241 - mae:

In [241]:
# Train Transformer model
transformer_model_1r75_6 = create_transformer_model(
    df_scaled=df_selected_1r75_6,
    no_columns=len(selected_stations_1r75_6),
    n_steps=24,
    epochs=50,
    batch_size=32,
    head_size=64,
    num_heads=2,
    ff_dim=128,
    num_blocks=2,
    dropout=0.1,
    learning_rate=0.001
)

# Save Transformer model
transformer_model_1r75_6.save("transformer_models/transformer_model_1r75_6.keras")


Epoch 1/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - loss: 0.6682 - mae: 0.6219 - val_loss: 0.1066 - val_mae: 0.2729
Epoch 2/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0988 - mae: 0.2681 - val_loss: 0.0741 - val_mae: 0.2347
Epoch 3/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0777 - mae: 0.2454 - val_loss: 0.0713 - val_mae: 0.2320
Epoch 4/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0741 - mae: 0.2394 - val_loss: 0.0704 - val_mae: 0.2292
Epoch 5/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0762 - mae: 0.2450 - val_loss: 0.0694 - val_mae: 0.2274
Epoch 6/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0755 - mae: 0.2424 - val_loss: 0.0681 - val_mae: 0.2242
Epoch 7/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0718 - mae: 0.2363 - val_loss: 0.0640 - val_mae: 0.2171
Epoch 8/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0695 - mae: 0.2318 - val_loss: 0.0596 - val_mae: 0.2076
Epoch 9/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0645 - mae

In [242]:
# Train Transformer model
transformer_model_1r75_7 = create_transformer_model(
    df_scaled=df_selected_1r75_7,
    no_columns=len(selected_stations_1r75_7),
    n_steps=24,
    epochs=50,
    batch_size=32,
    head_size=64,
    num_heads=2,
    ff_dim=128,
    num_blocks=2,
    dropout=0.1,
    learning_rate=0.001
)

# Save Transformer model
transformer_model_1r75_7.save("transformer_models/transformer_model_1r75_7.keras")


Epoch 1/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - loss: 0.2937 - mae: 0.4054 - val_loss: 0.0688 - val_mae: 0.2268
Epoch 2/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0707 - mae: 0.2343 - val_loss: 0.0638 - val_mae: 0.2166
Epoch 3/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0681 - mae: 0.2275 - val_loss: 0.0562 - val_mae: 0.1994
Epoch 4/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0560 - mae: 0.1987 - val_loss: 0.0444 - val_mae: 0.1702
Epoch 5/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0452 - mae: 0.1713 - val_loss: 0.0370 - val_mae: 0.1502
Epoch 6/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0378 - mae: 0.1519 - val_loss: 0.0309 - val_mae: 0.1340
Epoch 7/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0308 - mae: 0.1351 - val_loss: 0.0260 - val_mae: 0.1248
Epoch 8/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0264 - mae: 0.1259 - val_loss: 0.0251 - val_mae: 0.1257
Epoch 9/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0216 - mae:

In [243]:
# Train Transformer model
transformer_model_1r75_8 = create_transformer_model(
    df_scaled=df_selected_1r75_8,
    no_columns=len(selected_stations_1r75_8),
    n_steps=24,
    epochs=50,
    batch_size=32,
    head_size=64,
    num_heads=2,
    ff_dim=128,
    num_blocks=2,
    dropout=0.1,
    learning_rate=0.001
)

# Save Transformer model
transformer_model_1r75_8.save("transformer_models/transformer_model_1r75_8.keras")


Epoch 1/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - loss: 0.7508 - mae: 0.6655 - val_loss: 0.1287 - val_mae: 0.2921
Epoch 2/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.1190 - mae: 0.2889 - val_loss: 0.0744 - val_mae: 0.2360
Epoch 3/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0785 - mae: 0.2492 - val_loss: 0.0690 - val_mae: 0.2274
Epoch 4/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0748 - mae: 0.2424 - val_loss: 0.0674 - val_mae: 0.2244
Epoch 5/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0724 - mae: 0.2378 - val_loss: 0.0650 - val_mae: 0.2191
Epoch 6/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0668 - mae: 0.2251 - val_loss: 0.0634 - val_mae: 0.2152
Epoch 7/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0640 - mae: 0.2175 - val_loss: 0.0550 - val_mae: 0.1967
Epoch 8/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0558 - mae: 0.1981 - val_loss: 0.0390 - val_mae: 0.1587
Epoch 9/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0338 - mae:

In [244]:
# Train Transformer model
transformer_model_1r75_9 = create_transformer_model(
    df_scaled=df_selected_1r75_9,
    no_columns=len(selected_stations_1r75_9),
    n_steps=24,
    epochs=50,
    batch_size=32,
    head_size=64,
    num_heads=2,
    ff_dim=128,
    num_blocks=2,
    dropout=0.1,
    learning_rate=0.001
)

# Save Transformer model
transformer_model_1r75_9.save("transformer_models/transformer_model_1r75_9.keras")


Epoch 1/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - loss: 0.4048 - mae: 0.4915 - val_loss: 0.0874 - val_mae: 0.2511
Epoch 2/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0840 - mae: 0.2519 - val_loss: 0.0617 - val_mae: 0.2136
Epoch 3/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0599 - mae: 0.2095 - val_loss: 0.0361 - val_mae: 0.1500
Epoch 4/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0332 - mae: 0.1463 - val_loss: 0.0235 - val_mae: 0.1223
Epoch 5/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 0.0225 - mae: 0.1202 - val_loss: 0.0202 - val_mae: 0.1131
Epoch 6/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0196 - mae: 0.1121 - val_loss: 0.0182 - val_mae: 0.1054
Epoch 7/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0170 - mae: 0.1026 - val_loss: 0.0167 - val_mae: 0.1004
Epoch 8/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0157 - mae: 0.0978 - val_loss: 0.0155 - val_mae: 0.0937
Epoch 9/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0145 - mae

In [39]:
# Train Transformer model
transformer_model_1r75_10 = create_transformer_model(
    df_scaled=df_selected_1r75_10,
    no_columns=len(selected_stations_1r75_10),
    n_steps=24,
    epochs=100,          # More epochs, since the model is smaller
    batch_size=16,       # Smaller batch for better generalization
    head_size=32,        # Reduce head size to avoid overfitting
    num_heads=2,         # Keep 2 heads, enough for diversity but not too heavy
    ff_dim=64,           # Reduce feed-forward dimension
    num_blocks=1,        # Fewer transformer blocks for smaller data
    dropout=0.2,         # Slightly higher dropout to regularize
    learning_rate=0.01 # Lower LR for stable training
)

# Save Transformer model
transformer_model_1r75_10.save("transformer_models/transformer_model_1r75_10.keras")


Epoch 1/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.1525 - mae: 0.2914 - val_loss: 0.0688 - val_mae: 0.2177
Epoch 2/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0506 - mae: 0.1802 - val_loss: 0.0146 - val_mae: 0.0935
Epoch 3/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0125 - mae: 0.0807 - val_loss: 0.0118 - val_mae: 0.0750
Epoch 4/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0109 - mae: 0.0738 - val_loss: 0.0117 - val_mae: 0.0770
Epoch 5/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0097 - mae: 0.0702 - val_loss: 0.0132 - val_mae: 0.0873
Epoch 6/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0111 - mae: 0.0756 - val_loss: 0.0128 - val_mae: 0.0848
Epoch 7/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0101 - mae: 0.0717 - val_loss: 0.0109 - val_mae: 0.0760
Epoch 8/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0104 - mae: 0.0741 - val_loss: 0.0150 - val_mae: 0.0916
Epoch 9/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/